In [ ]:
# =============================================================================
# FINAL INTEGRATED EDA — PRODUCTION-READY V5.1 (COMPLETE REVISION)
# ÖZELLİKLER:
#   ✅ Data Leakage Tamamen Temizlendi + Temporal Features Eklendi
#   ✅ matches_all Loading Problemi Çözüldü (Phase 1 Eklendi)
#   ✅ Multicollinearity Detection & Auto Drop List
#   ✅ SMOTE + VIF + t-SNE + Feature Engineering Visualizations
#
# YENİ (V5.1):
#   - Phase 1: CSV Loading + Data Merging + Verification
#   - matches_all NameError çözüldü
#   - Comprehensive error handling
#   - 15+ validation checkpoints
# =============================================================================

import os
import re
import gc
import glob
import warnings
import random
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from pandas.api.types import is_numeric_dtype, is_categorical_dtype, is_object_dtype
from scipy.stats import chi2_contingency, zscore
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import mutual_info_classif
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
import plotly.offline as py
import textwrap
from scipy import stats
from scipy.stats import skew
!pip install fuzzywuzzy
!pip install unidecode python-levenshtein
# 🆕 YENİ EKLEMELER
from fuzzywuzzy import fuzz
from unidecode import unidecode

# ════════════════════════════════════════════════════════════════════════
# 🆕 V5.1 İÇİN YENİ KÜTÜPHANELER (Kontrollü Import)
# ════════════════════════════════════════════════════════════════════════

# SMOTE için
try:
    from imblearn.over_sampling import SMOTE
    SMOTE_AVAILABLE = True
    print("[OK] imbalanced-learn (SMOTE) available.")
except ImportError:
    SMOTE_AVAILABLE = False
    print("[WARN] imbalanced-learn not installed. SMOTE analysis will be skipped.")
    print("       Install with: pip install imbalanced-learn")

# VIF için
try:
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    VIF_AVAILABLE = True
    print("[OK] statsmodels VIF available.")
except ImportError:
    VIF_AVAILABLE = False
    print("[WARN] statsmodels VIF not available. VIF analysis will be skipped.")

# Silhouette Score için (t-SNE)
try:
    from sklearn.metrics import silhouette_score
    SILHOUETTE_AVAILABLE = True
    print("[OK] silhouette_score available.")
except ImportError:
    SILHOUETTE_AVAILABLE = False
    print("[WARN] silhouette_score not available.")

try:
    py.init_notebook_mode(connected=True)
except ImportError:
    print("[WARN] Plotly 'init_notebook_mode' failed. (Colab dışında normaldir)")

warnings.filterwarnings("ignore")

# =======================
# TIMING & LOGGING
# =======================
SCRIPT_START = time.time()
print(f"[INFO] EDA Script V5.1 (COMPLETE REVISION) started at {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 100)
print("🚀 MAJOR UPDATES IN V5.1:")
print("   ✅ FIXED: matches_all loading (Phase 1 added)")
print("   ✅ ALL match-event features converted to temporal averages")
print("   ✅ Data leakage completely eliminated")
print("   ✅ High correlation pairs automatically handled")
print("   ✅ SMOTE + VIF + t-SNE + Feature Engineering visualizations")
print("=" * 100 + "\n")

# =======================
# REPRODUCIBILITY & PUBLICATION STYLE
# =======================
RNG_SEED = 42
np.random.seed(RNG_SEED)
random.seed(RNG_SEED)

# ════════════════════════════════════════════════════════════════════════
# 🎨 THESIS-COMPLIANT VISUALIZATION SETTINGS (V5.2)
# ════════════════════════════════════════════════════════════════════════
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.backends.backend_pdf import PdfPages

# ────────────────────────────────────────────────────────────────────────
# 1. THESIS COLOR PALETTE (Mavi Tonları - Akademik Standart)
# ────────────────────────────────────────────────────────────────────────
THESIS_COLORS = {
    'primary': {
        'dark_blue': '#1f3a93',      # Ana mavi (en koyu) - Başlıklar, vurgu
        'medium_blue': '#2e59d9',    # Orta mavi - Ana veriler
        'light_blue': '#6c8cd5',     # Açık mavi - İkincil veriler
        'sky_blue': '#a4c2f4',       # Çok açık mavi - Arka plan, dolgu
    },
    'accent': {
        'highlight': '#ffa500',      # Turuncu (vurgu için)
        'warning': '#ff6b6b',        # Açık kırmızı (uyarı)
        'success': '#27ae60',        # Yeşil (başarı)
    },
    'neutral': {
        'dark_gray': '#2c3e50',      # Koyu gri (text)
        'medium_gray': '#7f8c8d',    # Orta gri
        'light_gray': '#ecf0f1',     # Açık gri (background)
    },
    # Geriye uyumluluk için eski isimler
    'main_blue': '#1f3a93',
    'light_blue': '#6c8cd5',
    'home_green': '#27ae60',
    'away_red': '#ff6b6b',
    'draw_orange': '#ffa500',
    'gray': '#7f8c8d'
}

# Kısa erişim için alias
TC = THESIS_COLORS

# ════════════════════════════════════════════════════════════════════════
# 🔄 BACKWARD COMPATIBILITY - Eski kod için UNIFIED_COLORS
# ════════════════════════════════════════════════════════════════════════

UNIFIED_COLORS = {
    'main_blue': TC['primary']['dark_blue'],      # '#1f3a93'
    'light_blue': TC['primary']['light_blue'],    # '#6c8cd5'
    'home_green': TC['accent']['success'],        # '#27ae60'
    'away_red': TC['accent']['warning'],          # '#ff6b6b'
    'draw_orange': TC['accent']['highlight'],     # '#ffa500'
    'gray': TC['neutral']['medium_gray'],         # '#7f8c8d'
}

# ────────────────────────────────────────────────────────────────────────
# 2. MATCH OUTCOME COLORS (Futbol Analizi için)
# ────────────────────────────────────────────────────────────────────────
OUTCOME_COLORS = {
    0: TC['accent']['highlight'],    # Draw - Turuncu
    1: TC['primary']['medium_blue'], # Home Win - Mavi
    2: TC['accent']['warning'],      # Away Win - Kırmızı
}
OUTCOME_NAMES = {0: 'Draw', 1: 'Home Win', 2: 'Away Win'}

# ────────────────────────────────────────────────────────────────────────
# 3. CUSTOM COLORMAPS (Heatmaps için)
# ────────────────────────────────────────────────────────────────────────
# Mavi tonları (correlation, feature importance)
CMAP_BLUES = LinearSegmentedColormap.from_list(
    'thesis_blues',
    [TC['neutral']['light_gray'], TC['primary']['sky_blue'],
     TC['primary']['light_blue'], TC['primary']['medium_blue'],
     TC['primary']['dark_blue']],
    N=256
)

# Diverging (correlation matrix - negatif/pozitif)
CMAP_DIVERGING = LinearSegmentedColormap.from_list(
    'thesis_diverging',
    [TC['accent']['warning'], TC['neutral']['light_gray'], TC['primary']['dark_blue']],
    N=256
)

# Performance colormap (kötü → iyi)
CMAP_PERFORMANCE = LinearSegmentedColormap.from_list(
    'thesis_performance',
    [TC['accent']['warning'], TC['accent']['highlight'],
     TC['primary']['light_blue'], TC['primary']['dark_blue']],
    N=256
)

# Matplotlib'e kaydet
mpl.colormaps.register(CMAP_BLUES)
mpl.colormaps.register(CMAP_DIVERGING)
mpl.colormaps.register(CMAP_PERFORMANCE)

# ────────────────────────────────────────────────────────────────────────
# 4. GLOBAL MATPLOTLIB SETTINGS (Yüksek Kalite PDF Çıktısı)
# ────────────────────────────────────────────────────────────────────────
plt.rcParams.update({
    # Font ayarları
    "font.family": "serif",
    "font.serif": ["Times New Roman", "DejaVu Serif", "Liberation Serif"],
    "font.size": 12,
    "font.weight": "normal",

    # Eksen ayarları
    "axes.labelsize": 12,
    "axes.titlesize": 14,
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.linewidth": 1.5,
    "axes.edgecolor": TC['neutral']['dark_gray'],
    "axes.facecolor": "white",
    "axes.grid": True,
    "axes.axisbelow": True,

    # Tick ayarları
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "xtick.major.width": 1.2,
    "ytick.major.width": 1.2,

    # Grid ayarları
    "grid.alpha": 0.4,
    "grid.linestyle": "--",
    "grid.linewidth": 0.8,
    "grid.color": TC['neutral']['medium_gray'],

    # Çizgi ayarları
    "lines.linewidth": 2.5,
    "lines.markersize": 8,

    # Legend ayarları
    "legend.fontsize": 10,
    "legend.framealpha": 0.9,
    "legend.edgecolor": TC['neutral']['medium_gray'],

    # Figure ayarları
    "figure.facecolor": "white",
    "figure.edgecolor": "white",
    "figure.autolayout": True,
    "figure.dpi": 100,

    # Kaydetme ayarları (KRİTİK!)
    "savefig.dpi": 600,
    "savefig.format": "pdf",
    "savefig.facecolor": "white",
    "savefig.edgecolor": "white",
    "savefig.bbox": "tight",
    "savefig.pad_inches": 0.1,

    # PDF vektörel çıktı için
    "pdf.fonttype": 42,
    "ps.fonttype": 42,

    # LaTeX desteği (opsiyonel)
    "text.usetex": False,
    "mathtext.fontset": "dejavuserif",
})

# Seaborn tema ayarları
sns.set_style("whitegrid", {
    "axes.facecolor": "white",
    "grid.color": TC['neutral']['medium_gray'],
    "grid.linestyle": "--",
})

# Seaborn palette (thesis renkleriyle)
sns.set_palette([
    TC['primary']['dark_blue'],
    TC['primary']['medium_blue'],
    TC['primary']['light_blue'],
    TC['accent']['highlight'],
    TC['accent']['warning'],
])

print("✅ Thesis-compliant visualization settings loaded (V5.2)")
print(f"   • Color palette: Mavi tonları (akademik standart)")
print(f"   • Output format: PDF (vektörel) + PNG (preview)")
print(f"   • DPI: 600 (yayın kalitesi)")

# =======================
# GOOGLE DRIVE SETUP
# =======================
print("[INFO] Mounting Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive', timeout_ms=120000)
    print("[OK] Drive mounted successfully")
    BASE_PATH = "/content/drive/My Drive/Thesis Data/"
except ImportError:
    print("[INFO] Not in Colab environment, assuming local BASE_PATH='./Thesis Data/'")
    BASE_PATH = "./Thesis Data/"
    os.makedirs(BASE_PATH, exist_ok=True)
except Exception as e:
    print(f"[ERROR] Drive mount failed: {e}. Assuming local BASE_PATH='./Thesis Data/'")
    BASE_PATH = "./Thesis Data/"
    os.makedirs(BASE_PATH, exist_ok=True)

# =======================
# CONFIG & PATHS & OUTPUT DIRS
# =======================
OUTPUT_BASE_PATH = os.path.join(BASE_PATH, "EDA_Outputs_27.11.25 V1")
OUT_DIR = OUTPUT_BASE_PATH

try:
    os.makedirs(OUT_DIR, exist_ok=True)
    COMPACT_DIR = os.path.join(OUT_DIR, "compact_report")
    os.makedirs(COMPACT_DIR, exist_ok=True)
    IMAGES_DIR = os.path.join(OUT_DIR, "Images")
    os.makedirs(IMAGES_DIR, exist_ok=True)
    CSV_DIR = os.path.join(OUT_DIR, "Data_Exports")
    os.makedirs(CSV_DIR, exist_ok=True)
    print(f"[OK] Output directories validated/created: {OUT_DIR}\n")
except OSError as e:
    print(f"[ERROR] Cannot create output directories: {e}")
    OUT_DIR = os.getcwd()
    COMPACT_DIR = os.path.join(OUT_DIR, "compact_report")
    os.makedirs(COMPACT_DIR, exist_ok=True)
    IMAGES_DIR = os.path.join(OUT_DIR, "Images")
    os.makedirs(IMAGES_DIR, exist_ok=True)
    CSV_DIR = os.path.join(OUT_DIR, "Data_Exports")
    os.makedirs(CSV_DIR, exist_ok=True)
    print(f"[WARN] Using current working directory for outputs: {OUT_DIR}\n")

# PDF Raporlama için ayarlar
_pdf = PdfPages(os.path.join(COMPACT_DIR, "EDA_Outputs_24.11.25 V1.pdf"))
_pdf_page_count = 0
_pdf_part = 1
PAGES_PER_PDF = 100

# =======================
# UTILITY FUNCTIONS
# =======================

# ════════════════════════════════════════════════════════════════════════
# 📐 THESIS-COMPLIANT SAVE FUNCTIONS (V5.2)
# ════════════════════════════════════════════════════════════════════════

def savefig_report(page_name_hint=None, also_png=True, also_pdf_standalone=True):
    """
    Enhanced save function - saves to report PDF + standalone PNG + standalone PDF.

    V5.2 Updates:
        - Always saves both PNG and standalone PDF (vektörel)
        - Thesis-compliant metadata
        - High DPI (600) for publication quality
        - White background enforced

    Args:
        page_name_hint: Filename hint (without extension)
        also_png: Save standalone PNG (default: True)
        also_pdf_standalone: Save standalone PDF (default: True)

    Returns:
        tuple: (png_path, pdf_path) or None if failed
    """
    global _pdf, _pdf_page_count, _pdf_part, PAGES_PER_PDF, COMPACT_DIR, IMAGES_DIR

    current_fig = plt.gcf()

    # Boş figür kontrolü
    if not current_fig.get_axes():
        print(f"[WARN] Attempted to save an empty figure (Hint: {page_name_hint}). Skipping.")
        plt.close(current_fig)
        return None, None

    # Tight layout uygula
    try:
        current_fig.tight_layout()
    except Exception as e_layout:
        print(f"[WARN] tight_layout failed for '{page_name_hint}': {e_layout}")

    # Kaydetme parametreleri (thesis-compliant)
    save_params_common = {
        'bbox_inches': 'tight',
        'facecolor': 'white',
        'edgecolor': 'none',
        'pad_inches': 0.1,
    }

    # PDF metadata
    pdf_metadata = {
        'Title': page_name_hint or 'EDA Figure',
        'Author': 'Dokumus - Tilburg University',
        'Subject': 'Football Match Prediction - EDA Analysis',
        'Keywords': 'Machine Learning, XAI, Football Prediction, LTCN',
        'Creator': 'Matplotlib / EDA V5.2',
        'Producer': 'Thesis Research - Data Science & Society'
    }

    png_path = None
    pdf_path = None

    def _safe_savefig(path, format_type='pdf', dpi=600):
        """Internal save function with error handling."""
        try:
            if format_type == 'report_pdf':
                # Ana PDF raporuna ekle
                _pdf.savefig(current_fig, **save_params_common)
            elif format_type == 'png':
                os.makedirs(os.path.dirname(path), exist_ok=True)
                current_fig.savefig(
                    path,
                    format='png',
                    dpi=dpi,
                    **save_params_common,
                    pil_kwargs={"optimize": True, "compress_level": 6}
                )
            elif format_type == 'pdf':
                os.makedirs(os.path.dirname(path), exist_ok=True)
                current_fig.savefig(
                    path,
                    format='pdf',
                    dpi=600,  # PDF için 300 yeterli (vektörel)
                    metadata=pdf_metadata,
                    **save_params_common
                )
            return True
        except Exception as e_save:
            print(f"[ERROR] Failed to save figure ({format_type}): {e_save}")
            return False

    try:
        # ─────────────────────────────────────────────────────────────────
        # 1. ANA PDF RAPORUNA EKLE
        # ─────────────────────────────────────────────────────────────────
        if _safe_savefig(None, format_type='report_pdf'):
            _pdf_page_count += 1

        # ─────────────────────────────────────────────────────────────────
        # 2. STANDALONE DOSYALAR (PNG + PDF)
        # ─────────────────────────────────────────────────────────────────
        if page_name_hint:
            # Güvenli dosya adı oluştur
            safe_hint = re.sub(r'[\\/*?:"<>|]+', "_", page_name_hint)
            safe_hint = safe_hint.replace('.png', '').replace('.pdf', '')

            base_path = os.path.join(IMAGES_DIR, safe_hint)

            # PNG kaydet (preview ve web için)
            if also_png:
                png_path = f"{base_path}.png"
                if not _safe_savefig(png_path, format_type='png', dpi=600):
                    png_path = None

            # Standalone PDF kaydet (thesis için - vektörel!)
            if also_pdf_standalone:
                pdf_path = f"{base_path}.pdf"
                if not _safe_savefig(pdf_path, format_type='pdf'):
                    pdf_path = None

        # ─────────────────────────────────────────────────────────────────
        # 3. PDF SAYFA KONTROLÜ (Çok sayfalı raporlar için)
        # ─────────────────────────────────────────────────────────────────
        if _pdf_page_count >= PAGES_PER_PDF:
            print(f"[INFO] Closing PDF part {_pdf_part} with {_pdf_page_count} pages...")
            _pdf.close()
            _pdf_part += 1
            new_pdf_path = os.path.join(COMPACT_DIR, f"EDA_Report_V5_2_part{_pdf_part:02d}.pdf")
            _pdf = PdfPages(new_pdf_path)
            _pdf_page_count = 0
            print(f"[INFO] Starting new PDF part: {os.path.basename(new_pdf_path)}")

    except Exception as e_report:
        print(f"[ERROR] Error occurred in savefig_report for '{page_name_hint}': {e_report}")

    finally:
        plt.close(current_fig)

    return png_path, pdf_path


def save_df_as_image(df_to_save, filename_hint, title=""):
    """
    Saves a pandas DataFrame as a styled image table (PNG + PDF).

    V5.2 Updates:
        - Thesis-compliant color scheme (mavi tonları)
        - Both PNG and PDF output
        - Professional table styling
        - Better readability

    Args:
        df_to_save: pandas DataFrame to save
        filename_hint: Filename hint (without extension)
        title: Optional title for the table
    """
    if df_to_save is None or df_to_save.empty:
        print(f"[WARN] DataFrame to save as image '{filename_hint}' is empty. Skipping.")
        return

    # ─────────────────────────────────────────────────────────────────────
    # THESIS COLORS (Tablo için)
    # ─────────────────────────────────────────────────────────────────────
    TABLE_COLORS = {
        'header_bg': '#1f3a93',           # Koyu mavi (header)
        'header_text': 'white',           # Beyaz text
        'row_label_bg': '#a4c2f4',        # Açık mavi (row labels)
        'row_label_text': '#2c3e50',      # Koyu gri text
        'cell_even': '#f8f9fa',           # Çok açık gri (çift satırlar)
        'cell_odd': 'white',              # Beyaz (tek satırlar)
        'cell_text': '#2c3e50',           # Koyu gri text
        'border': '#d1d5db',              # Açık gri border
        'title': '#1f3a93',               # Koyu mavi başlık
    }

    # ─────────────────────────────────────────────────────────────────────
    # FIGÜR BOYUTU HESAPLAMA
    # ─────────────────────────────────────────────────────────────────────
    nrows, ncols = df_to_save.shape

    # Dinamik boyutlandırma
    fig_w = max(10, 1.4 * ncols + 3)
    fig_h = max(5, 0.35 * nrows + 2.5)

    # Maksimum sınırlar
    fig_w = min(fig_w, 45)
    fig_h = min(fig_h, 55)

    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.axis('off')

    # Beyaz arka plan
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # ─────────────────────────────────────────────────────────────────────
    # SÜTUN ETİKETLERİ
    # ─────────────────────────────────────────────────────────────────────
    if isinstance(df_to_save.columns, pd.MultiIndex):
        col_labels = ['_'.join(map(str, col)).strip() for col in df_to_save.columns.values]
    else:
        col_labels = [str(c) for c in df_to_save.columns]

    # Uzun sütun isimlerini wrap et
    WRAP_WIDTH = 18
    wrapped_col_labels = []
    for c in col_labels:
        if len(c) > WRAP_WIDTH:
            wrapped_col_labels.append(textwrap.fill(c, width=WRAP_WIDTH))
        else:
            wrapped_col_labels.append(c)
    col_labels = wrapped_col_labels

    # ─────────────────────────────────────────────────────────────────────
    # SATIR ETİKETLERİ
    # ─────────────────────────────────────────────────────────────────────
    if isinstance(df_to_save.index, pd.MultiIndex):
        row_labels = ['_'.join(map(str, idx)).strip() for idx in df_to_save.index.values]
    else:
        row_labels = [str(i) for i in df_to_save.index]

    # Hücre içeriği
    cell_text = df_to_save.fillna('—').astype(str).values  # NaN yerine em-dash

    try:
        # ─────────────────────────────────────────────────────────────────
        # TABLO OLUŞTUR
        # ─────────────────────────────────────────────────────────────────
        table = ax.table(
            cellText=cell_text,
            colLabels=col_labels,
            rowLabels=row_labels,
            cellLoc='center',
            loc='center',
        )

        # Font boyutu (dinamik)
        base_fs = 10
        if ncols > 20 or nrows > 50:
            base_fs = 7
        elif ncols > 15 or nrows > 35:
            base_fs = 8
        elif ncols > 10 or nrows > 20:
            base_fs = 9

        table.auto_set_font_size(False)
        table.set_fontsize(base_fs)
        table.scale(1.2, 1.5)

        # ─────────────────────────────────────────────────────────────────
        # HÜCRE STİLLERİ (Thesis-Compliant)
        # ─────────────────────────────────────────────────────────────────
        for (row_idx, col_idx), cell in table.get_celld().items():
            cell.set_edgecolor(TABLE_COLORS['border'])
            cell.set_linewidth(0.8)

            if row_idx == 0:
                # HEADER ROW (Koyu mavi arka plan, beyaz text)
                cell.set_text_props(
                    weight='bold',
                    color=TABLE_COLORS['header_text'],
                    fontsize=base_fs
                )
                cell.set_facecolor(TABLE_COLORS['header_bg'])
                cell.set_height(cell.get_height() * 1.3)

            elif col_idx == -1:
                # ROW LABELS (Açık mavi arka plan)
                cell.set_text_props(
                    weight='bold',
                    color=TABLE_COLORS['row_label_text'],
                    fontsize=base_fs - 1
                )
                cell.set_facecolor(TABLE_COLORS['row_label_bg'])

            else:
                # DATA CELLS (Alternatif renklendirme)
                cell.set_text_props(
                    color=TABLE_COLORS['cell_text'],
                    fontsize=base_fs - 1
                )
                if (row_idx - 1) % 2 == 0:
                    cell.set_facecolor(TABLE_COLORS['cell_even'])
                else:
                    cell.set_facecolor(TABLE_COLORS['cell_odd'])

        # ─────────────────────────────────────────────────────────────────
        # BAŞLIK
        # ─────────────────────────────────────────────────────────────────
        if title:
            fig.suptitle(
                title,
                fontsize=14,
                fontweight='bold',
                color=TABLE_COLORS['title'],
                y=0.98
            )

        plt.subplots_adjust(left=0.05, right=0.95, top=0.92, bottom=0.05)

        # ─────────────────────────────────────────────────────────────────
        # KAYDET (PNG + PDF)
        # ─────────────────────────────────────────────────────────────────
        savefig_report(filename_hint, also_png=True, also_pdf_standalone=True)

        print(f"[OK] DataFrame table saved: {filename_hint} (PNG + PDF)")

    except Exception as e_table:
        print(f"[ERROR] Failed to save DataFrame as image ({filename_hint}): {e_table}")
        plt.close(fig)


def save_figure_dual_format(fig, base_path, dpi=600, close_fig=True):
    """
    Standalone function to save any figure in both PNG and PDF formats.

    Use this for custom figures that don't go through savefig_report.

    Args:
        fig: matplotlib figure object
        base_path: Path without extension (e.g., '/path/to/figure_01')
        dpi: Resolution for PNG (PDF is always vector)
        close_fig: Whether to close figure after saving

    Returns:
        tuple: (png_path, pdf_path)

    Example:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(x, y)
        png_path, pdf_path = save_figure_dual_format(fig, '/output/my_plot')
    """

    save_params = {
        'bbox_inches': 'tight',
        'facecolor': 'white',
        'edgecolor': 'none',
        'pad_inches': 0.1,
    }

    # Dizin oluştur
    os.makedirs(os.path.dirname(base_path), exist_ok=True)

    png_path = None
    pdf_path = None

    # PNG kaydet
    try:
        png_path = f"{base_path}.png"
        fig.savefig(png_path, dpi=dpi, format='png', **save_params)
    except Exception as e:
        print(f"[WARN] PNG save failed for {base_path}: {e}")
        png_path = None

    # PDF kaydet (vektörel)
    try:
        pdf_path = f"{base_path}.pdf"
        pdf_metadata = {
            'Title': os.path.basename(base_path),
            'Author': 'Dokumus - Tilburg University',
            'Subject': 'Football Match Prediction - EDA',
            'Creator': 'Matplotlib / EDA V5.2',
        }
        fig.savefig(pdf_path, format='pdf', dpi=600,
                   metadata=pdf_metadata, **save_params)
    except Exception as e:
        print(f"[WARN] PDF save failed for {base_path}: {e}")
        pdf_path = None

    if close_fig:
        plt.close(fig)

    return png_path, pdf_path


# =============================================================================
# 🆕 PHASE 1: DATA LOADING & PREPROCESSING (COMPLETE REVISION)
# =============================================================================
print("\n[INFO] PHASE 1: LOADING AND PREPROCESSING DATA")
print("="*90)

# =============================================================================
# 🆕 PHASE 1.0: FUZZY MATCHING SETUP
# =============================================================================
print("\n[INFO] PHASE 1.0: FUZZY MATCHING & DATA INTEGRATION SETUP")
print("="*90)

class DataIntegration:
    """Advanced data integration with fuzzy matching"""

    def __init__(self, base_path):
        self.base_path = base_path
        self.df_matches = None
        self.df_elo = None
        self.df_tm = None
        self.team_mapping = {}
        self.team_scores = {}

        # ════════════════════════════════════════════════════════════════════
        # 🔥 MODEL KODUNDAN: NORMALIZATION HELPER (AYNI KALIYOR)
        # ════════════════════════════════════════════════════════════════════
        def _helper_normalize_for_key(s):
            """MODEL KODUNDAN - %97 EŞLEŞMENİN ANAHTARI!"""
            if pd.isna(s):
                return None
            s = str(s).strip()
            s = unidecode(s)

            suffixes = [
                ' FC', ' fc', ' Fc', ' fC', ' SC', ' sc', ' Sc',
                ' SK', ' sk', ' Sk', ' FK', ' fk', ' Fk',
                ' AC', ' ac', ' Ac', ' AS', ' as', ' As',
                ' SS', ' ss', ' Ss', ' CF', ' cf', ' Cf',
                ' HSC', ' hsc', ' Hsc', ' BC', ' bc', ' Bc',
                ' SSC', ' ssc', ' Ssc', ' US', ' us', ' Us',
                ' CA', ' ca', ' Ca', ' UD', ' ud', ' Ud',
                ' 1909', ' 1846', ' 1860', ' 2010', ' 1913',
                ' 1907', ' 1936', ' 1919', ' 2025',
                ' spor', ' kulübü', ' klub',
            ]

            for suffix in suffixes:
                s = s.replace(suffix, '')

            s = s.replace("'", "").replace("(", "").replace(")", "")
            words = sorted(s.lower().split())
            s = ' '.join(words)
            s = re.sub(r'\s+', ' ', s).strip()

            return s if s else None

        # ════════════════════════════════════════════════════════════════════
        # 🔥 DEĞİŞİKLİK 1: CSV YÜKLEME - MODEL KODUNDAN APPLY YAKLAŞIMI
        # ════════════════════════════════════════════════════════════════════
        self.MANUAL_OVERRIDES = {}
        team_list_csv = os.path.join(base_path, "Takim_Listesi_Temiz_1.csv")

        try:
            print(f"  📄 Loading manual overrides from CSV...")

            df_temp = None
            for encoding in ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']:
                try:
                    df_temp = pd.read_csv(
                        team_list_csv,
                        header=None,
                        skiprows=1,
                        quotechar='"',
                        sep='|',
                        encoding=encoding
                    )
                    print(f"     ✓ CSV read with encoding: {encoding}")
                    break
                except UnicodeDecodeError:
                    continue
                except FileNotFoundError:
                    raise

            if df_temp is None:
                raise Exception("CSV could not be read with any encoding!")

            df_team_list = df_temp[0].str.split('\t', expand=True)
            df_team_list.columns = ['Matches', 'Transfermarkt']

            # 🔥 YENİ: MODEL KODUNDAN - _process_team_row FONKSIYONU
            def _process_team_row(row):
                """MODEL KODUNDAN: Her iki tarafı da normalize et"""
                matches_name = row['Matches']
                tm_name = row['Transfermarkt']

                if pd.notna(matches_name) and pd.notna(tm_name):
                    # 🔥 KRİTİK: İKİ TARAF DA NORMALIZE EDİLİYOR
                    normalized_key = _helper_normalize_for_key(matches_name)
                    normalized_value = _helper_normalize_for_key(tm_name)

                    if normalized_key and normalized_value:
                        return normalized_key, normalized_value
                return None

            # 🔥 YENİ: APPLY İLE TÜM SATIRLARI İŞLE (MODEL KODUNDAN)
            for result in df_team_list.apply(_process_team_row, axis=1):
                if result is not None:
                    key, value = result
                    self.MANUAL_OVERRIDES[key] = value

            print(f"     ✓ Loaded {len(self.MANUAL_OVERRIDES)} overrides from CSV")

        except FileNotFoundError:
            print(f"     ⚠️  CSV not found: {team_list_csv}")
            print("        Using hardcoded overrides only...")
        except Exception as e:
            print(f"     ⚠️  Error loading CSV: {e}")
            print("        Using hardcoded overrides only...")

        # ════════════════════════════════════════════════════════════════════
        # 🔥 DEĞİŞİKLİK 2: HARDCODED OVERRIDES - MODEL KODUNDAN TAM MANTIK
        # ════════════════════════════════════════════════════════════════════
        print(f"  🔧 Applying hardcoded critical overrides...")

        # 🔥 YENİ: ÖNCE TRANSFERMARKT TARGET İSİMLERİNİ NORMALIZE ET
        tm_ac_ajaccio_normalized = _helper_normalize_for_key("AC Ajaccio")
        tm_gfc_ajaccio_normalized = _helper_normalize_for_key("GFC Ajaccio")
        tm_cordoba_normalized = _helper_normalize_for_key("Córdoba CF")
        tm_eibar_normalized = _helper_normalize_for_key("SD Eibar")
        tm_mallorca_normalized = _helper_normalize_for_key("RCD Mallorca")
        tm_celta_normalized = _helper_normalize_for_key("RC Celta de Vigo")
        tm_betis_normalized = _helper_normalize_for_key("Real Betis")
        tm_sociedad_normalized = _helper_normalize_for_key("Real Sociedad")
        tm_alaves_normalized = _helper_normalize_for_key("Deportivo Alavés")
        tm_cadiz_normalized = _helper_normalize_for_key("Cádiz CF")
        tm_getafe_normalized = _helper_normalize_for_key("Getafe CF")

        # 🔥 YENİ: SONRA MATCHES SOURCE İSİMLERİNİ NORMALIZE ET VE OVERRIDE'LARI OLUŞTUR

        # Ajaccio → AC Ajaccio
        key_aja = _helper_normalize_for_key("Ajaccio")
        if key_aja and tm_ac_ajaccio_normalized:
            self.MANUAL_OVERRIDES.pop(key_aja, None)  # Eski override'ı sil
            self.MANUAL_OVERRIDES[key_aja] = tm_ac_ajaccio_normalized
            print(f"     • Override: '{key_aja}' → '{tm_ac_ajaccio_normalized}' (AC Ajaccio)")

        # Ajaccio GFCO → GFC Ajaccio
        key_gfco = _helper_normalize_for_key("Ajaccio GFCO")
        if key_gfco and tm_gfc_ajaccio_normalized:
            self.MANUAL_OVERRIDES[key_gfco] = tm_gfc_ajaccio_normalized
            print(f"     • Override: '{key_gfco}' → '{tm_gfc_ajaccio_normalized}' (GFC Ajaccio)")

        # Cordoba → Córdoba CF
        key_cor = _helper_normalize_for_key("Cordoba")
        if key_cor and tm_cordoba_normalized:
            self.MANUAL_OVERRIDES[key_cor] = tm_cordoba_normalized
            print(f"     • Override: '{key_cor}' → '{tm_cordoba_normalized}' (Córdoba CF)")

        # Eibar → SD Eibar
        key_eibar = _helper_normalize_for_key("Eibar")
        if key_eibar and tm_eibar_normalized:
            self.MANUAL_OVERRIDES[key_eibar] = tm_eibar_normalized
            print(f"     • Override: '{key_eibar}' → '{tm_eibar_normalized}' (SD Eibar)")

        # Mallorca → RCD Mallorca
        key_mallorca = _helper_normalize_for_key("Mallorca")
        if key_mallorca and tm_mallorca_normalized:
            self.MANUAL_OVERRIDES[key_mallorca] = tm_mallorca_normalized
            print(f"     • Override: '{key_mallorca}' → '{tm_mallorca_normalized}' (RCD Mallorca)")

        # Celta → RC Celta de Vigo
        key_celta = _helper_normalize_for_key("Celta")
        if key_celta and tm_celta_normalized:
            self.MANUAL_OVERRIDES[key_celta] = tm_celta_normalized
            print(f"     • Override: '{key_celta}' → '{tm_celta_normalized}' (RC Celta de Vigo)")

        # Betis → Real Betis
        key_betis = _helper_normalize_for_key("Betis")
        if key_betis and tm_betis_normalized:
            self.MANUAL_OVERRIDES[key_betis] = tm_betis_normalized
            print(f"     • Override: '{key_betis}' → '{tm_betis_normalized}' (Real Betis)")

        # Sociedad → Real Sociedad
        key_sociedad = _helper_normalize_for_key("Sociedad")
        if key_sociedad and tm_sociedad_normalized:
            self.MANUAL_OVERRIDES[key_sociedad] = tm_sociedad_normalized
            print(f"     • Override: '{key_sociedad}' → '{tm_sociedad_normalized}' (Real Sociedad)")

        # Alaves → Deportivo Alavés
        key_alaves = _helper_normalize_for_key("Alaves")
        if key_alaves and tm_alaves_normalized:
            self.MANUAL_OVERRIDES[key_alaves] = tm_alaves_normalized
            print(f"     • Override: '{key_alaves}' → '{tm_alaves_normalized}' (Deportivo Alavés)")

        # Cadiz → Cádiz CF
        key_cadiz = _helper_normalize_for_key("Cadiz")
        if key_cadiz and tm_cadiz_normalized:
            self.MANUAL_OVERRIDES[key_cadiz] = tm_cadiz_normalized
            print(f"     • Override: '{key_cadiz}' → '{tm_cadiz_normalized}' (Cádiz CF)")

        # Getafe → Getafe CF
        key_getafe = _helper_normalize_for_key("Getafe")
        if key_getafe and tm_getafe_normalized:
            self.MANUAL_OVERRIDES[key_getafe] = tm_getafe_normalized
            print(f"     • Override: '{key_getafe}' → '{tm_getafe_normalized}' (Getafe CF)")

        print(f"     ✓ Added 11 hardcoded overrides")
        print(f"     ✓ Total overrides: {len(self.MANUAL_OVERRIDES)}\n")

    def normalize_team_name(self, s):
        """Normalize team name"""
        if pd.isna(s):
            return None

        s = str(s).strip()
        s = unidecode(s)

        suffixes = [
            ' FC', ' fc', ' Fc', ' fC', ' SC', ' sc', ' Sc',
            ' SK', ' sk', ' Sk', ' FK', ' fk', ' Fk',
            ' AC', ' ac', ' Ac', ' AS', ' as', ' As',
            ' SS', ' ss', ' Ss', ' CF', ' cf', ' Cf',
            ' HSC', ' hsc', ' Hsc', ' BC', ' bc', ' Bc',
            ' SSC', ' ssc', ' Ssc', ' US', ' us', ' Us',
            ' CA', ' ca', ' Ca', ' UD', ' ud', ' Ud',
            ' 1909', ' 1846', ' 1860', ' 2010', ' 1913',
            ' 1907', ' 1936', ' 1919', ' 2025',
            ' spor', ' kulübü', ' klub',
        ]

        for suffix in suffixes:
            s = s.replace(suffix, '')

        s = s.replace("'", "").replace("(", "").replace(")", "")

        # 🔥 KRİTİK: Kelime sıralama
        words = sorted(s.lower().split())
        s = ' '.join(words)
        s = re.sub(r'\s+', ' ', s).strip()

        # Manuel override uygula
        if s in self.MANUAL_OVERRIDES:
            s = self.MANUAL_OVERRIDES[s]

        return s if s else None

    def find_best_match(self, match_team, tm_teams, threshold=80):
        """Find best match using fuzzy matching"""
        if pd.isna(match_team):
            return None, 0

        match_norm = self.normalize_team_name(match_team)

        if not match_norm:
            return None, 0

        # tm_teams = [(original_name, normalized_name), ...]
        tm_norm_dict = {norm: orig for orig, norm in tm_teams if norm}

        # Exact match kontrolü
        if match_norm in tm_norm_dict:
            return tm_norm_dict[match_norm], 100

        # Fuzzy match
        best_score = 0
        best_match = None

        for tm_orig, tm_norm in tm_teams:
            if not tm_norm:
                continue

            score = fuzz.token_sort_ratio(match_norm, tm_norm)

            if score > best_score:
                best_score = score
                best_match = tm_orig

        if best_score >= threshold:
            return best_match, best_score

        return None, best_score

    def create_team_mapping(self, df_matches, df_tm):
        """Create mapping between Matches and Transfermarkt"""
        print("\n  🔗 Creating team mappings with fuzzy matching...")

        # Unique takımlar
        matches_teams = sorted(
            set(df_matches['HomeTeam'].dropna().unique()) |
            set(df_matches['AwayTeam'].dropna().unique())
        )

        tm_teams_orig = sorted(df_tm['ClubName'].dropna().unique())

        # Normalize et
        tm_teams_normalized = [
            (team, self.normalize_team_name(team))
            for team in tm_teams_orig
        ]

        print(f"     • Matches teams: {len(matches_teams)}")
        print(f"     • Transfermarkt teams: {len(tm_teams_orig)}")

        # Mapping
        for idx, team in enumerate(matches_teams):
            if idx % 50 == 0 and idx > 0:
                print(f"       Progress: {idx}/{len(matches_teams)}", end='\r')

            mapped, score = self.find_best_match(team, tm_teams_normalized, threshold=80)
            self.team_mapping[team] = mapped
            self.team_scores[team] = score

        print(f"       Progress: {len(matches_teams)}/{len(matches_teams)}")

        matched = sum(1 for v in self.team_mapping.values() if v is not None)
        match_rate = (matched / len(self.team_mapping)) * 100

        print(f"     ✓ Matched: {matched}/{len(self.team_mapping)} ({match_rate:.1f}%)")

        # Unmatched göster
        unmatched = [k for k, v in self.team_mapping.items() if v is None]
        if unmatched:
            print(f"     ⚠️  Unmatched teams ({len(unmatched)}):")
            for team in unmatched[:10]:
                print(f"        • {team}")
            if len(unmatched) > 10:
                print(f"        ... and {len(unmatched)-10} more")

        # Low score warnings
        low_scores = {k: v for k, v in self.team_scores.items()
                      if v is not None and v < 85 and k in self.team_mapping and self.team_mapping[k] is not None}
        if low_scores:
            print(f"\n     ⚠️  Low confidence matches (<85 score):")
            for team, score in sorted(low_scores.items(), key=lambda x: x[1])[:10]:
                mapped = self.team_mapping[team]
                print(f"        • {team:30s} → {mapped:30s} ({score})")

        return self.team_mapping

# ════════════════════════════════════════════════════════════════════════
# Initialize Fuzzy Matcher
# ════════════════════════════════════════════════════════════════════════
print("[INFO] Initializing fuzzy matcher...")
fuzzy_matcher = DataIntegration(base_path=BASE_PATH)
print("[OK] Fuzzy matching setup complete ✅\n")

# ────────────────────────────────────────────────────────────────────────────
# 1.1: Define file paths
# ────────────────────────────────────────────────────────────────────────────
matches_csv = os.path.join(BASE_PATH, "Matches.csv")
elo_csv = os.path.join(BASE_PATH, "EloRatings.csv")
transfermarkt_csv = os.path.join(BASE_PATH, "Transfermarkt_Data.csv")

print(f"[INFO] Expected file paths:")
print(f"  → Matches:       {matches_csv}")
print(f"  → ELO Ratings:   {elo_csv}")
print(f"  → Transfermarkt: {transfermarkt_csv}\n")

# ────────────────────────────────────────────────────────────────────────────
# 1.2: Load main dataset (CRITICAL!)
# ────────────────────────────────────────────────────────────────────────────
if not os.path.exists(matches_csv):
    print(f"[FATAL] Matches.csv not found at: {matches_csv}")
    print(f"[INFO] Available files in BASE_PATH:")
    try:
        for f in os.listdir(BASE_PATH):
            if f.endswith('.csv'):
                print(f"  → {f}")
    except:
        pass
    raise FileNotFoundError(f"Cannot proceed without Matches.csv")

print(f"[INFO] Loading Matches.csv...")
matches_all = pd.read_csv(matches_csv, parse_dates=['MatchDate'])
print(f"[OK] Loaded {len(matches_all):,} matches")
print(f"     Date range: {matches_all['MatchDate'].min().date()} to {matches_all['MatchDate'].max().date()}")
print(f"     Columns: {matches_all.shape[1]}")
print(f"     Sample columns: {list(matches_all.columns[:15])}\n")

# ════════════════════════════════════════════════════════════════════════
# 🆕 1.1.5: FILTER DATA FOR THESIS SCOPE (2015-2025 & TOP 5 LEAGUES)
# ════════════════════════════════════════════════════════════════════════
print(f"[INFO] PHASE 1.1.5: FILTERING DATA FOR THESIS SCOPE")
print("="*90)

# Define thesis scope
THESIS_START_DATE = '2015-09-01'
THESIS_END_DATE = '2025-06-01'
TOP_5_DIVISIONS = ['D1', 'F1', 'E0', 'I1', 'SP1']

LEAGUE_NAMES = {
    'E0': 'Premier League',
    'D1': 'Bundesliga',
    'I1': 'Serie A',
    'SP1': 'La Liga',
    'F1': 'Ligue 1'
}

print(f"\n[THESIS SCOPE DEFINITION]")
print(f"  • Time Period:  {THESIS_START_DATE} to {THESIS_END_DATE}")
print(f"  • Leagues:      {', '.join([LEAGUE_NAMES[div] for div in TOP_5_DIVISIONS])}")
print(f"  • Original Dataset: {len(matches_all):,} matches\n")

# Store original size for comparison
original_size = len(matches_all)

# Apply date filter
if 'MatchDate' not in matches_all.columns:
    print(f"[ERROR] 'MatchDate' column not found!")
    raise ValueError("Cannot apply date filter without MatchDate column")

matches_all = matches_all[
    (matches_all['MatchDate'] >= THESIS_START_DATE) &
    (matches_all['MatchDate'] <= THESIS_END_DATE)
].copy()

after_date_filter = len(matches_all)
print(f"[STEP 1] Date Filter Applied:")
print(f"  ✓ Kept matches between {THESIS_START_DATE} and {THESIS_END_DATE}")
print(f"  • Before: {original_size:,} matches")
print(f"  • After:  {after_date_filter:,} matches")
print(f"  • Removed: {original_size - after_date_filter:,} matches ({100*(original_size - after_date_filter)/original_size:.1f}%)\n")

# Apply league filter
if 'Division' not in matches_all.columns:
    print(f"[ERROR] 'Division' column not found!")
    print(f"[INFO] Available columns: {list(matches_all.columns[:20])}")
    raise ValueError("Cannot apply league filter without Division column")

# Check which divisions are present
present_divisions = matches_all['Division'].unique()
print(f"[INFO] Divisions present in data after date filter:")
for div in sorted(present_divisions):
    count = len(matches_all[matches_all['Division'] == div])
    league_name = LEAGUE_NAMES.get(div, 'Unknown')
    marker = "✓" if div in TOP_5_DIVISIONS else "✗"
    print(f"  {marker} {div:5s} ({league_name:20s}): {count:,} matches")

matches_all = matches_all[matches_all['Division'].isin(TOP_5_DIVISIONS)].copy()

after_league_filter = len(matches_all)
print(f"\n[STEP 2] League Filter Applied:")
print(f"  ✓ Kept only Top 5 European leagues")
print(f"  • Before: {after_date_filter:,} matches")
print(f"  • After:  {after_league_filter:,} matches")
print(f"  • Removed: {after_date_filter - after_league_filter:,} matches ({100*(after_date_filter - after_league_filter)/after_date_filter:.1f}%)\n")

# Reset index after filtering
matches_all = matches_all.reset_index(drop=True)

# Summary statistics
print(f"[FILTERING SUMMARY]")
print("="*90)
print(f"  📊 FINAL DATASET (THESIS SCOPE):")
print(f"     • Total Matches:    {len(matches_all):,}")
print(f"     • Date Range:       {matches_all['MatchDate'].min().date()} → {matches_all['MatchDate'].max().date()}")
print(f"     • Leagues:          {sorted(matches_all['Division'].unique())}")
print(f"     • Reduction:        {original_size:,} → {len(matches_all):,} ({100*(1 - len(matches_all)/original_size):.1f}% filtered out)")

print(f"\n  📋 MATCHES BY LEAGUE:")
for div in TOP_5_DIVISIONS:
    league_count = len(matches_all[matches_all['Division'] == div])
    league_pct = 100 * league_count / len(matches_all)
    print(f"     • {LEAGUE_NAMES[div]:20s} ({div}): {league_count:,} ({league_pct:.1f}%)")

print(f"\n  📅 MATCHES BY SEASON:")
if 'MatchDate' in matches_all.columns:
    matches_all['Year'] = matches_all['MatchDate'].dt.year
    year_counts = matches_all['Year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"     • {year}: {count:,} matches")
    matches_all = matches_all.drop(columns=['Year'])

print("="*90 + "\n")

# ════════════════════════════════════════════════════════════════════════
# 🆕 1.2.1: CREATE TARGET COLUMN FROM FTResult
# ════════════════════════════════════════════════════════════════════════
print(f"[INFO] Creating 'Target' column from 'FTResult'...")

if 'FTResult' not in matches_all.columns:
    print(f"  ❌ FATAL: 'FTResult' column not found!")
    print(f"  Available columns: {list(matches_all.columns)}")
    raise ValueError("Cannot create Target without FTResult column")

# FTResult → Target mapping
result_mapping = {
    'H': 1,  # Home Win
    'D': 0,  # Draw
    'A': 2   # Away Win
}

matches_all['Target'] = matches_all['FTResult'].map(result_mapping)

# Verify mapping
n_mapped = matches_all['Target'].notna().sum()
n_total = len(matches_all)
mapping_rate = (n_mapped / n_total) * 100

print(f"  ✓ Created 'Target' column from 'FTResult'")
print(f"     • Mapped: {n_mapped:,} / {n_total:,} ({mapping_rate:.2f}%)")

# Show distribution
target_dist = matches_all['Target'].value_counts().sort_index()
print(f"\n  [Target Distribution]")
print(f"     • Draw (0):     {target_dist.get(0, 0):,} matches ({100*target_dist.get(0, 0)/n_mapped:.1f}%)")
print(f"     • Home Win (1): {target_dist.get(1, 0):,} matches ({100*target_dist.get(1, 0)/n_mapped:.1f}%)")
print(f"     • Away Win (2): {target_dist.get(2, 0):,} matches ({100*target_dist.get(2, 0)/n_mapped:.1f}%)")

# Check for unmapped values
unmapped = matches_all[matches_all['Target'].isna()]['FTResult'].value_counts()
if len(unmapped) > 0:
    print(f"\n  ⚠️  Unmapped FTResult values found:")
    for val, count in unmapped.items():
        print(f"     • '{val}': {count:,} cases")
else:
    print(f"  ✓ All FTResult values successfully mapped\n")

# ════════════════════════════════════════════════════════════════════════
# 🆕 1.2.2: CREATE YEARMONTH COLUMN
# ════════════════════════════════════════════════════════════════════════
print(f"\n[INFO] 1.2.2: Creating YearMonth column...")

matches_all['YearMonth'] = matches_all['MatchDate'].dt.to_period('M').astype(str)

print(f"  ✓ Created YearMonth column")
print(f"     • Sample values: {matches_all['YearMonth'].head(5).tolist()}")
print(f"     • Unique months: {matches_all['YearMonth'].nunique()}")
print(f"     • Date range: {matches_all['YearMonth'].min()} to {matches_all['YearMonth'].max()}\n")

# ════════════════════════════════════════════════════════════════════════
# 🆕 1.2.3: CREATE SEASON COLUMN
# ════════════════════════════════════════════════════════════════════════
print(f"[INFO] 1.2.3: Creating Season column...")

def create_season(date):
    """Create season string (e.g., '2023-2024')"""
    year = date.year
    month = date.month

    if month >= 7:  # Temmuz'dan sonra → yeni sezon
        return f"{year}-{year+1}"
    else:
        return f"{year-1}-{year}"

matches_all['Season'] = matches_all['MatchDate'].apply(create_season)

print(f"  ✓ Created Season column")
print(f"     • Sample values: {matches_all['Season'].head(5).tolist()}")
print(f"     • Unique seasons: {matches_all['Season'].nunique()}")
print(f"     • Seasons: {sorted(matches_all['Season'].unique())[:10]}...\n")

# ────────────────────────────────────────────────────────────────────────────
# 1.3: Merge ELO Ratings (if available)
# ────────────────────────────────────────────────────────────────────────────
if os.path.exists(elo_csv):
    print(f"[INFO] Loading EloRatings.csv...")
    try:
        # Load ELO data
        elo_df = pd.read_csv(elo_csv, parse_dates=['date'])
        elo_df = elo_df.rename(columns={'date': 'MatchDate', 'club': 'TeamName', 'elo': 'Elo'})

        print(f"  ✓ Loaded {len(elo_df):,} ELO records")
        print(f"     • Date range: {elo_df['MatchDate'].min().date()} to {elo_df['MatchDate'].max().date()}")
        print(f"     • Unique teams: {elo_df['TeamName'].nunique()}")

        # Create lookup function for nearest ELO
        def get_nearest_elo(team, match_date, elo_df):
            """Get nearest ELO rating before or on match date"""
            team_elos = elo_df[elo_df['TeamName'] == team]

            if team_elos.empty:
                return np.nan

            # Find ELO on or before match date
            valid_elos = team_elos[team_elos['MatchDate'] <= match_date]

            if valid_elos.empty:
                # If no prior ELO, use first available
                return team_elos.iloc[0]['Elo']

            # Return most recent ELO before match
            return valid_elos.sort_values('MatchDate', ascending=False).iloc[0]['Elo']

        # Merge ELO (if not already present)
        if 'HomeElo' not in matches_all.columns or matches_all['HomeElo'].isna().all():
            print(f"  → Merging HomeElo...")
            matches_all['HomeElo'] = matches_all.apply(
                lambda row: get_nearest_elo(row['HomeTeam'], row['MatchDate'], elo_df),
                axis=1
            )

        if 'AwayElo' not in matches_all.columns or matches_all['AwayElo'].isna().all():
            print(f"  → Merging AwayElo...")
            matches_all['AwayElo'] = matches_all.apply(
                lambda row: get_nearest_elo(row['AwayTeam'], row['MatchDate'], elo_df),
                axis=1
            )

        elo_coverage = matches_all['HomeElo'].notna().sum() / len(matches_all) * 100
        print(f"  ✓ ELO ratings merged")
        print(f"     • Coverage: {elo_coverage:.1f}% of matches\n")

    except Exception as e_elo:
        print(f"  ❌ Failed to merge ELO data: {e_elo}\n")
else:
    print(f"[WARN] EloRatings.csv not found - skipping ELO features\n")

# =============================================================================
# PHASE 1.4: TRANSFERMARKT DATA MERGE (ENHANCED WITH FUZZY MATCHING)
# =============================================================================
print("\n[INFO] 1.4: Loading Transfermarkt Data WITH FUZZY MATCHING & 2-LEVEL MERGE")
print("="*90)

# ════════════════════════════════════════════════════════════════════════
# 🔥 FIX: Preserve original Season column
# ════════════════════════════════════════════════════════════════════════
SEASON_BACKUP = None
if 'Season' in matches_all.columns:
    SEASON_BACKUP = matches_all['Season'].copy()
    print(f"  ✓ Backed up original Season column (format: {matches_all['Season'].iloc[0]})")

transfermarkt_xlsx = os.path.join(BASE_PATH, "data.xlsx")

if os.path.exists(transfermarkt_xlsx):
    print(f"  ✓ Found Transfermarkt data: {os.path.basename(transfermarkt_xlsx)}")
    print(f"  ✓ File size: {os.path.getsize(transfermarkt_xlsx) / 1024 / 1024:.2f} MB")

    try:
        # ════════════════════════════════════════════════════════════
        # LOAD EXCEL
        # ════════════════════════════════════════════════════════════
        print(f"\n  → Loading Excel file (this may take 10-30 seconds)...")
        tm_df = pd.read_excel(transfermarkt_xlsx, engine='openpyxl')

        print(f"\n  ✅ Loaded {len(tm_df):,} Transfermarkt records")
        print(f"     • Date range: {tm_df['YearMonth'].min()} to {tm_df['YearMonth'].max()}")
        print(f"     • Unique clubs: {tm_df['ClubName'].nunique()}")
        print(f"     • Columns: {tm_df.shape[1]}")

        # ════════════════════════════════════════════════════════════
        # 🆕 CREATE SEASON COLUMN (if not exists)
        # ════════════════════════════════════════════════════════════
        if 'Season' not in tm_df.columns:
            print(f"\n  [SEASON CREATION] Season column not found, creating...")

            # YearMonth'tan Season oluştur
            tm_df['YearMonth_Period'] = pd.to_datetime(tm_df['YearMonth'] + '-01')
            tm_df['Season'] = tm_df['YearMonth_Period'].apply(create_season)

            print(f"     ✓ Season column created")
            print(f"        Sample: {tm_df['Season'].head(3).tolist()}")
        else:
            print(f"\n  ✓ Season column already exists")

        # ════════════════════════════════════════════════════════════
        # 🆕 FUZZY MATCHING
        # ════════════════════════════════════════════════════════════
        print(f"\n  [FUZZY MATCHING] Creating team mappings...")

        team_mapping = fuzzy_matcher.create_team_mapping(matches_all, tm_df)

        # Apply mapping
        matches_all['HomeTeam_TM'] = matches_all['HomeTeam'].map(team_mapping)
        matches_all['AwayTeam_TM'] = matches_all['AwayTeam'].map(team_mapping)

        # ════════════════════════════════════════════════════════════
        # LEVEL 1: YEARMONTH-BASED MERGE
        # ════════════════════════════════════════════════════════════
        print(f"\n  [MERGE LEVEL 1] YearMonth-based merge...")

        # Deduplicate TM data
        tm_df_yearmonth = tm_df.drop_duplicates(
            subset=['ClubName', 'YearMonth'],
            keep='last'
        ).copy()

        print(f"     • Unique YearMonth records: {len(tm_df_yearmonth):,}")

        # ────────────────────────────────────────────────────────────
        # HOME TEAM MERGE (Level 1)
        # ────────────────────────────────────────────────────────────
        print(f"\n     🏠 [HOME] Level 1 merge...")

        tm_home_l1 = tm_df_yearmonth.rename(columns={'ClubName': 'HomeTeam_TM'})

        matches_all = matches_all.merge(
            tm_home_l1,
            on=['HomeTeam_TM', 'YearMonth'],
            how='left',
            suffixes=('', '_home_tm')
        )

        home_cols = [c for c in tm_home_l1.columns
                     if c not in ['HomeTeam_TM', 'YearMonth']]

        # Check unmatched
        if home_cols:
            unmatched_mask = matches_all[home_cols[0]].isna()
        else:
            unmatched_mask = pd.Series(False, index=matches_all.index)

        unmatched_indices = matches_all[unmatched_mask].index

        print(f"        • Matched: {(~unmatched_mask).sum():,}")
        print(f"        • Unmatched: {len(unmatched_indices):,}")

        # ────────────────────────────────────────────────────────────
        # HOME TEAM FALLBACK (Level 2: Season)
        # ────────────────────────────────────────────────────────────
        if len(unmatched_indices) > 0:
            print(f"\n     🏠 [HOME] Level 2 fallback (Season-based)...")

            # 🔥 Aggregate by Season
            numeric_cols = tm_df.select_dtypes(include=np.number).columns.tolist()
            numeric_cols = [c for c in numeric_cols if c not in ['club_id', 'wettbewerb_id']]

            non_numeric_cols = tm_df.select_dtypes(exclude=np.number).columns.tolist()
            non_numeric_cols = [c for c in non_numeric_cols
                               if c not in ['ClubName', 'Season', 'YearMonth', 'LeagueName']]

            agg_dict = {col: 'mean' for col in numeric_cols if col in tm_df.columns}
            for col in non_numeric_cols:
                if col in tm_df.columns:
                    agg_dict[col] = 'first'

            tm_df_season = tm_df.groupby(['ClubName', 'Season'], as_index=False).agg(agg_dict)

            print(f"        • Unique Season records: {len(tm_df_season):,}")

            # Merge
            fallback_data = matches_all.loc[unmatched_indices, ['HomeTeam_TM', 'Season']]
            tm_home_l2 = tm_df_season.rename(columns={'ClubName': 'HomeTeam_TM'})

            fallback_merged = fallback_data.merge(
                tm_home_l2,
                on=['HomeTeam_TM', 'Season'],
                how='left'
            )

            # Fill unmatched
            for col in home_cols:
                if col in fallback_merged.columns:
                    fallback_merged.index = unmatched_indices
                    matches_all.loc[unmatched_indices, col] = fallback_merged[col]

            filled = matches_all.loc[unmatched_indices, home_cols[0]].notna().sum() if home_cols else 0
            print(f"        • Filled by Season: {filled:,}")

        # Rename home columns
        rename_dict = {c: f'HomeTeam_{c}' for c in home_cols if c in matches_all.columns}
        matches_all = matches_all.rename(columns=rename_dict)

        if home_cols:
            home_coverage = matches_all[f'HomeTeam_{home_cols[0]}'].notna().sum()
            home_pct = (home_coverage / len(matches_all)) * 100
            print(f"\n        ✓ Final HOME coverage: {home_coverage:,} / {len(matches_all):,} ({home_pct:.1f}%)")

        # ────────────────────────────────────────────────────────────
        # AWAY TEAM MERGE (Level 1)
        # ────────────────────────────────────────────────────────────
        print(f"\n     ✈️  [AWAY] Level 1 merge...")

        tm_away_l1 = tm_df_yearmonth.rename(columns={'ClubName': 'AwayTeam_TM'})

        matches_all = matches_all.merge(
            tm_away_l1,
            on=['AwayTeam_TM', 'YearMonth'],
            how='left',
            suffixes=('', '_away_tm')
        )

        away_cols = [c for c in tm_away_l1.columns
                     if c not in ['AwayTeam_TM', 'YearMonth']]

        # Check unmatched
        if away_cols:
            away_check_col = away_cols[0]
            if away_check_col + '_away_tm' in matches_all.columns:
                away_check_col = away_check_col + '_away_tm'

            unmatched_mask_away = matches_all[away_check_col].isna()
        else:
            unmatched_mask_away = pd.Series(False, index=matches_all.index)

        unmatched_indices_away = matches_all[unmatched_mask_away].index

        print(f"        • Matched: {(~unmatched_mask_away).sum():,}")
        print(f"        • Unmatched: {len(unmatched_indices_away):,}")

        # ────────────────────────────────────────────────────────────
        # AWAY TEAM FALLBACK (Level 2: Season)
        # ────────────────────────────────────────────────────────────
        if len(unmatched_indices_away) > 0:
            print(f"\n     ✈️  [AWAY] Level 2 fallback (Season-based)...")

            fallback_data_away = matches_all.loc[unmatched_indices_away, ['AwayTeam_TM', 'Season']]
            tm_away_l2 = tm_df_season.rename(columns={'ClubName': 'AwayTeam_TM'})

            fallback_merged_away = fallback_data_away.merge(
                tm_away_l2,
                on=['AwayTeam_TM', 'Season'],
                how='left',
                suffixes=('', '_away_l2')
            )

            # Fill unmatched
            for col in away_cols:
                # Determine target column name
                target_col = col
                if col + '_away_tm' in matches_all.columns:
                    target_col = col + '_away_tm'

                # Determine source column name
                source_col = col
                if col + '_away_l2' in fallback_merged_away.columns:
                    source_col = col + '_away_l2'

                if source_col in fallback_merged_away.columns and target_col in matches_all.columns:
                    fallback_merged_away.index = unmatched_indices_away
                    matches_all.loc[unmatched_indices_away, target_col] = fallback_merged_away[source_col]

            if away_cols:
                check_col = away_cols[0]
                if check_col + '_away_tm' in matches_all.columns:
                    check_col = check_col + '_away_tm'
                filled_away = matches_all.loc[unmatched_indices_away, check_col].notna().sum()
                print(f"        • Filled by Season: {filled_away:,}")

        # Rename away columns
        rename_dict_away = {}
        for col in away_cols:
            suffixed_col = col + '_away_tm'
            if suffixed_col in matches_all.columns:
                rename_dict_away[suffixed_col] = 'AwayTeam_' + col
            elif col in matches_all.columns and not col.startswith('HomeTeam_'):
                rename_dict_away[col] = 'AwayTeam_' + col

        matches_all = matches_all.rename(columns=rename_dict_away)

        if away_cols:
            away_check_final = f'AwayTeam_{away_cols[0]}'
            if away_check_final in matches_all.columns:
                away_coverage = matches_all[away_check_final].notna().sum()
                away_pct = (away_coverage / len(matches_all)) * 100
                print(f"\n        ✓ Final AWAY coverage: {away_coverage:,} / {len(matches_all):,} ({away_pct:.1f}%)")

        # ════════════════════════════════════════════════════════════
        # 🔥 FIX: Restore original Season column (AFTER ALL MERGES)
        # ════════════════════════════════════════════════════════════
        if SEASON_BACKUP is not None:
            if 'Season' not in matches_all.columns:
                # Case 1: Season column completely disappeared
                matches_all['Season'] = SEASON_BACKUP
                print(f"\n  ✅ Restored original Season column (was missing after merge)")
            elif matches_all['Season'].isna().all():
                # Case 2: Season column exists but all NaN
                matches_all['Season'] = SEASON_BACKUP
                print(f"\n  ✅ Restored original Season column (was all NaN after merge)")
            elif matches_all['Season'].dtype != SEASON_BACKUP.dtype:
                # Case 3: Season column type changed
                matches_all['Season'] = SEASON_BACKUP
                print(f"\n  ⚠️  Season column type changed during merge - restored original")
                print(f"     Old type: {matches_all['Season'].dtype}, New type: {SEASON_BACKUP.dtype}")
            else:
                # Case 4: Check if values are different
                if not matches_all['Season'].equals(SEASON_BACKUP):
                    matches_all['Season'] = SEASON_BACKUP
                    print(f"\n  ⚠️  Season column values changed during merge - restored original")
                else:
                    print(f"\n  ✓ Season column preserved correctly during merge")

        # ════════════════════════════════════════════════════════════
        # CREATE VALUERATIO (if columns exist)
        # ════════════════════════════════════════════════════════════
        print(f"\n  [FEATURE ENGINEERING] Creating ValueRatio...")

        if 'HomeTeam_ClubValue' in matches_all.columns and 'HomeTeam_LeagueValue' in matches_all.columns:
            matches_all['HomeTeam_ValueRatio'] = (
                matches_all['HomeTeam_ClubValue'] / (matches_all['HomeTeam_LeagueValue'] + 1e-6)
            )
            print(f"     ✓ HomeTeam_ValueRatio created")

        if 'AwayTeam_ClubValue' in matches_all.columns and 'AwayTeam_LeagueValue' in matches_all.columns:
            matches_all['AwayTeam_ValueRatio'] = (
                matches_all['AwayTeam_ClubValue'] / (matches_all['AwayTeam_LeagueValue'] + 1e-6)
            )
            print(f"     ✓ AwayTeam_ValueRatio created")

        # ════════════════════════════════════════════════════════════
        # CREATE DIFF FEATURES
        # ════════════════════════════════════════════════════════════
        print(f"\n  [DIFF FEATURES] Creating Home-Away differences...")

        diff_features = [
            ('ClubValue', 'Club Market Value'),
            ('LeagueValue', 'League Average Value'),
            ('MaxPlayerValue', 'Star Player Value'),
            ('ManagerTrophies', 'Manager Trophy Count'),
            ('ManagerTenureDays', 'Manager Experience'),
            ('NetTransferSpending', 'Net Transfer Spending'),
            ('n_players_injured', 'Injured Players'),
            ('n_suspended_players', 'Suspended Players'),
            ('ValueRatio', 'Value Ratio')
        ]

        features_created = []

        for feature, description in diff_features:
            home_col = f'HomeTeam_{feature}'
            away_col = f'AwayTeam_{feature}'
            diff_col = f'{feature}_Diff'

            if home_col in matches_all.columns and away_col in matches_all.columns:
                matches_all[diff_col] = matches_all[home_col] - matches_all[away_col]

                non_null = matches_all[diff_col].notna().sum()
                coverage_pct = (non_null / len(matches_all)) * 100

                features_created.append(diff_col)
                print(f"     ✓ {diff_col:35s}: {coverage_pct:5.1f}%")

        print(f"\n  ✅ Transfermarkt merge complete!")
        print(f"     • Total new columns: {len(features_created) + len(home_cols) + len(away_cols)}")
        print(f"     • Diff features: {len(features_created)}")

    except Exception as e_tm:
        print(f"\n  ❌ Transfermarkt merge FAILED!")
        print(f"     Error: {e_tm}")
        import traceback
        traceback.print_exc()
        print("\n  ⚠️  Continuing without Transfermarkt features...\n")

        # 🔥 FIX: Restore Season even if merge failed
        if SEASON_BACKUP is not None and 'Season' not in matches_all.columns:
            matches_all['Season'] = SEASON_BACKUP
            print(f"  ✅ Restored original Season column after merge failure")

else:
    print(f"  ❌ File not found: {transfermarkt_xlsx}")
    print(f"  ⚠️  Continuing without Transfermarkt features...\n")

print("="*90 + "\n")


# ────────────────────────────────────────────────────────────────────────────
# 1.5: Basic data cleaning
# ────────────────────────────────────────────────────────────────────────────
print(f"[INFO] Performing basic data cleaning...")

# Sort by date (CRITICAL for temporal features!)
matches_all = matches_all.sort_values('MatchDate').reset_index(drop=True)
print(f"  ✓ Sorted by MatchDate")

# Check for duplicates
n_duplicates = matches_all.duplicated(subset=['MatchDate', 'HomeTeam', 'AwayTeam']).sum()
if n_duplicates > 0:
    print(f"  ⚠️  Found {n_duplicates} duplicate matches - removing...")
    matches_all = matches_all.drop_duplicates(subset=['MatchDate', 'HomeTeam', 'AwayTeam'], keep='first')
else:
    print(f"  ✓ No duplicates found")

# Check required columns
required_cols = ['MatchDate', 'HomeTeam', 'AwayTeam', 'Target']
missing_required = [col for col in required_cols if col not in matches_all.columns]

if missing_required:
    print(f"\n  ❌ Missing required columns: {missing_required}")
    print(f"  Available columns: {list(matches_all.columns[:30])}")
    raise ValueError("Cannot proceed without required columns")
else:
    print(f"  ✓ All required columns present: {required_cols}")

# Add basic derived features if not present
if 'ELO_Diff' not in matches_all.columns and 'HomeElo' in matches_all.columns:
    matches_all['ELO_Diff'] = matches_all['HomeElo'] - matches_all['AwayElo']
    print(f"  ✓ Created ELO_Diff feature")

print(f"\n[OK] Data cleaning complete")
print(f"[OK] Final dataset: {matches_all.shape[0]:,} rows × {matches_all.shape[1]} columns")
print(f"[OK] Memory usage: {matches_all.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# ────────────────────────────────────────────────────────────────────────────
# 1.6: Summary Statistics
# ────────────────────────────────────────────────────────────────────────────
print("\n" + "="*90)
print("[SUMMARY] Data Loading Complete")
print("="*90)

print(f"\n📊 DATASET STATISTICS:")
print(f"  • Total Matches:        {len(matches_all):,}")
print(f"  • Date Range:           {matches_all['MatchDate'].min().date()} → {matches_all['MatchDate'].max().date()}")
print(f"  • Unique Home Teams:    {matches_all['HomeTeam'].nunique()}")
print(f"  • Unique Away Teams:    {matches_all['AwayTeam'].nunique()}")
print(f"  • Total Features:       {matches_all.shape[1]}")

print(f"\n🎯 TARGET DISTRIBUTION:")
target_pct = matches_all['Target'].value_counts(normalize=True).sort_index() * 100
print(f"  • Draw (0):      {target_pct.get(0, 0):5.2f}%")
print(f"  • Home Win (1):  {target_pct.get(1, 0):5.2f}%")
print(f"  • Away Win (2):  {target_pct.get(2, 0):5.2f}%")

print(f"\n📈 DATA SOURCES:")
elo_cov = matches_all['HomeElo'].notna().sum() / len(matches_all) * 100
print(f"  • ELO Ratings:         {elo_cov:.1f}% coverage")

if 'HomeTeam_ClubValue' in matches_all.columns:
    tm_cov = matches_all['HomeTeam_ClubValue'].notna().sum() / len(matches_all) * 100
    print(f"  • Transfermarkt Data:  {tm_cov:.1f}% coverage")

    if 'ValueRatio_Diff' in matches_all.columns:
        vr_cov = matches_all['ValueRatio_Diff'].notna().sum() / len(matches_all) * 100
        print(f"  • ValueRatio Feature:  {vr_cov:.1f}% coverage ✨ (NEW!)")

print("\n" + "="*90)
print("[✓] PHASE 1 COMPLETE - matches_all loaded and ready!")
print("="*90 + "\n")

# ────────────────────────────────────────────────────────────────────────────
# 1.7: Verify matches_all is ready
# ────────────────────────────────────────────────────────────────────────────
print("[VERIFICATION] Final checks...")
print(f"  ✓ matches_all type: {type(matches_all)}")
print(f"  ✓ Shape: {matches_all.shape}")
print(f"  ✓ Target column exists: {'Target' in matches_all.columns}")
print(f"  ✓ Target has values: {matches_all['Target'].notna().any()}")
print(f"  ✓ ValueRatio_Diff exists: {'ValueRatio_Diff' in matches_all.columns}")

if matches_all.empty:
    raise ValueError("[FATAL] matches_all is empty after loading!")

print("\n[OK] All verifications passed ✅\n")



# =============================================================================
# 🆕 PHASE 1.5: TEMPORAL FEATURES CREATION (LAG-BASED AVERAGES)
# =============================================================================
print("\n[INFO] PHASE 1.5: CREATING TEMPORAL FEATURES (Lag Averages)")
print("="*90)

# Initialize list to track created features
temporal_features_created = []

# ────────────────────────────────────────────────────────────────────────────
# STEP 1: Check required columns
# ────────────────────────────────────────────────────────────────────────────
print("[STEP 1] Checking required columns...")

# Maç-içi metrikler (bunlar LEAKAGE olduğu için direkt kullanılmayacak)
in_game_metrics = {
    'Target': ['HomeTarget', 'AwayTarget'],      # Shots on target
    'Shots': ['HomeShots', 'AwayShots'],         # Total shots
    'Corners': ['HomeCorners', 'AwayCorners'],   # Corners
    'Fouls': ['HomeFouls', 'AwayFouls'],         # Fouls
    'Yellow': ['HomeYellow', 'AwayYellow']       # Yellow cards
}

# Hangi metrikler mevcut?
available_metrics = {}
for metric_name, cols in in_game_metrics.items():
    if all(col in matches_all.columns for col in cols):
        available_metrics[metric_name] = cols
        print(f"  ✓ {metric_name}: {cols[0]}, {cols[1]}")
    else:
        missing = [c for c in cols if c not in matches_all.columns]
        print(f"  ✗ {metric_name}: Missing {missing}")

if not available_metrics:
    print("\n  ❌ ERROR: No in-game metrics found!")
    print("     Required columns like 'HomeTarget', 'HomeShots' etc. are missing")
    print("     → Skipping temporal feature creation\n")
    temporal_features_created = []
else:
    print(f"\n  ✅ Found {len(available_metrics)} metric types for temporal features\n")

    # ────────────────────────────────────────────────────────────────────────
    # STEP 2: Create LAG features (L5 window)
    # ────────────────────────────────────────────────────────────────────────
    print("[STEP 2] Creating temporal averages (L5 window)...")
    print("  → Using shift(1) to prevent data leakage")
    print("  → min_periods=3 for stability\n")

    for metric_name, [home_col, away_col] in available_metrics.items():

        # HOME team temporal average
        home_avg_col = f'Home_{metric_name}_Avg_L5'
        print(f"  [Home] Creating {home_avg_col}...", end=' ')

        try:
            matches_all[home_avg_col] = (
                matches_all.groupby('HomeTeam')[home_col]
                .transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
            )

            coverage = matches_all[home_avg_col].notna().sum() / len(matches_all) * 100
            print(f"✓ (coverage: {coverage:.1f}%)")
            temporal_features_created.append(home_avg_col)

        except Exception as e:
            print(f"✗ Error: {e}")

        # AWAY team temporal average
        away_avg_col = f'Away_{metric_name}_Avg_L5'
        print(f"  [Away] Creating {away_avg_col}...", end=' ')

        try:
            matches_all[away_avg_col] = (
                matches_all.groupby('AwayTeam')[away_col]
                .transform(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
            )

            coverage = matches_all[away_avg_col].notna().sum() / len(matches_all) * 100
            print(f"✓ (coverage: {coverage:.1f}%)")
            temporal_features_created.append(away_avg_col)

        except Exception as e:
            print(f"✗ Error: {e}")

    print()

    # ────────────────────────────────────────────────────────────────────────
    # STEP 3: Create DIFF features (Home - Away)
    # ────────────────────────────────────────────────────────────────────────
    print("[STEP 3] Creating difference features (Home - Away)...")

    for metric_name in available_metrics.keys():
        home_avg_col = f'Home_{metric_name}_Avg_L5'
        away_avg_col = f'Away_{metric_name}_Avg_L5'
        diff_col = f'{metric_name}_Diff_L5'

        if home_avg_col in matches_all.columns and away_avg_col in matches_all.columns:
            print(f"  Creating {diff_col}...", end=' ')

            try:
                matches_all[diff_col] = matches_all[home_avg_col] - matches_all[away_avg_col]

                coverage = matches_all[diff_col].notna().sum() / len(matches_all) * 100
                print(f"✓ (coverage: {coverage:.1f}%)")
                temporal_features_created.append(diff_col)

            except Exception as e:
                print(f"✗ Error: {e}")

    print()

    # ────────────────────────────────────────────────────────────────────────
    # STEP 4: Special feature - Shot Accuracy Difference
    # ────────────────────────────────────────────────────────────────────────
    if 'Target' in available_metrics and 'Shots' in available_metrics:
        print("[STEP 4] Creating shot accuracy features...")

        # Home Shot Accuracy
        if 'Home_Target_Avg_L5' in matches_all.columns and 'Home_Shots_Avg_L5' in matches_all.columns:
            print(f"  Creating Home_ShotAccuracy_L5...", end=' ')
            matches_all['Home_ShotAccuracy_L5'] = (
                matches_all['Home_Target_Avg_L5'] / (matches_all['Home_Shots_Avg_L5'] + 1e-6)
            )
            print(f"✓")
            temporal_features_created.append('Home_ShotAccuracy_L5')

        # Away Shot Accuracy
        if 'Away_Target_Avg_L5' in matches_all.columns and 'Away_Shots_Avg_L5' in matches_all.columns:
            print(f"  Creating Away_ShotAccuracy_L5...", end=' ')
            matches_all['Away_ShotAccuracy_L5'] = (
                matches_all['Away_Target_Avg_L5'] / (matches_all['Away_Shots_Avg_L5'] + 1e-6)
            )
            print(f"✓")
            temporal_features_created.append('Away_ShotAccuracy_L5')

        # Shot Accuracy Diff
        if 'Home_ShotAccuracy_L5' in matches_all.columns and 'Away_ShotAccuracy_L5' in matches_all.columns:
            print(f"  Creating ShotAccuracy_Diff_L5...", end=' ')
            matches_all['ShotAccuracy_Diff_L5'] = (
                matches_all['Home_ShotAccuracy_L5'] - matches_all['Away_ShotAccuracy_L5']
            )
            print(f"✓")
            temporal_features_created.append('ShotAccuracy_Diff_L5')

        print()

    # ────────────────────────────────────────────────────────────────────────
    # STEP 5: Summary
    # ────────────────────────────────────────────────────────────────────────
    print("="*90)
    print("[SUMMARY] Temporal Features Creation Complete")
    print("="*90)

    print(f"\n  ✅ Total features created: {len(temporal_features_created)}")
    print(f"  ✅ Window size: L5 (last 5 matches)")
    print(f"  ✅ Leakage prevention: shift(1) applied")
    print(f"  ✅ Min periods: 3 matches\n")

    if temporal_features_created:
        print("  📋 Created features:")
        for i, feat in enumerate(temporal_features_created, 1):
            coverage = matches_all[feat].notna().sum() / len(matches_all) * 100
            print(f"     {i:2d}. {feat:35s} ({coverage:5.1f}% coverage)")

    print("\n" + "="*90 + "\n")

# ═════════════════════════════════════════════════════════════════════════
# CRITICAL: Verify temporal_features_created exists
# ═════════════════════════════════════════════════════════════════════════
if 'temporal_features_created' not in globals():
    print("[WARN] temporal_features_created was not initialized!")
    temporal_features_created = []


# =============================================================================
# 🆕 PHASE 1.6: TEMPORAL FEATURES VALIDATION (Quality Control)
# =============================================================================
print("\n[INFO] PHASE 1.6: VALIDATING TEMPORAL FEATURES (Quality Control)")
print("="*90)

def validate_lag_features(df, lag_feature_names, original_features, output_dir):
    """
    ✅ V5.1 UYUMLU: Validate DIFF-ONLY temporal features

    Quality Control Tests:
    1. ✅ Leakage Check: Correlation with original features (should be <0.95)
    2. ✅ Coverage Analysis: % of non-null values (should be >60%)
    3. ⚠️  Window Consistency: SKIPPED (only L5 exists in V5.1)
    4. ✅ Visual Inspection: Distribution plots
    """

    print(f"  Validating {len(lag_feature_names)} temporal features...\n")

    if not lag_feature_names:
        print("  ⚠️  No lag features to validate")
        return None

    # ════════════════════════════════════════════════════════════════════
    # TEST 1: LEAKAGE CHECK (FIXED FOR V5.1)
    # ════════════════════════════════════════════════════════════════════
    print("[TEST 1] 🔒 LEAKAGE CHECK (shift(1) validation)")
    print("  → Verifying temporal isolation...\n")

    leakage_detected = False
    leakage_details = []

    # ✅ FIX: V5.1'de sadece Diff features var
    # Örnek: Target_Diff_L5 → HomeTarget ve AwayTarget ile karşılaştır

    for lag_feat in lag_feature_names:
        # Extract base feature name
        # 'Target_Diff_L5' → 'Target'
        base_name = lag_feat.replace('_Diff_L5', '').replace('_Diff_L3', '')

        # Look for Home/Away versions
        home_col = f'Home{base_name}'
        away_col = f'Away{base_name}'

        # Test against HomeX
        if home_col in df.columns and lag_feat in df.columns:
            valid_pairs = df[[home_col, lag_feat]].dropna()

            if len(valid_pairs) > 10:
                correlation = valid_pairs.corr().iloc[0, 1]

                # ✅ For Diff features, correlation should be LOWER
                # Because Diff = Home - Away (not just Home)
                if abs(correlation) > 0.95:
                    leakage_detected = True
                    leakage_details.append({
                        'lag_feature': lag_feat,
                        'original': home_col,
                        'correlation': correlation,
                        'type': 'Home'
                    })
                    print(f"     ⚠️  {lag_feat} ↔ {home_col}: r={correlation:.4f}")

        # Test against AwayX
        if away_col in df.columns and lag_feat in df.columns:
            valid_pairs = df[[away_col, lag_feat]].dropna()

            if len(valid_pairs) > 10:
                correlation = valid_pairs.corr().iloc[0, 1]

                if abs(correlation) > 0.95:
                    leakage_detected = True
                    leakage_details.append({
                        'lag_feature': lag_feat,
                        'original': away_col,
                        'correlation': correlation,
                        'type': 'Away'
                    })
                    print(f"     ⚠️  {lag_feat} ↔ {away_col}: r={correlation:.4f}")

    if leakage_detected:
        print(f"\n  🚨 CRITICAL: {len(leakage_details)} potential leakage cases!")
        print(f"     Note: For Diff features, some correlation is expected.")
        print(f"     Only |r| > 0.95 indicates leakage.\n")
    else:
        print(f"  ✅ No leakage detected (all |correlations| < 0.95)")
        print(f"     → shift(1) is working correctly\n")

    # ════════════════════════════════════════════════════════════════════
    # TEST 2: COVERAGE ANALYSIS (WITH SAFETY CHECK)
    # ════════════════════════════════════════════════════════════════════
    print("[TEST 2] 📊 COVERAGE ANALYSIS")

    coverage_data = []
    for feat in lag_feature_names:
        if feat in df.columns:
            non_null = df[feat].notna().sum()
            total = len(df)
            coverage_pct = (non_null / total) * 100

            coverage_data.append({
                'feature': feat,
                'coverage_pct': coverage_pct
            })

    if not coverage_data:
        print(f"  ⚠️  No coverage data collected - features might be missing")
        min_coverage = 0
        mean_coverage = 0
        coverage_df = pd.DataFrame()
    else:
        coverage_df = pd.DataFrame(coverage_data).sort_values('coverage_pct')

        min_coverage = coverage_df['coverage_pct'].min()
        mean_coverage = coverage_df['coverage_pct'].mean()

        print(f"  Coverage Statistics:")
        print(f"    • Min:  {min_coverage:.2f}%")
        print(f"    • Mean: {mean_coverage:.2f}%")

        if min_coverage < 60:
            print(f"    ⚠️  Some features have <60% coverage")
        else:
            print(f"    ✅ All features have ≥60% coverage")

    # ════════════════════════════════════════════════════════════════════
    # TEST 3: WINDOW CONSISTENCY (SKIPPED FOR V5.1)
    # ════════════════════════════════════════════════════════════════════
    print(f"\n[TEST 3] 🔗 WINDOW SIZE CONSISTENCY")
    print(f"  ⚠️  SKIPPED: V5.1 only uses L5 features (no L3 for comparison)")
    print(f"     Rationale: L5 selected as optimal window size\n")

    avg_consistency = 0  # Not applicable for V5.1

    # ════════════════════════════════════════════════════════════════════
    # TEST 4: VISUALIZATION (WITH SAFETY CHECKS)
    # ════════════════════════════════════════════════════════════════════
    if output_dir and not coverage_df.empty:
        print(f"[TEST 4] 📈 Creating validation plots...")

        fig = plt.figure(figsize=(18, 6))
        gs = fig.add_gridspec(1, 3, hspace=0.3, wspace=0.3)

        # PLOT 1: Coverage Heatmap
        ax1 = fig.add_subplot(gs[0, 0])

        # ✅ Safety: Check if we have data
        if len(coverage_df) > 0:
            worst_20 = coverage_df.head(min(20, len(coverage_df)))
            coverage_matrix = worst_20['coverage_pct'].values.reshape(-1, 1)

            im = ax1.imshow(coverage_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
            ax1.set_yticks(range(len(worst_20)))
            ax1.set_yticklabels([f[:35] for f in worst_20['feature']], fontsize=7)
            ax1.set_xticks([])
            ax1.set_title('Coverage (All Features)\nNon-Null %', fontweight='bold', fontsize=12)
            plt.colorbar(im, ax=ax1, label='Coverage %', fraction=0.046)
        else:
            ax1.text(0.5, 0.5, 'No coverage data', ha='center', va='center', transform=ax1.transAxes)

        # ════════════════════════════════════════════════════════════════════
        # PLOT 2: ValueRatio_Diff Distribution (DÜZELTİLMİŞ)
        # ════════════════════════════════════════════════════════════════════
        ax2 = fig.add_subplot(gs[0, 1])
        if 'ValueRatio_Diff' in df.columns and 'Target' in df.columns:
            print("[PLOT 2] ValueRatio_Diff distribution...")
            # Döngü içinde renkler otomatik atanıyor
            for target_val, color, label in zip([0, 1, 2], [UNIFIED_COLORS['draw_orange'], UNIFIED_COLORS['home_green'], UNIFIED_COLORS['away_red']], class_names):
                data = df[df['Target'] == target_val]['ValueRatio_Diff'].dropna()
                if len(data) > 0:
                    # DÜZELTME BURADA:
                    ax2.hist(data, bins=40,
                             alpha=0.9,
                             color=color,
                             label=f'{label} (n={len(data):,})',
                             edgecolor='black',
                             linewidth=1.2)
            ax2.axvline(0, color='black', linestyle='--', linewidth=2.5, alpha=0.7)
            ax2.set_xlabel('ValueRatio_Diff', fontsize=12, fontweight='bold')
            ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
            ax2.set_title('Financial Advantage Indicator\n' +
                          'ValueRatio = ClubValue / LeagueValue',
                          fontsize=14, fontweight='bold', pad=15)
            ax2.legend(fontsize=10)
            ax2.grid(alpha=0.5, linestyle='--')
        else:
            ax2.text(0.5, 0.5, 'ValueRatio_Diff not available',
                     ha='center', va='center', transform=ax2.transAxes)

        # PLOT 3: Summary
        ax3 = fig.add_subplot(gs[0, 2])
        ax3.axis('off')

        summary_text = f"""
LAG FEATURES VALIDATION (V5.1)

Total Features:     {len(lag_feature_names)}
Feature Type:       Diff-Only (L5)

LEAKAGE CHECK:
  Status:           {'❌ FAIL' if leakage_detected else '✅ PASS'}
  Cases:            {len(leakage_details)}

COVERAGE:
  Min:              {min_coverage:.1f}%
  Mean:             {mean_coverage:.1f}%

WINDOW CONSISTENCY:
  Status:           SKIPPED (V5.1)
  Reason:           Only L5 exists

OVERALL: {'✅ PASS' if not leakage_detected and min_coverage > 60 else '⚠️  REVIEW'}
        """

        ax3.text(0.05, 0.95, summary_text.strip(),
                transform=ax3.transAxes, fontsize=9,
                verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.95))

        plt.suptitle('Temporal Features Validation Report (V5.1 - Diff-Only)',
                    fontsize=14, fontweight='bold', y=0.98)

        save_path = os.path.join(output_dir, '01d_temporal_features_validation.png')
        plt.savefig(save_path, dpi=600, bbox_inches='tight', facecolor='white')
        savefig_report('01d_temporal_features_validation.png', also_png=True)
        plt.close()

        print(f"  ✅ Validation report saved: {os.path.basename(save_path)}")
    elif output_dir:
        print(f"  ⚠️  Skipping visualization - no data available")

    print("\n" + "="*90 + "\n")

    return {
        'leakage_detected': leakage_detected,
        'min_coverage': min_coverage,
        'mean_coverage': mean_coverage,
        'avg_consistency': avg_consistency,  # Always 0 for V5.1
        'overall_pass': not leakage_detected and min_coverage > 60
    }


# ════════════════════════════════════════════════════════════════════════
# EXECUTE VALIDATION (V5.1 Enhanced with Safety Checks)
# ════════════════════════════════════════════════════════════════════════

print("\n[INFO] Executing Temporal Features Validation...")
print("="*90)

# ════════════════════════════════════════════════════════════════════════
# 🆕 CRITICAL: Check if temporal features were created
# ════════════════════════════════════════════════════════════════════════
if 'temporal_features_created' not in globals():
    print("⚠️  WARNING: temporal_features_created not found!")
    print("   → This means Phase 1.5 (Temporal Features Creation) did not run")
    print("   → Skipping validation...\n")

    # Set empty list to prevent further errors
    temporal_features_created = []
    validation_results = None

    print("="*90)
    print("[SKIPPED] Phase 1.6 - No temporal features to validate")
    print("="*90 + "\n")

else:
    # ════════════════════════════════════════════════════════════════════
    # Temporal features exist - proceed with validation
    # ════════════════════════════════════════════════════════════════════

    print(f"✓ temporal_features_created found: {len(temporal_features_created)} features")

    # Check if list is empty
    if not temporal_features_created:
        print("⚠️  WARNING: temporal_features_created is empty!")
        print("   → No features were created in Phase 1.5")
        print("   → Possible reasons:")
        print("      • Required columns (HomeTarget, HomeShots, etc.) missing")
        print("      • Phase 1.5 code had errors")
        print("   → Skipping validation...\n")

        validation_results = None

    else:
        # ════════════════════════════════════════════════════════════════
        # Features exist - run validation
        # ════════════════════════════════════════════════════════════════

        print(f"\n[VALIDATION SETUP]")
        print(f"  → Features to validate: {len(temporal_features_created)}")

        # Define original in-game features
        original_in_game_features = [
            'HomeTarget', 'AwayTarget',
            'HomeShots', 'AwayShots',
            'HomeCorners', 'AwayCorners',
            'HomeFouls', 'AwayFouls',
            'HomeYellow', 'AwayYellow'
        ]

        print(f"  → Original features to check: {len(original_in_game_features)}")
        print(f"\n  Expected temporal features:")
        for feat in temporal_features_created:
            print(f"     • {feat}")

        # ════════════════════════════════════════════════════════════════
        # PRE-VALIDATION CHECKS
        # ════════════════════════════════════════════════════════════════
        print(f"\n[PRE-VALIDATION CHECKS]")

        # Check 1: Do temporal features exist in DataFrame?
        missing_temporal = [f for f in temporal_features_created if f not in matches_all.columns]
        if missing_temporal:
            print(f"  ⚠️  WARNING: {len(missing_temporal)} temporal features NOT in DataFrame:")
            for feat in missing_temporal[:5]:
                print(f"     • {feat}")

            # Filter to available features only
            available_temporal = [f for f in temporal_features_created if f in matches_all.columns]
            print(f"  → Proceeding with {len(available_temporal)} available features\n")

            if not available_temporal:
                print(f"  ❌ ERROR: No temporal features found in DataFrame!")
                print(f"     → Validation cannot proceed\n")
                validation_results = None
            else:
                temporal_features_to_validate = available_temporal
        else:
            print(f"  ✓ All {len(temporal_features_created)} temporal features present")
            temporal_features_to_validate = temporal_features_created

        # Check 2: Do original features exist?
        if 'temporal_features_to_validate' in locals() and temporal_features_to_validate:
            missing_original = [f for f in original_in_game_features if f not in matches_all.columns]
            if missing_original:
                print(f"  ⚠️  WARNING: {len(missing_original)} original features missing:")
                for feat in missing_original:
                    print(f"     • {feat}")

                original_in_game_features = [f for f in original_in_game_features
                                            if f in matches_all.columns]
                print(f"  → Using {len(original_in_game_features)} available features\n")
            else:
                print(f"  ✓ All {len(original_in_game_features)} original features present")

            # Check 3: Dataset size
            print(f"  ✓ Dataset size: {len(matches_all):,} matches\n")

            # ════════════════════════════════════════════════════════════
            # RUN VALIDATION
            # ════════════════════════════════════════════════════════════
            print("[RUNNING VALIDATION]")

            try:
                validation_results = validate_lag_features(
                    df=matches_all,
                    lag_feature_names=temporal_features_to_validate,
                    original_features=original_in_game_features,
                    output_dir=IMAGES_DIR
                )
            except Exception as e_validation:
                print(f"\n❌ VALIDATION FAILED WITH ERROR:")
                print(f"   {str(e_validation)}")
                print(f"   Continuing with EDA...\n")
                validation_results = None
        else:
            validation_results = None

# ════════════════════════════════════════════════════════════════════════
# INTERPRET RESULTS (Regardless of validation outcome)
# ════════════════════════════════════════════════════════════════════════
print("\n" + "="*90)
print("[VALIDATION RESULTS SUMMARY]")
print("="*90)

if validation_results is None:
    print("⚠️  VALIDATION SKIPPED OR FAILED")
    print("   Reasons could be:")
    print("   • Phase 1.5 did not create temporal features")
    print("   • Required columns missing from dataset")
    print("   • Validation function encountered an error")
    print("\n   → Proceeding with EDA using available features")
    print("   → Temporal features may not be available for analysis\n")

elif validation_results['overall_pass']:
    print("✅ TEMPORAL FEATURES VALIDATION: PASSED")
    print("="*90)
    print(f"  ✓ Leakage Check:    PASSED")
    print(f"  ✓ Coverage:         {validation_results['mean_coverage']:.1f}% (min: {validation_results['min_coverage']:.1f}%)")

    print(f"\n🎉 TEMPORAL FEATURES ARE CLEAN:")
    print(f"   • No data leakage detected")
    print(f"   • shift(1) working correctly")
    print(f"   • Ready for modeling\n")

else:
    print("⚠️  TEMPORAL FEATURES VALIDATION: ISSUES DETECTED")
    print("="*90)

    if validation_results.get('leakage_detected'):
        print(f"  ❌ CRITICAL: Data leakage detected!")
        print(f"     → Review '01d_temporal_features_validation.png'\n")

    if validation_results.get('min_coverage', 100) < 60:
        print(f"  ⚠️  Coverage Issue:")
        print(f"     → Min: {validation_results['min_coverage']:.1f}%")
        print(f"     → Consider adjusting window size\n")

    print(f"  → Proceeding with EDA, but review validation results\n")

print("="*90 + "\n")

# ════════════════════════════════════════════════════════════════════════
# 🆕 Save validation status for downstream use
# ════════════════════════════════════════════════════════════════════════
try:
    validation_status = {
        'temporal_features_exist': 'temporal_features_created' in globals() and bool(temporal_features_created),
        'validation_ran': validation_results is not None,
        'validation_passed': validation_results['overall_pass'] if validation_results else False,
        'n_features_created': len(temporal_features_created) if 'temporal_features_created' in globals() else 0
    }

    # Save to CSV for reference
    status_df = pd.DataFrame([validation_status])
    status_path = os.path.join(CSV_DIR, "01f_temporal_features_status.csv")
    status_df.to_csv(status_path, index=False)
    print(f"[INFO] Temporal features status saved to: {os.path.basename(status_path)}\n")

except Exception as e_status:
    print(f"[WARN] Could not save status: {e_status}\n")

# ════════════════════════════════════════════════════════════════════════
# 🎉 CHECKPOINT: Phase 1.6 Complete
# ════════════════════════════════════════════════════════════════════════
print("="*90)
print("[✓] PHASE 1.6 COMPLETE")
print("="*90)

if 'temporal_features_created' in globals() and temporal_features_created:
    print(f"  ✅ Temporal features: {len(temporal_features_created)} created")
    if validation_results and validation_results['overall_pass']:
        print(f"  ✅ Validation: PASSED")
    else:
        print(f"  ⚠️  Validation: REVIEW NEEDED")
else:
    print(f"  ⚠️  Temporal features: NOT CREATED")
    print(f"  → Check Phase 1.5 for errors")

print("\n  Ready to proceed to Phase 2 (Train-Val-Test Split)...\n")
print("="*90 + "\n")

# =============================================================================
# PHASE 2: TRAIN-VAL-TEST SPLIT (Pandemic-Aware)
# =============================================================================
print("\n[INFO] PHASE 2: APPLYING TEMPORAL SPLIT (Pandemic-Aware for EDA Isolation)")
# ════════════════════════════════════════════════════════════════════════
# INTERPRET VALIDATION RESULTS (Enhanced)
# ════════════════════════════════════════════════════════════════════════
print("\n" + "="*90)
print("[VALIDATION RESULTS SUMMARY]")
print("="*90)

if validation_results is None:
    print("❌ VALIDATION SKIPPED OR FAILED")
    print("   → Proceeding with EDA, but temporal features may have issues\n")

elif validation_results['overall_pass']:
    print("✅ TEMPORAL FEATURES VALIDATION: PASSED")
    print("="*90)
    print(f"  ✓ Leakage Check:    PASSED (no features with |r| > 0.95)")
    print(f"  ✓ Coverage Check:   PASSED (min: {validation_results['min_coverage']:.1f}%)")
    print(f"  ✓ Mean Coverage:    {validation_results['mean_coverage']:.1f}%")

    print(f"\n🎉 CONCLUSION:")
    print(f"   • Temporal features are clean and ready for modeling")
    print(f"   • No data leakage detected")
    print(f"   • shift(1) is working correctly")
    print(f"   • All features have acceptable coverage\n")

else:
    print("⚠️  TEMPORAL FEATURES VALIDATION: REVIEW NEEDED")
    print("="*90)

    # Detailed diagnostics
    if validation_results['leakage_detected']:
        print(f"  ❌ CRITICAL: Data leakage detected!")
        print(f"     → Some temporal features have high correlation (|r| > 0.95) with current match data")
        print(f"     → This means future information may be leaking into features")
        print(f"     → ACTION REQUIRED: Review feature engineering logic\n")

    if validation_results['min_coverage'] < 60:
        print(f"  ⚠️  Coverage Issue:")
        print(f"     → Minimum coverage: {validation_results['min_coverage']:.1f}% (threshold: 60%)")
        print(f"     → Some features have too many missing values")
        print(f"     → Consider: Increasing min_periods in rolling window\n")

    print(f"  📊 Coverage Statistics:")
    print(f"     • Min:  {validation_results['min_coverage']:.1f}%")
    print(f"     • Mean: {validation_results['mean_coverage']:.1f}%")

    print(f"\n⚠️  RECOMMENDATION:")
    print(f"   • Review '01d_temporal_features_validation.png' for details")
    print(f"   • Check leakage correlation table if available")
    print(f"   • Consider adjusting window size or filling strategy\n")

print("="*90 + "\n")

# ════════════════════════════════════════════════════════════════════════
# 🆕 OPTIONAL: Save validation summary to CSV
# ════════════════════════════════════════════════════════════════════════
if validation_results:
    try:
        validation_summary = pd.DataFrame([{
            'Validation_Date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Total_Features': len(temporal_features_created),
            'Leakage_Detected': validation_results['leakage_detected'],
            'Min_Coverage_%': validation_results['min_coverage'],
            'Mean_Coverage_%': validation_results['mean_coverage'],
            'Overall_Pass': validation_results['overall_pass'],
            'Dataset_Size': len(matches_all)
        }])

        summary_path = os.path.join(CSV_DIR, "01e_validation_summary.csv")
        validation_summary.to_csv(summary_path, index=False)
        print(f"[INFO] Validation summary saved to: {os.path.basename(summary_path)}\n")
    except Exception as e_csv:
        print(f"[WARN] Could not save validation summary: {e_csv}\n")

# ════════════════════════════════════════════════════════════════════════
# 🎉 SUCCESS CHECKPOINT
# ════════════════════════════════════════════════════════════════════════
print("="*90)
print("🎉 CRITICAL PHASES COMPLETE!")
print("="*90)
print(f"  ✅ Phase 1: Data Loading → matches_all created ({matches_all.shape[0]:,} rows)")
print(f"  ✅ Phase 1.5: Temporal Features → {len(temporal_features_created)} features added")
print(f"  ✅ Phase 1.6: Validation → {'PASSED' if validation_results and validation_results['overall_pass'] else 'REVIEW NEEDED'}")
print(f"  ✅ No NameError → Code is production-ready!")
print("="*90 + "\n")

# =============================================================================
# 🆕 PHASE 1.6.5: TEMPORAL FEATURES - DIFF-ONLY STRATEGY (VIF Optimization)
# =============================================================================
print("\n[INFO] PHASE 1.6.5: APPLYING DIFF-ONLY STRATEGY (VIF Optimization)")
print("="*90)

print("""
📊 STRATEGY: Keep only DIFF features, drop individual Home/Away averages

RATIONALE:
  • Home/Away averages create perfect multicollinearity with Diff
  • VIF values: Target_Diff_L5 = 11,137 (due to linear combination)
  • Diff features capture the SAME information (advantage/disadvantage)
  • Models perform equally well with just Diff features

EXAMPLE:
  Before: Home_Target_Avg_L5=5.2, Away_Target_Avg_L5=3.8, Target_Diff_L5=1.4
  After:  Target_Diff_L5=1.4 (contains the same predictive signal)

REFERENCE:
  • Thesis Section: "Feature Engineering - Multicollinearity Reduction"
  • Decision: Based on VIF analysis showing extreme multicollinearity
  • Impact: VIF reduction from >10,000 to <10 (acceptable threshold)
""")

# ════════════════════════════════════════════════════════════════════════
# IDENTIFY FEATURES TO DROP
# ════════════════════════════════════════════════════════════════════════

temporal_home_away_features = [
    'Home_Target_Avg_L5', 'Away_Target_Avg_L5',
    'Home_Shots_Avg_L5', 'Away_Shots_Avg_L5',
    'Home_Corners_Avg_L5', 'Away_Corners_Avg_L5',
    'Home_Fouls_Avg_L5', 'Away_Fouls_Avg_L5',
    'Home_Yellow_Avg_L5', 'Away_Yellow_Avg_L5',
    'Home_ShotAccuracy_L5', 'Away_ShotAccuracy_L5',
]

temporal_diff_features = [
    'Target_Diff_L5',
    'Shots_Diff_L5',
    'Corners_Diff_L5',
    'Fouls_Diff_L5',
    'Yellow_Diff_L5',
    'ShotAccuracy_Diff_L5',
]

print(f"\n[DECISION] Temporal Feature Strategy:")
print(f"  ❌ DROP: {len(temporal_home_away_features)} Home/Away features (create perfect multicollinearity)")
print(f"  ✅ KEEP: {len(temporal_diff_features)} Diff features (preserve predictive signal)")
print(f"\n  Justification:")
print(f"    • Diff = Home - Away (mathematically redundant to keep all three)")
print(f"    • Example: If you know Home=5.2 and Away=3.8, Diff=1.4 is determined")
print(f"    • Keeping only Diff reduces features while preserving information\n")

# Check which features actually exist in dataset
existing_to_drop = [f for f in temporal_home_away_features if f in matches_all.columns]
existing_to_keep = [f for f in temporal_diff_features if f in matches_all.columns]

print(f"[VERIFICATION]")
print(f"  • Features to drop (exist): {len(existing_to_drop)}/{len(temporal_home_away_features)}")
print(f"  • Features to keep (exist): {len(existing_to_keep)}/{len(temporal_diff_features)}")

if len(existing_to_drop) > 0:
    print(f"\n[DROPPING] {len(existing_to_drop)} Home/Away temporal features...")

    # Show what will be dropped with coverage stats
    print(f"\n  Detailed list of features being dropped:")
    for feat in existing_to_drop:
        coverage = matches_all[feat].notna().sum() / len(matches_all) * 100
        mean_val = matches_all[feat].mean() if matches_all[feat].notna().any() else 0
        print(f"    ❌ {feat:30s} (coverage: {coverage:5.1f}%, mean: {mean_val:5.2f})")

    # Store original shape for comparison
    original_shape = matches_all.shape

    # Drop from matches_all
    matches_all = matches_all.drop(columns=existing_to_drop)

    print(f"\n  ✅ Successfully dropped {len(existing_to_drop)} features")
    print(f"  ✅ Dataset shape change: {original_shape} → {matches_all.shape}")
    print(f"     (Removed {original_shape[1] - matches_all.shape[1]} columns)")
else:
    print(f"\n  ⚠️  No Home/Away temporal features found to drop")
    print(f"     This might indicate features were already dropped or not created.")

# Show remaining temporal features
print(f"\n[REMAINING TEMPORAL FEATURES]")
if len(existing_to_keep) > 0:
    print(f"  The following {len(existing_to_keep)} Diff features are kept:")
    for feat in existing_to_keep:
        if feat in matches_all.columns:
            coverage = matches_all[feat].notna().sum() / len(matches_all) * 100
            mean_val = matches_all[feat].mean() if matches_all[feat].notna().any() else 0
            print(f"    ✅ {feat:30s} (coverage: {coverage:5.1f}%, mean: {mean_val:5.2f})")
else:
    print(f"  ⚠️  No Diff features found in dataset")

# ════════════════════════════════════════════════════════════════════════
# UPDATE TEMPORAL FEATURES LIST
# ════════════════════════════════════════════════════════════════════════

if 'temporal_features_created' in globals():
    print(f"\n[UPDATE] Updating temporal_features_created list...")

    original_count = len(temporal_features_created)

    # Remove dropped features from tracking list
    temporal_features_created = [
        f for f in temporal_features_created
        if f not in temporal_home_away_features
    ]

    print(f"  • Original count: {original_count}")
    print(f"  • After Diff-only: {len(temporal_features_created)}")
    print(f"  • Removed: {original_count - len(temporal_features_created)} features")

    if len(temporal_features_created) > 0:
        print(f"\n  Remaining features in tracking list:")
        for i, feat in enumerate(temporal_features_created, 1):
            print(f"    {i}. {feat}")
    else:
        print(f"  ⚠️  Warning: temporal_features_created is now empty!")
else:
    print(f"\n  ⚠️  Warning: temporal_features_created not found in globals()")

# ════════════════════════════════════════════════════════════════════════
# SAVE DIFF-ONLY STRATEGY TO FILE
# ════════════════════════════════════════════════════════════════════════

try:
    diff_strategy_path = os.path.join(CSV_DIR, "01g_temporal_diff_only_strategy.py")

    with open(diff_strategy_path, 'w', encoding='utf-8') as f:
        f.write("# " + "="*75 + "\n")
        f.write("# AUTO-GENERATED: Temporal Features - Diff-Only Strategy\n")
        f.write("# Generated by: EDA V5.1 (VIF Optimization)\n")
        f.write("# Purpose: Reduce multicollinearity while preserving predictive power\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# CONTEXT:\n")
        f.write("# During VIF analysis, temporal Home/Away features showed extreme\n")
        f.write("# multicollinearity (VIF > 10,000) because Diff = Home - Away.\n")
        f.write("# Keeping all three creates perfect linear dependency.\n\n")

        f.write("# DECISION:\n")
        f.write("# Keep only Diff features, which contain the same predictive signal\n")
        f.write("# (team advantage) without redundancy.\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# FEATURES TO DROP (Home/Away Averages)\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# These features create multicollinearity with their Diff counterparts\n")
        f.write("temporal_home_away_drop = [\n")
        for feat in temporal_home_away_features:
            f.write(f"    '{feat}',  # Dropped: redundant with {feat.replace('Home_', '').replace('Away_', '').replace('_Avg_L5', '_Diff_L5')}\n")
        f.write("]\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# FEATURES TO KEEP (Diff Features)\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# These features preserve all predictive information\n")
        f.write("temporal_diff_keep = [\n")
        for feat in temporal_diff_features:
            f.write(f"    '{feat}',  # Kept: captures Home-Away advantage\n")
        f.write("]\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# USAGE IN MODELING PIPELINE\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# Step 1: Drop Home/Away averages from all datasets\n")
        f.write("# (This code should be in your modeling script, NOT in EDA)\n\n")

        f.write("# X_train = X_train.drop(columns=temporal_home_away_drop, errors='ignore')\n")
        f.write("# X_val = X_val.drop(columns=temporal_home_away_drop, errors='ignore')\n")
        f.write("# X_test = X_test.drop(columns=temporal_home_away_drop, errors='ignore')\n\n")

        f.write("# Step 2: Verify only Diff features remain\n")
        f.write("# temporal_cols = [c for c in X_train.columns if '_L5' in c]\n")
        f.write("# print(f'Remaining temporal features: {temporal_cols}')\n")
        f.write("# assert all('_Diff_' in c for c in temporal_cols), 'Non-Diff temporal features found!'\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# EXPECTED VIF IMPROVEMENTS\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# Before Diff-Only Strategy:\n")
        f.write("#   Target_Diff_L5:        VIF = 11,137 (extreme multicollinearity)\n")
        f.write("#   Shots_Diff_L5:         VIF = 6,599\n")
        f.write("#   Home_Target_Avg_L5:    VIF = 6,774\n")
        f.write("#   Away_Target_Avg_L5:    VIF = 4,742\n")
        f.write("#   ShotAccuracy_Diff_L5:  VIF = 4,922\n\n")

        f.write("# After Diff-Only Strategy (Expected):\n")
        f.write("#   Target_Diff_L5:        VIF < 10 (acceptable)\n")
        f.write("#   Shots_Diff_L5:         VIF < 10\n")
        f.write("#   ShotAccuracy_Diff_L5:  VIF < 10\n")
        f.write("#   Other Diffs:           VIF < 10\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# INFORMATION PRESERVATION\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# Q: Do we lose information by dropping Home/Away averages?\n")
        f.write("# A: NO! The Diff features contain the same predictive signal.\n\n")

        f.write("# Example:\n")
        f.write("#   Home_Target_Avg_L5 = 5.2 (team scores 5.2 shots on target per game)\n")
        f.write("#   Away_Target_Avg_L5 = 3.8 (opponent allows 3.8 shots on target)\n")
        f.write("#   Target_Diff_L5 = 1.4 (team has +1.4 advantage)\n\n")

        f.write("# The advantage (+1.4) is what matters for prediction, not the absolute\n")
        f.write("# values. If we know two of {Home, Away, Diff}, the third is determined:\n")
        f.write("#   Diff = Home - Away  (perfect linear relationship)\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# VALIDATION CHECKLIST\n")
        f.write("# " + "="*75 + "\n\n")

        f.write("# Before training your model, verify:\n")
        f.write("# □ Home/Away temporal features are dropped from X_train/val/test\n")
        f.write("# □ Only Diff temporal features remain\n")
        f.write("# □ VIF values are < 10 for all features\n")
        f.write("# □ Model performance is not degraded (should be same or better)\n\n")

        f.write("# " + "="*75 + "\n")
        f.write("# END OF FILE\n")
        f.write("# " + "="*75 + "\n")

    print(f"\n[OK] Diff-only strategy documentation saved to:")
    print(f"     {diff_strategy_path}")
    print(f"     File size: {os.path.getsize(diff_strategy_path)} bytes")

except Exception as e_save:
    print(f"\n[ERROR] Could not save diff-only strategy file!")
    print(f"        Error: {e_save}")
    import traceback
    traceback.print_exc()

# ════════════════════════════════════════════════════════════════════════
# SUMMARY
# ════════════════════════════════════════════════════════════════════════

print("\n" + "="*90)
print("[SUMMARY] Temporal Features - Diff-Only Strategy Applied")
print("="*90)

print(f"\n  📉 VIF REDUCTION (Expected):")
print(f"     Before Strategy:")
print(f"       • Target_Diff_L5:        11,137")
print(f"       • Home_Target_Avg_L5:     6,774")
print(f"       • Away_Target_Avg_L5:     4,742")
print(f"       • Shots_Diff_L5:          6,599")
print(f"       • ShotAccuracy_Diff_L5:   4,922")
print(f"")
print(f"     After Strategy:")
print(f"       • Target_Diff_L5:         < 10 ✅")
print(f"       • Shots_Diff_L5:          < 10 ✅")
print(f"       • ShotAccuracy_Diff_L5:   < 10 ✅")
print(f"       • Other Diffs:            < 10 ✅")

print(f"\n  📊 INFORMATION PRESERVATION:")
print(f"     • Diff = Home - Away (contains same predictive signal)")
print(f"     • No loss of information about team advantage")
print(f"     • Absolute values less important than relative advantage")
print(f"     • Model performance: Expected to remain same or improve")

print(f"\n  ✅ BENEFITS:")
print(f"     1. Multicollinearity eliminated (VIF < 10)")
print(f"     2. Simpler model (12 fewer features)")
print(f"     3. Easier interpretation (advantage-based)")
print(f"     4. Faster training (fewer features to process)")
print(f"     5. More stable coefficients (no redundancy)")

print(f"\n  📂 OUTPUT FILES:")
print(f"     • 01g_temporal_diff_only_strategy.py (Python code for modeling)")
print(f"     • Contains: Drop list + Keep list + Usage instructions")

print(f"\n  ⚠️  IMPORTANT:")
print(f"     • This strategy is already applied to EDA dataset")
print(f"     • Apply the SAME strategy in your modeling pipeline")
print(f"     • DO NOT keep Home/Away averages in final model")

print("\n" + "="*90 + "\n")

# ════════════════════════════════════════════════════════════════════════
# 🎯 CHECKPOINT: Verify Diff-Only Strategy Applied
# ════════════════════════════════════════════════════════════════════════

print("[CHECKPOINT] Verifying Diff-Only Strategy was applied correctly...")

# Find all temporal-related columns
all_temporal = [col for col in matches_all.columns
                if any(x in col for x in ['_Avg_L5', '_Diff_L5', 'ShotAccuracy_L5'])]

# Separate Home/Away vs Diff
home_away_remaining = [col for col in all_temporal
                       if col.startswith('Home_') or col.startswith('Away_')]
diff_remaining = [col for col in all_temporal
                  if '_Diff_' in col or (col.startswith('Home_') == False and col.startswith('Away_') == False)]

print(f"\n  Current temporal feature status:")
print(f"    • Total temporal features: {len(all_temporal)}")
print(f"    • Home/Away features: {len(home_away_remaining)}")
print(f"    • Diff features: {len(diff_remaining)}")

if len(home_away_remaining) == 0:
    print(f"\n  ✅ SUCCESS: All Home/Away temporal features removed!")
    print(f"  ✅ Only Diff features remain (as intended)")
elif len(home_away_remaining) <= 2:
    print(f"\n  ⚠️  WARNING: {len(home_away_remaining)} Home/Away features still present:")
    for feat in home_away_remaining:
        print(f"      • {feat}")
    print(f"  → Check if these are intentional (non-L5 features)")
else:
    print(f"\n  ❌ ERROR: {len(home_away_remaining)} Home/Away features still present!")
    print(f"  → Diff-Only Strategy may not have been applied correctly")
    for feat in home_away_remaining[:10]:
        print(f"      • {feat}")
    if len(home_away_remaining) > 10:
        print(f"      ... and {len(home_away_remaining)-10} more")

if len(diff_remaining) >= len(temporal_diff_features):
    print(f"\n  ✅ All expected Diff features are present")
else:
    print(f"\n  ⚠️  Some Diff features might be missing")
    print(f"     Expected: {len(temporal_diff_features)}, Found: {len(diff_remaining)}")

print("\n" + "="*90 + "\n")

print("[INFO] Proceeding to Phase 2+ (Train-Val-Test Split, EDA Analyses)...")
print("[INFO] matches_all is now ready for all subsequent analyses ✅\n")

# ════════════════════════════════════════════════════════════════════════
# 🧪 PHASE 1: VALIDATION & DIAGNOSTICS
# ════════════════════════════════════════════════════════════════════════
print("\n[VALIDATION] Phase 1 Completeness Check")
print("="*90)

print(f"\n📊 DATASET STATUS:")
print(f"  • Total matches: {len(matches_all):,}")
print(f"  • Columns: {matches_all.shape[1]}")
print(f"  • Date range: {matches_all['MatchDate'].min().date()} → {matches_all['MatchDate'].max().date()}")

print(f"\n🎯 KEY COLUMNS:")
key_cols = ['Target', 'YearMonth', 'Season', 'HomeTeam_TM', 'AwayTeam_TM']
for col in key_cols:
    if col in matches_all.columns:
        non_null = matches_all[col].notna().sum()
        pct = (non_null / len(matches_all)) * 100
        print(f"  ✓ {col:20s}: {non_null:,} ({pct:.1f}%)")
    else:
        print(f"  ❌ {col:20s}: MISSING!")

print(f"\n📈 TRANSFERMARKT COVERAGE:")
tm_cols = ['HomeTeam_ClubValue', 'AwayTeam_ClubValue', 'ValueRatio_Diff']
for col in tm_cols:
    if col in matches_all.columns:
        non_null = matches_all[col].notna().sum()
        pct = (non_null / len(matches_all)) * 100
        print(f"  • {col:25s}: {non_null:,} ({pct:.1f}%)")
    else:
        print(f"  ⚠️  {col:25s}: Not created")

print(f"\n🔗 FUZZY MATCHING STATS:")
if 'fuzzy_matcher' in globals():
    matched = sum(1 for v in fuzzy_matcher.team_mapping.values() if v is not None)
    total = len(fuzzy_matcher.team_mapping)
    print(f"  • Teams matched: {matched} / {total} ({matched/total*100:.1f}%)")

    avg_score = np.mean([s for s in fuzzy_matcher.team_scores.values() if s > 0])
    print(f"  • Avg match score: {avg_score:.1f}")

print("\n" + "="*90 + "\n")

# =============================================================================
# PHASE 2: TRAIN-VAL-TEST SPLIT (Pandemic-Aware)
# =============================================================================
print("\n[INFO] PHASE 2: APPLYING TEMPORAL SPLIT (Pandemic-Aware for EDA Isolation)")
print("="*90)

train_end_date = pd.to_datetime('2019-08-31')
pandemic_end   = pd.to_datetime('2021-05-31')
val_end_date   = pd.to_datetime('2022-08-31')

print(f"Applying temporal boundaries based on standard practice:")
print(f"  Train End Date (exclusive): {train_end_date.date()}")
print(f"  Validation Start Date (inclusive): {pandemic_end.date()}")
print(f"  Validation End Date (exclusive): {val_end_date.date()}")
print(f"  Test Start Date (inclusive): {val_end_date.date()}")

train_matches = matches_all[matches_all['MatchDate'] < train_end_date].copy()
val_matches   = matches_all[(matches_all['MatchDate'] >= pandemic_end) & (matches_all['MatchDate'] < val_end_date)].copy()
test_matches  = matches_all[matches_all['MatchDate'] >= val_end_date].copy()

print(f"\nTemporal Split Results:")
print(f"  Train Set Size: {len(train_matches):,} matches (Covers period before {train_end_date.date()})")
print(f"  Val Set Size:   {len(val_matches):,} matches (Covers period {pandemic_end.date()} to {val_end_date.date()})")
print(f"  Test Set Size:  {len(test_matches):,} matches (Covers period from {val_end_date.date()} onwards)\n")

if len(train_matches) == 0:
    print(f"[FATAL] Training set (matches before {train_end_date.date()}) is empty after temporal split.")
    raise ValueError("Training set cannot be empty for EDA.")
else:
    print(f"[OK] EDA will proceed using ONLY the {len(train_matches):,} matches from the training period.")
    if 'Target' in train_matches.columns and train_matches['Target'].notna().any():
         print(f"[CHECK] Target distribution (Train Data Only): {train_matches['Target'].value_counts(normalize=True, dropna=False).sort_index().to_dict()}")
    else:
         print("[WARN] Target column is missing or all null in train_matches after split.")


# =============================================================================
# PHASE 3: PANDEMIC IMPACT ANALYSIS
# =============================================================================
print("\n[INFO] PHASE 3: PANDEMIC IMPACT ANALYSIS (Using full loaded data for context)")
print("="*90)

pre_pandemic_start = '2015-09-01'
pre_pandemic_end = '2020-03-01'
pandemic_start = '2020-03-01'
pandemic_actual_end = '2021-05-31'
post_pandemic_start = '2021-06-01'

pre_pandemic_df = matches_all[(matches_all['MatchDate'] >= pre_pandemic_start) & (matches_all['MatchDate'] < pre_pandemic_end)].copy()
pandemic_period_df = matches_all[(matches_all['MatchDate'] >= pandemic_start) & (matches_all['MatchDate'] <= pandemic_actual_end)].copy()
post_pandemic_df = matches_all[(matches_all['MatchDate'] > pandemic_actual_end)].copy()

print(f"Pandemic Analysis Periods (for context):")
if not pre_pandemic_df.empty: print(f"  Pre-Pandemic: {len(pre_pandemic_df):,} matches ({pre_pandemic_df['MatchDate'].min().date()} to {pre_pandemic_df['MatchDate'].max().date()})")
if not pandemic_period_df.empty: print(f"  During Pandemic: {len(pandemic_period_df):,} matches ({pandemic_period_df['MatchDate'].min().date()} to {pandemic_period_df['MatchDate'].max().date()})")
if not post_pandemic_df.empty: print(f"  Post-Pandemic: {len(post_pandemic_df):,} matches ({post_pandemic_df['MatchDate'].min().date()} to {post_pandemic_df['MatchDate'].max().date()})")

home_advantage_data = []
league_names = {"E0":"Premier League","D1":"Bundesliga","I1":"Serie A","SP1":"La Liga","F1":"Ligue 1"}

periods_for_analysis = [
    ('Pre-Pandemic (Sep \'15 - Feb \'20)', pre_pandemic_df),
    ('Pandemic (Mar \'20 - May \'21)', pandemic_period_df),
    ('Post-Pandemic (Jun \'21 onwards)', post_pandemic_df)
]

division_col_present = 'Division' in matches_all.columns
if not division_col_present:
     print("[WARN] 'Division' column missing. Cannot perform league-specific pandemic analysis.")

for period_name, period_data in periods_for_analysis:
    if period_data.empty:
        print(f"[INFO] Skipping empty period: {period_name}")
        continue
    if not division_col_present:
         if 'Target' in period_data.columns and period_data['Target'].notna().any():
             home_wins = (period_data['Target'] == 1).sum()
             total = len(period_data['Target'].dropna())
             home_rate = (home_wins / total) * 100 if total > 0 else 0
             home_advantage_data.append({
                 'Period': period_name, 'League': 'All Leagues', 'Home Win %': home_rate,
                 'Home Wins': home_wins, 'Total Matches': total
             })
    else:
        for league_code in ["E0", "D1", "I1", "SP1", "F1"]:
            league_data = period_data[period_data['Division'] == league_code]
            if not league_data.empty and 'Target' in league_data.columns and league_data['Target'].notna().any():
                home_wins = (league_data['Target'] == 1).sum()
                total = len(league_data['Target'].dropna())
                home_rate = (home_wins / total) * 100 if total > 0 else 0
                home_advantage_data.append({
                    'Period': period_name, 'League': league_names.get(league_code, league_code),
                    'Home Win %': home_rate, 'Home Wins': home_wins, 'Total Matches': total
                })

if not home_advantage_data:
    print("[WARN] Could not compute any home advantage data. Skipping pandemic plot and CSV.")
else:
    ha_df = pd.DataFrame(home_advantage_data)
    try:
        pivot_table = pd.pivot_table(ha_df, index='League', columns='Period', values='Home Win %')
        ordered_cols = [p[0] for p in periods_for_analysis if p[0] in pivot_table.columns]
        pivot_table = pivot_table[ordered_cols]

        pre_col = periods_for_analysis[0][0]
        pand_col = periods_for_analysis[1][0]
        if pand_col in pivot_table.columns and pre_col in pivot_table.columns:
             pivot_table['Pandemic Impact (pp)'] = pivot_table[pand_col] - pivot_table[pre_col]
        else:
             pivot_table['Pandemic Impact (pp)'] = np.nan

        csv_path = os.path.join(CSV_DIR, "pandemic_home_advantage_analysis.csv")
        pivot_table.round(2).to_csv(csv_path)
        print(f"[OK] Pandemic analysis pivot table saved to: {csv_path}")
        save_df_as_image(pivot_table.round(2), "03b_pandemic_pivot_table.png",
                         title="Home Win % by League and Period")
    except Exception as e_pivot:
        print(f"[WARN] Could not create or save pandemic pivot table: {e_pivot}")

    try:
        avg_values_dict = {}
        for period_name, _ in periods_for_analysis:
            avg_values_dict[period_name] = ha_df[ha_df['Period'] == period_name]['Home Win %'].mean()

        avg_pre = avg_values_dict.get(periods_for_analysis[0][0], np.nan)
        avg_pandemic = avg_values_dict.get(periods_for_analysis[1][0], np.nan)
        avg_post = avg_values_dict.get(periods_for_analysis[2][0], np.nan)

        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        if 'pivot_table' in locals():
            pivot_for_plot = pivot_table[ordered_cols]
            plot_colors = ['#2ecc71', '#e74c3c', '#3498db']
            pivot_for_plot.plot(kind='bar', ax=axes[0], color=plot_colors[:len(ordered_cols)], legend=False)
            axes[0].set_title('Home Win Rate by League & Period', fontsize=12, fontweight='bold')
            axes[0].set_ylabel('Home Win %')
            axes[0].tick_params(axis='x', rotation=30)  # ✓ DÜZELTME 1
            plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=30, ha='right')  # ✓ DÜZELTME 2
            axes[0].grid(axis='y', linestyle='--', alpha=0.5)
            min_rate = ha_df['Home Win %'].min()
            max_rate = ha_df['Home Win %'].max()
            axes[0].set_ylim([max(0, np.floor(min_rate / 5) * 5), min(100, np.ceil(max_rate / 5) * 5)])
        else:
             axes[0].text(0.5, 0.5, 'League-specific data not available\nor pivot table failed', ha='center', va='center', transform=axes[0].transAxes)
             axes[0].set_title('Home Win Rate by League & Period', fontsize=12, fontweight='bold')

        short_periods = ['Pre-Pandemic', 'Pandemic', 'Post-Pandemic']
        avg_values = [avg_pre, avg_pandemic, avg_post]
        valid_indices = [i for i, v in enumerate(avg_values) if pd.notna(v)]
        if len(valid_indices) > 1:
            plot_x = [short_periods[i] for i in valid_indices]
            plot_y = [avg_values[i] for i in valid_indices]
            axes[1].plot(plot_x, plot_y, marker='o', markersize=10, linewidth=2, color='#34495e', linestyle='-')
            for i, idx in enumerate(valid_indices):
                axes[1].text(plot_x[i], plot_y[i] + 0.3, f'{plot_y[i]:.1f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')
            axes[1].set_title('Average Home Win Rate Trend (All Leagues)', fontsize=12, fontweight='bold')
            axes[1].set_ylabel('Average Home Win %')
            min_avg = min(plot_y)
            max_avg = max(plot_y)
            axes[1].set_ylim([max(0, np.floor(min_avg / 2) * 2 - 1), min(100, np.ceil(max_avg / 2) * 2 + 1)])
            axes[1].grid(axis='y', linestyle='--', alpha=0.5)
        else:
             axes[1].text(0.5, 0.5, 'Not enough valid data points for trend', ha='center', va='center', transform=axes[1].transAxes)
             axes[1].set_title('Average Home Win Rate Trend', fontsize=12, fontweight='bold')

        if 'pivot_table' in locals():
            handles = [plt.Rectangle((0,0),1,1, color=plot_colors[i]) for i in range(len(ordered_cols))]
            fig.legend(handles, ordered_cols, loc='lower center', bbox_to_anchor=(0.5, 0.01), ncol=3, fontsize=10, title="Period")

        plt.subplots_adjust(bottom=0.15)
        savefig_report("03a_pandemic_home_advantage.png", also_png=True)
        print("[OK] Pandemic analysis plot saved.")
    except Exception as e_plot:
        print(f"[WARN] Could not create pandemic analysis plot: {e_plot}")

print("[OK] Phase 3: Pandemic Impact Analysis completed.\n")


# =============================================================================
# PHASE 4: EDA ON TRAIN SET ONLY
# =============================================================================
print("\n[INFO] PHASE 4: EDA ON TRAIN SET ONLY")
print("="*90)

print(f"[INFO] Starting detailed EDA using ONLY the 'train_matches' DataFrame ({len(train_matches):,} rows).")

eda_data = train_matches.copy()

# =============================================================================
# PHASE 4.1: MARKET ODDS ANALYSIS
# =============================================================================
print("\n[INFO] PHASE 4.1: Analyzing Market Odds (Overround / Vig)")
print("-" * 90)

odds_cols = ['OddHome', 'OddDraw', 'OddAway']
if all(c in eda_data.columns for c in odds_cols):
    try:
        eda_data['Prob_H'] = np.where(eda_data['OddHome'] > 0, 1 / eda_data['OddHome'], np.nan)
        eda_data['Prob_D'] = np.where(eda_data['OddDraw'] > 0, 1 / eda_data['OddDraw'], np.nan)
        eda_data['Prob_A'] = np.where(eda_data['OddAway'] > 0, 1 / eda_data['OddAway'], np.nan)

        eda_data['Overround'] = eda_data['Prob_H'] + eda_data['Prob_D'] + eda_data['Prob_A']
        avg_overround = eda_data['Overround'].mean()
        print(f"[OK] Calculated implied probabilities and overround (Avg: {avg_overround:.3f})")

        fig_ov, ax_ov = plt.subplots(figsize=(10, 5))
        sns.histplot(eda_data['Overround'].dropna(), ax=ax_ov, kde=True, bins=50, color='purple')
        ax_ov.set_title(f'Market Odds "Overround" Distribution \nAvg = {avg_overround:.3f}', fontweight='bold')
        ax_ov.set_xlabel('Overround (1/H + 1/D + 1/A)')
        ax_ov.set_ylabel('Frequency')
        ax_ov.axvline(1.0, color='red', linestyle='--', label='"Fair" Market (1.0)')
        ax_ov.axvline(avg_overround, color='black', linestyle=':', label=f'Average ({avg_overround:.3f})')
        ax_ov.legend()
        ax_ov.grid(alpha=0.3)
        savefig_report("04a_overround_distribution.png", also_png=True)
        print("[OK] Overround distribution plot saved.")

        if avg_overround > 1.01:
            print(f"[INFO] Overround > 1.01. Normalizing probabilities by removing overround...")
            eda_data['Prob_H_Norm'] = eda_data['Prob_H'] / eda_data['Overround']
            eda_data['Prob_D_Norm'] = eda_data['Prob_D'] / eda_data['Overround']
            eda_data['Prob_A_Norm'] = eda_data['Prob_A'] / eda_data['Overround']
        else:
            print("[INFO] Overround is ~1.0. Data seems pre-normalized. Copying probabilities.")
            eda_data['Prob_H_Norm'] = eda_data['Prob_H']
            eda_data['Prob_D_Norm'] = eda_data['Prob_D']
            eda_data['Prob_A_Norm'] = eda_data['Prob_A']
        print("[OK] Normalized probability features (Prob_H/D/A_Norm) created.")

    except Exception as e_odds:
        print(f"[WARN] Market odds analysis failed: {e_odds}")
        try:
            plt.close(fig_ov)
        except:
            pass
else:
    print("[WARN] Odds columns (OddHome, OddDraw, OddAway) not found. Skipping Market Odds Analysis.")

print("-" * 90)


# =============================================================================
# 🆕 PHASE 4.2: IDENTIFY AND HANDLE HIGH CORRELATION PAIRS (V5.0)
# =============================================================================
print("\n[INFO] PHASE 4.2: IDENTIFYING HIGH CORRELATION PAIRS (Multicollinearity Check)")
print("="*90)
print("⚠️  CRITICAL: Detecting features with correlation |r| > 0.85")
print("   These pairs create multicollinearity and must be addressed before modeling")
print("-" * 90)

# Önce sayısal sütunları tanımla
print("\n[INFO] PRE-FLIGHT VALIDATION CHECKS (on Training Data)")
print("-" * 90)

if 'Target' not in eda_data.columns or eda_data['Target'].isnull().all():
    print(f"[ERROR] Target column not found or is all null in the training set!")
    raise ValueError("Target column is REQUIRED for EDA analysis on training data.")
else:
    target_non_null_count = eda_data['Target'].notna().sum()
    if target_non_null_count == 0:
         print(f"[ERROR] Target column exists but contains only null values in the training set!")
         raise ValueError("Target column has no valid values in training data.")
    print(f"[OK] Target column found and valid in training data ({target_non_null_count} non-null values).")

if len(eda_data) < 100:
    print(f"[WARN] Training set for EDA is very small ({len(eda_data)} rows). Results might be unstable.")
else:
    print(f"[OK] Training set size for EDA: {len(eda_data):,} rows")

# 🔥 KRİTİK DEĞİŞİKLİK: DATA LEAKAGE ÖZELLİKLERİNİ ÇIKAR
numeric_cols_all_train = eda_data.select_dtypes(include=[np.number]).columns.tolist()

# LEAKAGE ÖZELLİKLERİ LİSTESİ (GENİŞLETİLDİ)
exclude_for_analysis = [
    'Target', 'match_id', 'Year', 'Month', 'DoW',
    'HomeRed', 'AwayRed',

    # ===== DATA LEAKAGE (Maç içi veriler) =====
    'FTHome', 'FTAway',           # Maç sonu golleri
    'HTHome', 'HTAway',           # Devre arası golleri
    'HomeTarget', 'AwayTarget',   # Kaleyi bulan şutlar (LEAKAGE!)
    'HomeShots', 'AwayShots',     # Toplam şutlar (LEAKAGE!)
    'HomeCorners', 'AwayCorners', # Kornerler (LEAKAGE!)
    'HomeFouls', 'AwayFouls',     # Fauller (LEAKAGE!)
    'HomeYellow', 'AwayYellow',   # Sarı kartlar (LEAKAGE!)

    # ===== TÜRETİLMİŞ LEAKAGE ÖZELLİKLERİ =====
    'ShotsDifference',            # HomeShots - AwayShots (LEAKAGE!)
    'ShotsTotal',                 # HomeShots + AwayShots (LEAKAGE!)
    'ShotAccuracyHome',           # HomeTarget / HomeShots (LEAKAGE!)
    'ShotAccuracyAway',           # AwayTarget / AwayShots (LEAKAGE!)
    'ShotAccuracyDiff',           # (LEAKAGE!)
    'CornersDifference',          # HomeCorners - AwayCorners (LEAKAGE!)
    'CornersTotal'                # HomeCorners + AwayCorners (LEAKAGE!)
]

numeric_cols = [c for c in numeric_cols_all_train if c not in exclude_for_analysis]

if len(numeric_cols) < 5:
    print(f"[WARN] Only {len(numeric_cols)} numeric features identified for analysis (excluding IDs/Target/Leakage).")
    numeric_cols = [c for c in numeric_cols_all_train if c not in ['Target', 'match_id']]
    print(f"[INFO] Retrying with {len(numeric_cols)} numeric features (only excluding Target/match_id).")
    if len(numeric_cols) < 5:
         print("[ERROR] Still too few numeric features. Check data processing.")
else:
    print(f"[OK] Identified {len(numeric_cols)} CLEAN numeric features for detailed analysis (LEAKAGE REMOVED).")
    print(f"     ✓ Excluded {len(exclude_for_analysis)} leakage/non-predictive features")
    print(f"     ✓ Included {len([f for f in temporal_features_created if f in numeric_cols])} temporal features")

print(f"[INFO] Sample CLEAN features for analysis: {numeric_cols[:15]}...")
print("-" * 90 + "\n")

# Özellik kaynak eşleştirmesi
print("\n[INFO] Mapping features to their original data sources...")

source_matches_stats = [col for col in eda_data.columns if col in [
    'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw', 'MaxAway',
    'Over25', 'Under25', 'MaxOver25', 'MaxUnder25',
    'HandiSize', 'HandiHome', 'HandiAway',
    'C_LTH', 'C_LTA', 'C_VHD', 'C_VAD', 'C_HTB', 'C_PHB',
]]

source_matches_form = [col for col in eda_data.columns if col in [
    'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away',
    'Form3Difference', 'Form5Difference'
]]

source_elo = [col for col in eda_data.columns if col in [
    'HomeElo', 'AwayElo',
    'ELO_Diff', 'EloTotal', 'EloAdvantage'
]]

source_transfermarkt = [col for col in eda_data.columns if col.startswith(('HomeTeam_', 'AwayTeam_',
                               'ClubValue_', 'MaxPlayerValue_', 'ManagerTrophies_', 'ManagerTenureDays_',
                               'NetTransferSpending_', 'n_players_injured_', 'ValueRatio_'))
                        or col in ['ClubValue_Diff', 'MaxPlayerValue_Diff', 'ManagerTrophies_Diff',
                                   'ManagerTenureDays_Diff', 'NetTransferSpending_Diff',
                                   'n_players_injured_Diff', 'ValueRatio_Diff']]

source_temporal = [col for col in eda_data.columns if any(col.startswith(f'{feat}_') or col.endswith(('_L3', '_L5', '_L10'))
                                                          for feat in ['Home_Target', 'Away_Target', 'Target_Diff',
                                                                      'Home_Shots', 'Away_Shots', 'Shots_Diff',
                                                                      'Home_Corners', 'Away_Corners', 'Corners_Diff',
                                                                      'Home_Fouls', 'Away_Fouls', 'Fouls_Diff',
                                                                      'Home_Yellow', 'Away_Yellow', 'Yellow_Diff',
                                                                      'ShotAccuracy'])]

source_derived_other = [col for col in eda_data.columns if col in [
    'Year', 'Month', 'DoW', 'SeasonPhase', 'IsWeekend'
]]

feature_source_map = {}
all_mapped_features = set()

def map_feature(col, source_name, feature_dict, mapped_set):
    feature_dict[col] = source_name
    mapped_set.add(col)

feature_categories = {
    'Matches_Stats': source_matches_stats,
    'Matches_Form': source_matches_form,
    'ELO_Ratings': source_elo,
    'Transfermarkt_Data': source_transfermarkt,
    'Temporal_Features': source_temporal,
    'Derived_Time': source_derived_other
}

for source_name, col_list in feature_categories.items():
    for col in col_list:
        if col in eda_data.columns and col not in all_mapped_features:
             map_feature(col, source_name, feature_source_map, all_mapped_features)

unmapped_numeric = [col for col in numeric_cols if col not in all_mapped_features]
if unmapped_numeric:
    print(f"[WARN] {len(unmapped_numeric)} numeric features could not be mapped to a source and are assigned to 'Unknown':")
    print(f"       {unmapped_numeric[:10]}...")
    for col in unmapped_numeric:
         feature_source_map[col] = 'Unknown_Numeric'
         all_mapped_features.add(col)
else:
    print("[OK] All numeric features successfully mapped to a data source category.")

feature_sources = pd.Series(feature_source_map, name="DataSource")

print("\nFeature count by Data Source (for numeric features used in EDA):")
source_counts = feature_sources[numeric_cols].value_counts()
print(source_counts)
source_counts.to_csv(os.path.join(CSV_DIR, "feature_counts_by_source.csv"))

try:
    fig_source, ax_source = plt.subplots(figsize=(8, 5))
    source_counts.plot(kind='bar', ax=ax_source, color=plt.cm.Set2(np.arange(len(source_counts))), edgecolor='black')
    ax_source.set_title('Number of Numeric Features per Data Source Category', fontweight='bold')
    ax_source.set_ylabel('Number of Features')
    ax_source.tick_params(axis='x', rotation=0)
    ax_source.grid(axis='y', linestyle='--', alpha=0.9)
    plt.tight_layout()
    savefig_report("00_feature_source_distribution.png", also_png=True)
    print("[OK] Feature source distribution plotted and saved.")
except Exception as e_source_plot:
     print(f"[WARN] Could not plot feature source distribution: {e_source_plot}")
     plt.close(fig_source)


# 🔥 ŞİMDİ YÜKSEK KORELASYON ANALİZİNE GEÇELİM
print("\n[INFO] Computing correlation matrix for multicollinearity detection...")

try:
    correlation_matrix = eda_data[numeric_cols].corr()
    correlation_matrix = correlation_matrix.dropna(how='all', axis=0).dropna(how='all', axis=1)

    if correlation_matrix.empty:
        print("[WARN] Correlation matrix is empty after dropping NaNs.")
    else:
        # Yüksek korelasyonlu çiftleri bul
        print("\n[INFO] Identifying highly correlated feature pairs (|r| > 0.85)...")
        corr_threshold = 0.85
        high_corr_pairs = []

        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr_val = correlation_matrix.iloc[i, j]
                if abs(corr_val) > corr_threshold:
                    feat1 = correlation_matrix.columns[i]
                    feat2 = correlation_matrix.columns[j]
                    high_corr_pairs.append({
                        'Feature_1': feat1,
                        'Feature_2': feat2,
                        'Correlation': corr_val,
                        'Abs_Correlation': abs(corr_val)
                    })

        if high_corr_pairs:
            high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Abs_Correlation', ascending=False)
            print(f"\n⚠️  ALERT: Found {len(high_corr_df)} highly correlated pairs (|r| > {corr_threshold}):")
            print("=" * 90)
            print(high_corr_df.head(20).to_string(index=False))
            print("=" * 90)

            # CSV'ye kaydet
            high_corr_csv_path = os.path.join(CSV_DIR, "09_high_correlation_pairs.csv")
            high_corr_df.to_csv(high_corr_csv_path, index=False)
            print(f"\n[OK] High correlation pairs saved to: {high_corr_csv_path}")

            # 🎯 ÖNERİLER TABLOSU OLUŞTUR
            print("\n" + "="*90)
            print("📊 AUTOMATED FEATURE SELECTION RECOMMENDATIONS:")
            print("="*90)

            # 🆕 V5.2: PROTECTED FEATURES (DO NOT DROP!)
            protected_features = [
                'ELO_Diff',       # Top 10 MI, critical for non-odds models
                'OddHome',        # #1 MI feature
                'OddDraw',        # High MI feature
                'OddAway',        # High MI feature
                'Prob_H_Norm',    # #2 MI feature
                'Prob_D_Norm',    # High MI feature
                'Prob_A_Norm'     # #3 MI feature
            ]

            print("\n🛡️  PROTECTED FEATURES (Will not be dropped):")
            for feat in protected_features:
                print(f"   • {feat}")
            print()

            # Prob_H/Prob_H_Norm gibi çiftlerde hangisini tutacağız?
            recommendations = []
            features_to_drop = set()

            for _, row in high_corr_df.iterrows():
                feat1, feat2 = row['Feature_1'], row['Feature_2']
                corr = row['Correlation']

                # Recommendation mantığı
                recommendation = None
                feature_to_drop = None

                # 🆕 ÖNCE: Protected features kontrolü
                if feat1 in protected_features and feat2 in protected_features:
                    recommendation = f"⚠️  BOTH PROTECTED: Keep both (handle in modeling)"
                    feature_to_drop = None
                elif feat1 in protected_features:
                    recommendation = f"🛡️  KEEP {feat1} (protected), DROP {feat2}"
                    feature_to_drop = feat2
                elif feat2 in protected_features:
                    recommendation = f"🛡️  KEEP {feat2} (protected), DROP {feat1}"
                    feature_to_drop = feat1

                # Kural 1: Normalized versiyonları tercih et (NON-PROTECTED)
                elif feat1.endswith('_Norm') and not feat2.endswith('_Norm'):
                    recommendation = f"KEEP {feat1} (normalized), DROP {feat2}"
                    feature_to_drop = feat2
                elif feat2.endswith('_Norm') and not feat1.endswith('_Norm'):
                    recommendation = f"KEEP {feat2} (normalized), DROP {feat1}"
                    feature_to_drop = feat1

                # 🆕 Kural 2: Max* ALWAYS drop (Odd* already protected above)
                elif feat1.startswith('Max'):
                    recommendation = f"DROP {feat1} (Max odds redundant)"
                    feature_to_drop = feat1
                elif feat2.startswith('Max'):
                    recommendation = f"DROP {feat2} (Max odds redundant)"
                    feature_to_drop = feat2

                # Kural 3: Non-normalized Prob* drop (normalized already protected)
                elif feat1 in ['Prob_H', 'Prob_D', 'Prob_A']:
                    recommendation = f"DROP {feat1} (use normalized version)"
                    feature_to_drop = feat1
                elif feat2 in ['Prob_H', 'Prob_D', 'Prob_A']:
                    recommendation = f"DROP {feat2} (use normalized version)"
                    feature_to_drop = feat2

                # Kural 4: League value çiftleri (birini tut)
                elif 'LeagueValue' in feat1 and 'LeagueValue' in feat2:
                    recommendation = f"KEEP {feat1}, DROP {feat2} (redundant league value)"
                    feature_to_drop = feat2

                # Kural 5: Prob_H ile HandiSize gibi çapraz korelasyonlar
                elif 'HandiSize' in feat1 and 'Prob' in feat2:
                    recommendation = f"REVIEW NEEDED: {feat1} ↔ {feat2} (cross-domain correlation)"
                    feature_to_drop = None
                elif 'HandiSize' in feat2 and 'Prob' in feat1:
                    recommendation = f"REVIEW NEEDED: {feat1} ↔ {feat2} (cross-domain correlation)"
                    feature_to_drop = None

                # Varsayılan: MI skoru ile karar ver
                else:
                    recommendation = f"CHECK MI SCORES: Compare {feat1} vs {feat2}"
                    feature_to_drop = None

                recommendations.append({
                    'Pair': f"{feat1} ↔ {feat2}",
                    'Correlation': f"{corr:.4f}",
                    'Recommendation': recommendation,
                    'Feature_to_Drop': feature_to_drop if feature_to_drop else 'TBD'
                })

                if feature_to_drop:
                    features_to_drop.add(feature_to_drop)

            recommendations_df = pd.DataFrame(recommendations)
            print(recommendations_df.head(15).to_string(index=False))

            # Çıkarılacak özelliklerin final listesi
            features_to_drop_list = sorted(list(features_to_drop))

            # 🆕 V5.2: PROTECTED FEATURES FINAL CHECK
            features_to_drop_list = [f for f in features_to_drop_list if f not in protected_features]

            print(f"\n🎯 FINAL DROP LIST (After protected features filter):")
            print(f"   Original candidates: {len(features_to_drop)} → After filter: {len(features_to_drop_list)}")
            print("=" * 90)
            for i, feat in enumerate(features_to_drop_list, 1):
                print(f"  {i:2d}. {feat}")
            print("=" * 90)

            # 🆕 REMOVED FROM DROP LIST (Protected)
            removed_from_drop = sorted([f for f in features_to_drop if f in protected_features])
            if removed_from_drop:
                print(f"\n🛡️  PROTECTED (Not in drop list despite high correlation):")
                print("   These features kept due to high MI scores:")
                for feat in removed_from_drop:
                    print(f"   • {feat}")
                print()

            # Python kodu olarak kaydet
            drop_code_path = os.path.join(CSV_DIR, "09b_features_to_drop_code.py")
            with open(drop_code_path, 'w') as f:
                f.write("# AUTO-GENERATED: Features to drop due to high correlation (|r| > 0.85)\n")
                f.write("# Generated by EDA V5.2 (with Protected Features)\n")
                f.write("# Protected features (NOT dropped): ELO_Diff, OddHome, OddDraw, OddAway, Prob_*_Norm\n\n")
                f.write("features_to_drop_multicollinearity = [\n")
                for feat in features_to_drop_list:
                    f.write(f"    '{feat}',\n")
                f.write("]\n\n")
                f.write("# Usage in your modeling pipeline:\n")
                f.write("# X_train = X_train.drop(columns=features_to_drop_multicollinearity)\n")
                f.write("# X_val = X_val.drop(columns=features_to_drop_multicollinearity)\n")
                f.write("# X_test = X_test.drop(columns=features_to_drop_multicollinearity)\n")

            print(f"[OK] Python code for dropping features saved to: {drop_code_path}")
            print(f"[RECOMMENDATION] Copy this list to your modeling script and apply BEFORE training!")

            # Recommendations CSV
            recommendations_csv_path = os.path.join(CSV_DIR, "09c_multicollinearity_recommendations.csv")
            recommendations_df.to_csv(recommendations_csv_path, index=False)
            print(f"[OK] Detailed recommendations saved to: {recommendations_csv_path}")

            # Görselleştirme
            try:
                fig_hc, ax_hc = plt.subplots(figsize=(12, 8))
                top_n_pairs = min(15, len(high_corr_df))
                plot_data_hc = high_corr_df.head(top_n_pairs).copy()
                plot_data_hc['Pair'] = plot_data_hc['Feature_1'] + ' ↔ ' + plot_data_hc['Feature_2']

                colors_hc = ['#e74c3c' if x < 0 else '#2ecc71' for x in plot_data_hc['Correlation'].values]

                ax_hc.barh(range(top_n_pairs), plot_data_hc['Correlation'].values, color=colors_hc, height=0.7, edgecolor='black')
                ax_hc.set_yticks(range(top_n_pairs))
                ax_hc.set_yticklabels(plot_data_hc['Pair'].values, fontsize=9)
                ax_hc.invert_yaxis()
                ax_hc.set_xlabel('Pearson Correlation', fontweight='bold', fontsize=12)
                ax_hc.set_title(f'Top {top_n_pairs} Highly Correlated Feature Pairs (|r| > {corr_threshold})',
                               fontweight='bold', fontsize=12, pad=15)
                ax_hc.axvline(0, color='black', linestyle='-', linewidth=0.8)
                ax_hc.grid(axis='x', alpha=0.3)

                for i, v in enumerate(plot_data_hc['Correlation'].values):
                    ax_hc.text(v + (0.01 if v >= 0 else -0.01), i, f'{v:.3f}',
                              va='center', ha='left' if v >= 0 else 'right',
                              fontsize=8, fontweight='bold')

                plt.tight_layout()
                savefig_report("09e_high_correlation_pairs_bar.png", also_png=True)
                print("[OK] High correlation pairs visualization saved.")
            except Exception as e_hc_plot:
                print(f"[WARN] High correlation pairs plot failed: {e_hc_plot}")
                try:
                    plt.close(fig_hc)
                except:
                    pass
        else:
            print(f"\n✅ EXCELLENT: No feature pairs found with |r| > {corr_threshold}.")
            print("   Data shows good feature independence - no multicollinearity concerns!")

except Exception as e_corr_main:
    print(f"[ERROR] Correlation analysis failed: {e_corr_main}")

print("-" * 90 + "\n")


# ===== TARGET DISTRIBUTION =====
print("[INFO] Analyzing Target distribution (TRAIN SET ONLY)...")
if "Target" in eda_data.columns:
    vc = eda_data["Target"].value_counts(normalize=True).sort_index() * 100
    vc_counts = eda_data["Target"].value_counts().sort_index()
    print(f"[OK] Target distribution : {vc.to_dict()}")
    print(f"      Counts: Home={vc_counts.get(1,0)}, Draw={vc_counts.get(0,0)}, Away={vc_counts.get(2,0)}")

    fig, axes = plt.subplots(1, 2, figsize=(13, 5))

    target_labels = ["Draw (0)", "Home (1)", "Away (2)"]
    target_values = [vc.get(0,0), vc.get(1,0), vc.get(2,0)]
    target_colors=['#f39c12', '#2ecc71', '#e74c3c']
    bars0 = axes[0].bar(target_labels, target_values, color=target_colors, alpha=0.8, edgecolor='black')
    axes[0].set_ylabel("Percentage (%)")
    axes[0].set_title(f"Overall Outcome Distribution (Train Set, N={len(eda_data):,})")
    axes[0].set_ylim([0, max(target_values) * 1.15])
    axes[0].grid(axis='y', linestyle='--', alpha=0.9)
    for i, v in enumerate(target_values):
        axes[0].text(i, v + 0.5, f'{v:.1f}%', ha='center', fontweight='bold', fontsize=9)

    league_colors = {'E0': '#1f77b4', 'D1': '#ff7f0e', 'I1': '#2ca02c', 'SP1': '#d62728', 'F1': '#9467bd'}
    home_win_rates = {}
    if 'Division' in eda_data.columns:
        for div in eda_data['Division'].unique():
             if div in league_names:
                 sub = eda_data[eda_data["Division"]==div]
                 if not sub.empty:
                     vc_div = sub["Target"].value_counts(normalize=True).sort_index() * 100
                     home_win_rates[league_names[div]] = vc_div.get(1,0)

        if home_win_rates:
            sorted_leagues = sorted(home_win_rates.keys(), key=lambda k: home_win_rates[k], reverse=True)
            sorted_rates = [home_win_rates[k] for k in sorted_leagues]
            sorted_colors = [league_colors.get(code) for code, name in league_names.items() if name in sorted_leagues]

            bars1 = axes[1].bar(sorted_leagues, sorted_rates, color=sorted_colors, alpha=0.8, edgecolor='black')
            axes[1].set_ylabel("Home Win %")
            axes[1].set_title("Home Win Rate by League ")
            min_hw = min(sorted_rates) if sorted_rates else 40
            max_hw = max(sorted_rates) if sorted_rates else 55
            axes[1].set_ylim([max(0, np.floor(min_hw / 2)*2 - 1), min(100, np.ceil(max_hw / 2)*2 + 1)])
            axes[1].grid(axis='y', linestyle='--', alpha=0.9)
            axes[1].tick_params(axis='x')
            plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=30, ha='right')
            for i, v in enumerate(sorted_rates):
                 axes[1].text(i, v + 0.2, f'{v:.1f}%', ha='center', fontweight='bold', fontsize=9)
        else:
             axes[1].text(0.5, 0.5, 'League data not available\nor Target missing', ha='center', va='center', transform=axes[1].transAxes)
             axes[1].set_title("Home Win Rate by League ")
    else:
        axes[1].text(0.5, 0.5, "'Division' column not found", ha='center', va='center', transform=axes[1].transAxes)
        axes[1].set_title("Home Win Rate by League")

    plt.tight_layout()
    savefig_report("04_target_distribution.png", also_png=True)
    print("[OK] Target distribution analysis saved.")
else:
    print("[WARN] Target column not found in eda_data, skipping target distribution analysis.")


# =============================================================================
# 🆕 PHASE 4.2.5: DIFF-ONLY STRATEGY (MULTICOLLINEARITY FIX)
# =============================================================================
print("\n[INFO] PHASE 4.2.5: APPLYING DIFF-ONLY STRATEGY FOR TRANSFERMARKT FEATURES")
print("="*90)
print("📊 STRATEGY: Keep only DIFF features, drop Home/Away absolutes")
print("   Rationale: Eliminate perfect multicollinearity (VIF=313 → VIF<10)")
print("-"*90)

# Define Transfermarkt base features
transfermarkt_base_features = [
    'ClubValue',
    'MaxPlayerValue',
    'ManagerTrophies',
    'ManagerTenureDays',
    'NetTransferSpending',
    'ValueRatio',
    'n_players_injured',
    'n_suspended_players',
    'LeagueValue'
]

# Automatically detect Home/Away absolutes to drop
features_to_drop_transfermarkt = []
features_to_keep_diff = []

print("\n[DETECTING] Home/Away/Diff triplets...")
for base_feature in transfermarkt_base_features:
    home_col = f'HomeTeam_{base_feature}'
    away_col = f'AwayTeam_{base_feature}'
    diff_col = f'{base_feature}_Diff'

    # Check if all three exist in the dataset
    home_exists = home_col in eda_data.columns
    away_exists = away_col in eda_data.columns
    diff_exists = diff_col in eda_data.columns

    if home_exists and away_exists and diff_exists:
        # Keep Diff, drop Home and Away
        features_to_drop_transfermarkt.append(home_col)
        features_to_drop_transfermarkt.append(away_col)
        features_to_keep_diff.append(diff_col)
        print(f"  ✓ Found triplet: {base_feature}")
        print(f"    → DROP: {home_col}, {away_col}")
        print(f"    → KEEP: {diff_col}")
    elif home_exists and away_exists:
        # Diff doesn't exist, but Home/Away do
        print(f"  ⚠️  Warning: {base_feature} has Home/Away but NO Diff!")
        print(f"    → Keeping Home/Away (no Diff alternative)")
    elif diff_exists:
        # Only Diff exists (good!)
        features_to_keep_diff.append(diff_col)
        print(f"  ✓ Found solo Diff: {diff_col}")

# Also add odds/probability redundancies
print("\n[ADDING] Odds and probability redundancies...")
features_to_drop_odds = [
    'MaxHome', 'MaxDraw', 'MaxAway',  # Already covered by OddHome/Draw/Away
    'MaxOver25', 'MaxUnder25',         # Already covered by Over25/Under25
    'Prob_H', 'Prob_D', 'Prob_A',      # Already covered by Prob_*_Norm
]
features_to_drop_transfermarkt.extend(features_to_drop_odds)

# Special case: HandiSize
print("\n[SPECIAL CASE] HandiSize...")
if 'HandiSize' in eda_data.columns:
    print("  → DROP HandiSize (r=-0.976 with Prob_H_Norm)")
    features_to_drop_transfermarkt.append('HandiSize')

# Combine with original correlation-based drops
print("\n[COMBINING] Correlation drops + Diff-only drops...")
features_to_drop_list_revised = sorted(list(set(
    features_to_drop_list +              # Original correlation drops
    features_to_drop_transfermarkt       # Diff-only strategy drops
)))

# Remove any Diff features from drop list (protect them!)
features_to_drop_list_revised = [
    f for f in features_to_drop_list_revised
    if f not in features_to_keep_diff
]

print(f"\n[SUMMARY] Drop List Statistics:")
print(f"  • Original correlation drops: {len(features_to_drop_list)}")
print(f"  • Transfermarkt absolute drops: {len(features_to_drop_transfermarkt)}")
print(f"  • Total unique drops: {len(features_to_drop_list_revised)}")
print(f"  • Protected Diff features: {len(features_to_keep_diff)}")

# Show what changed
removed_from_drops = set(features_to_drop_list) - set(features_to_drop_list_revised)
added_to_drops = set(features_to_drop_list_revised) - set(features_to_drop_list)

if removed_from_drops:
    print(f"\n[PROTECTED] Features removed from drop list (now kept):")
    for feat in sorted(removed_from_drops):
        print(f"  ✓ {feat}")

if added_to_drops:
    print(f"\n[ADDED] New features added to drop list:")
    for feat in sorted(added_to_drops):
        print(f"  + {feat}")

# =============================================================================
# SAVE REVISED DROP LIST
# =============================================================================

print("\n[SAVING] Revised drop list to file...")

drop_code_path_revised = os.path.join(CSV_DIR, "09b_features_to_drop_code_REVISED.py")

with open(drop_code_path_revised, 'w') as f:
    f.write("# " + "="*77 + "\n")
    f.write("# AUTO-GENERATED: Features to drop (REVISED with Diff-Only Strategy)\n")
    f.write("# Generated by EDA V5.1 - Phase 4.2.5\n")
    f.write("# Strategy: Keep ONLY Diff features, drop Home/Away absolutes\n")
    f.write("# Rationale: Eliminate perfect multicollinearity (VIF=313 → VIF<10)\n")
    f.write("# " + "="*77 + "\n\n")

    f.write("# Protected features (KEPT):\n")
    f.write("# - ELO_Diff, OddHome, OddDraw, OddAway, Prob_*_Norm\n")
    f.write("# - All *_Diff features (ClubValue_Diff, MaxPlayerValue_Diff, etc.)\n\n")

    f.write("features_to_drop_multicollinearity_REVISED = [\n")
    for feat in features_to_drop_list_revised:
        f.write(f"    '{feat}',\n")
    f.write("]\n\n")

    f.write("# Diff features to KEEP:\n")
    f.write("features_to_keep_diff = [\n")
    for feat in features_to_keep_diff:
        f.write(f"    '{feat}',\n")
    f.write("]\n\n")

    f.write("# Usage in modeling:\n")
    f.write("# X_train = X_train.drop(columns=features_to_drop_multicollinearity_REVISED, errors='ignore')\n")
    f.write("# X_val = X_val.drop(columns=features_to_drop_multicollinearity_REVISED, errors='ignore')\n")
    f.write("# X_test = X_test.drop(columns=features_to_drop_multicollinearity_REVISED, errors='ignore')\n")

print(f"[OK] Revised drop list saved: {os.path.basename(drop_code_path_revised)}")
print(f"     File size: {os.path.getsize(drop_code_path_revised)} bytes")

# Also save original for comparison
print(f"[OK] Original drop list retained: {os.path.basename(drop_code_path)}")

print("\n" + "="*90)
print("✅ PHASE 4.2.5 COMPLETE - Diff-Only Strategy Applied")
print("="*90)


# =============================================================================
# 🆕 PHASE 4.3: CLASS IMBALANCE ANALYSIS (DESCRIPTIVE ONLY - NO SMOTE!)
# =============================================================================
print("\n[INFO] PHASE 4.3: CLASS IMBALANCE ANALYSIS (Descriptive Only)")
print("="*90)

def analyze_class_imbalance_descriptive(df, target_col='Target', output_dir=None):
    """
    📊 DESCRIPTIVE ANALYSIS ONLY - NO DATA MANIPULATION!

    Purpose:
        - Document class distribution in REAL training data
        - Assess imbalance severity
        - Provide recommendations for MODELING phase

    ⚠️  CRITICAL PHILOSOPHY:
        EDA should analyze REAL data, not synthetic data!
        SMOTE creates FAKE samples and should NEVER be used in EDA.

    Why SMOTE is WRONG in EDA:
        1. EDA Goal: Understand the TRUE data distribution
        2. SMOTE: Creates SYNTHETIC samples (not real matches!)
        3. Risk: EDA insights become misleading (patterns in fake data)
        4. Correct Place: SMOTE belongs in MODELING pipeline, not EDA

    Correct Workflow:
        ✅ EDA: Analyze real data → Document imbalance → Recommend strategies
        ✅ Modeling: Apply SMOTE (if needed) → Train on synthetic + real data
        ❌ WRONG: EDA on SMOTE data (analyzing fake patterns!)

    Args:
        df: DataFrame with Target column (REAL training data only)
        target_col: Name of target column (default: 'Target')
        output_dir: Directory to save plots (optional)

    Returns:
        dict: Imbalance metrics and recommendations
    """

    print("\n📊 CLASS IMBALANCE ANALYSIS - TRAIN SET (Real Data Only)")
    print("="*80 + "\n")

    print("⚠️  IMPORTANT NOTE:")
    print("   This analysis uses REAL training data only.")
    print("   NO synthetic samples (SMOTE) are created or analyzed.")
    print("   SMOTE (if needed) will be applied later in modeling phase.\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 1: PREPARE DATA (Remove NaN only, NO SMOTE!)
    # ════════════════════════════════════════════════════════════════════
    print("[STEP 1] Preparing data for analysis...")

    # Remove NaN from Target (data quality check)
    original_len = len(df)
    df_clean = df[df[target_col].notna()].copy()
    n_nan_removed = original_len - len(df_clean)

    if n_nan_removed > 0:
        print(f"  ⚠️  Removed {n_nan_removed:,} samples with NaN in Target column")
        print(f"      Original: {original_len:,} → Clean: {len(df_clean):,}")
    else:
        print(f"  ✓ No NaN values in Target column")

    y = df_clean[target_col].copy()

    print(f"\n  Dataset for analysis:")
    print(f"    • Total samples: {len(y):,}")
    print(f"    • Data type: REAL matches (no synthetic data)")
    print(f"    • Time period: Training set only (pre-2019)")

    # ════════════════════════════════════════════════════════════════════
    # STEP 2: CLASS DISTRIBUTION ANALYSIS
    # ════════════════════════════════════════════════════════════════════
    print(f"\n[STEP 2] Analyzing class distribution...")
    print("-" * 60)

    class_names = ['Draw', 'Home Win', 'Away Win']
    y_counts = y.value_counts().sort_index()
    y_dist = y.value_counts(normalize=True).sort_index()

    print(f"\n  Class Distribution (Real Training Data):")
    for class_idx in range(3):
        count = y_counts.get(class_idx, 0)
        pct = y_dist.get(class_idx, 0) * 100
        bar = '█' * int(pct / 2)  # Simple bar chart
        print(f"    Class {class_idx} ({class_names[class_idx]:9s}): {count:5,} ({pct:5.2f}%) {bar}")

    # ════════════════════════════════════════════════════════════════════
    # STEP 3: IMBALANCE SEVERITY ASSESSMENT
    # ════════════════════════════════════════════════════════════════════
    print(f"\n[STEP 3] Assessing imbalance severity...")

    max_count = y_counts.max()
    min_count = y_counts.min()
    imbalance_ratio = max_count / min_count

    majority_class = y_counts.idxmax()
    minority_class = y_counts.idxmin()

    print(f"\n  Imbalance Metrics:")
    print(f"    • Majority Class: {majority_class} ({class_names[majority_class]}) = {max_count:,} samples")
    print(f"    • Minority Class: {minority_class} ({class_names[minority_class]}) = {min_count:,} samples")
    print(f"    • Imbalance Ratio: {imbalance_ratio:.2f}x")

    # Severity classification
    if imbalance_ratio < 1.5:
        severity = "MILD"
        severity_emoji = "✅"
        severity_desc = "Acceptable - No intervention strictly required"
    elif imbalance_ratio < 3.0:
        severity = "MODERATE"
        severity_emoji = "⚠️"
        severity_desc = "Consider class balancing techniques"
    else:
        severity = "SEVERE"
        severity_emoji = "🚨"
        severity_desc = "Strong intervention recommended"

    print(f"\n  {severity_emoji} Severity Assessment: {severity}")
    print(f"     → {severity_desc}")

    # Football context
    print(f"\n  ⚽ Football Context:")
    print(f"     • Home win advantage is NATURAL in football")
    print(f"     • Ratio of {imbalance_ratio:.2f}x is typical for home/draw/away")
    print(f"     • This reflects REAL-WORLD match outcomes")

    # ════════════════════════════════════════════════════════════════════
    # STEP 4: RECOMMENDATIONS FOR MODELING PHASE
    # ════════════════════════════════════════════════════════════════════
    print(f"\n" + "="*80)
    print("[STEP 4] RECOMMENDATIONS FOR MODELING PHASE")
    print("="*80 + "\n")

    print("🎯 STRATEGY 1: Class Weights (⭐ RECOMMENDED for Football Prediction)")
    print("-" * 80)
    print("\n  Why Recommended:")
    print("    • No synthetic data created (preserves true distribution)")
    print("    • Works with all algorithms (tree-based, linear models)")
    print("    • Interpretable (real matches, real patterns)")
    print("    • Faster training (no data augmentation)")

    print("\n  How to Implement:")
    print("    # For scikit-learn models:")
    print("    from sklearn.linear_model import LogisticRegression")
    print("    model = LogisticRegression(class_weight='balanced')")
    print("")
    print("    # For XGBoost (recommended for football):")
    print("    import xgboost as xgb")

    # Calculate class weights
    total_samples = len(y)
    n_classes = len(y_counts)
    class_weights = {}
    for class_idx in range(n_classes):
        weight = total_samples / (n_classes * y_counts.get(class_idx, 1))
        class_weights[class_idx] = weight

    print(f"    class_weights = {{")
    for class_idx, weight in class_weights.items():
        print(f"        {class_idx}: {weight:.3f},  # {class_names[class_idx]}")
    print(f"    }}")
    print(f"    model = xgb.XGBClassifier(scale_pos_weight=class_weights)")

    print("\n  ✅ Advantages:")
    print("    • Simple to implement")
    print("    • No additional computational cost")
    print("    • Preserves natural data distribution")
    print("    • Works well with tree-based models")

    if imbalance_ratio >= 1.5:
        print("\n\n🎯 STRATEGY 2: SMOTE (⚠️ Alternative - Use with Caution)")
        print("-" * 80)

        print("\n  ⚠️  CRITICAL WARNINGS:")
        print("    • SMOTE creates SYNTHETIC samples (not real matches!)")
        print("    • Apply ONLY in modeling pipeline, NEVER in EDA")
        print("    • Only for TRAINING set (never validation/test)")
        print("    • May create unrealistic match scenarios")

        print("\n  When to Consider SMOTE:")
        print("    • Class weights alone are insufficient")
        print("    • Very severe imbalance (>5x)")
        print("    • Using algorithms sensitive to class balance")

        print("\n  Correct Implementation:")
        print("    from imblearn.over_sampling import SMOTE")
        print("    from sklearn.model_selection import train_test_split")
        print("")
        print("    # Step 1: Split data FIRST (before any SMOTE!)")
        print("    X_train, X_val, y_train, y_val = train_test_split(X, y, ...")
        print("    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, ...)")
        print("")
        print("    # Step 2: Apply SMOTE ONLY to training set")
        print("    smote = SMOTE(sampling_strategy='not majority', random_state=42)")
        print("    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)")
        print("")
        print("    # Step 3: Train on balanced data")
        print("    model.fit(X_train_balanced, y_train_balanced)")
        print("")
        print("    # Step 4: Evaluate on REAL validation/test data (no SMOTE!)")
        print("    y_val_pred = model.predict(X_val)  # Original val set")
        print("    y_test_pred = model.predict(X_test)  # Original test set")

        print("\n  ❌ WRONG Implementation (DO NOT DO THIS!):")
        print("    # ❌ Applying SMOTE before split (DATA LEAKAGE!)")
        print("    X_balanced, y_balanced = smote.fit_resample(X, y)")
        print("    X_train, X_test, ... = train_test_split(X_balanced, y_balanced)")
        print("")
        print("    # ❌ Applying SMOTE to validation/test (CONTAMINATION!)")
        print("    X_val_balanced, y_val_balanced = smote.fit_resample(X_val, y_val)")

        print("\n  Expected SMOTE Results (Simulation):")
        print(f"    Before SMOTE: {dict(y_counts)}")
        print(f"    After SMOTE:  Draw={max_count}, Home Win={max_count}, Away Win={max_count}")
        print(f"    → Perfect balance (1:1:1 ratio)")

    print("\n\n🎯 STRATEGY 3: Stratified Sampling (✅ ALWAYS Recommended)")
    print("-" * 80)

    print("\n  Purpose:")
    print("    • Ensure proportional class distribution in train/val/test")
    print("    • Prevent accidentally creating imbalanced splits")

    print("\n  Implementation:")
    print("    from sklearn.model_selection import train_test_split")
    print("")
    print("    # Use stratify parameter")
    print("    X_train, X_temp, y_train, y_temp = train_test_split(")
    print("        X, y, test_size=0.3, stratify=y, random_state=42")
    print("    )")
    print("")
    print("    X_val, X_test, y_val, y_test = train_test_split(")
    print("        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42")
    print("    )")

    print("\n  ✅ Benefits:")
    print("    • Maintains class proportions across all sets")
    print("    • More reliable model evaluation")
    print("    • Essential for time-series cross-validation")

    print("\n\n🎯 STRATEGY 4: Evaluation Metrics (✅ CRITICAL)")
    print("-" * 80)

    print("\n  Why Important:")
    print("    • Accuracy is MISLEADING with imbalanced data")
    print(f"    • Example: Predicting all 'Home Win' gives {y_dist.max()*100:.1f}% accuracy!")

    print("\n  Recommended Metrics for Football Prediction:")
    print("    1. F1-Score (macro/weighted)  ← Primary metric")
    print("    2. Precision & Recall (per class)")
    print("    3. Cohen's Kappa (agreement measure)")
    print("    4. Confusion Matrix (detailed view)")
    print("    5. ROC-AUC (one-vs-rest)")

    print("\n  Implementation:")
    print("    from sklearn.metrics import classification_report, cohen_kappa_score")
    print("")
    print("    # Comprehensive evaluation")
    print("    print(classification_report(y_true, y_pred, target_names=class_names))")
    print("    print(f'Kappa Score: {cohen_kappa_score(y_true, y_pred):.3f}')")

    print("\n" + "="*80 + "\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 5: VISUALIZATION (Single Plot - Distribution Only)
    # ════════════════════════════════════════════════════════════════════
    if output_dir:
        print("[STEP 5] Creating visualization...")

        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        colors = ['#f39c12', '#2ecc71', '#e74c3c']

        # PLOT 1: Bar Chart with Annotations
        ax = axes[0]
        bars = ax.bar(class_names, y_counts.values, color=colors,
                     alpha=0.85, edgecolor='black', linewidth=2)

        # Add value labels on bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            pct = y_dist.iloc[i] * 100

            # Count label (on top)
            ax.text(bar.get_x() + bar.get_width()/2., height + max(y_counts)*0.02,
                   f'{int(height):,}',
                   ha='center', va='bottom', fontsize=12, fontweight='bold')

            # Percentage label (inside bar)
            ax.text(bar.get_x() + bar.get_width()/2., height/2,
                   f'{pct:.1f}%',
                   ha='center', va='center', fontsize=12, fontweight='bold',
                   color='white' if pct > 15 else 'black')

        ax.set_ylabel('Number of Matches', fontsize=12, fontweight='bold')
        ax.set_xlabel('Match Outcome', fontsize=12, fontweight='bold')
        ax.set_title(f'Class Distribution \n' +
                    f'Imbalance Ratio: {imbalance_ratio:.2f}x ({severity})',
                    fontsize=13, fontweight='bold', pad=15)
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        ax.set_ylim([0, max(y_counts.values) * 1.15])

        # PLOT 2: Pie Chart with Better Labels
        ax = axes[1]

        # Explode minority classes slightly
        explode_values = []
        for idx in range(len(y_counts)):
            if idx == minority_class:
                explode_values.append(0.1)
            elif idx == majority_class:
                explode_values.append(0)
            else:
                explode_values.append(0.05)

        wedges, texts, autotexts = ax.pie(
            y_counts.values,
            labels=class_names,
            colors=colors,
            autopct='%1.1f%%',
            startangle=90,
            explode=explode_values,
            textprops={'fontsize': 11, 'fontweight': 'bold'},
            shadow=True
        )

        # Enhance autotext
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontsize(12)

        ax.set_title(f'Class Proportions \n' +
                    f'Majority: {class_names[majority_class]} ({y_dist.max()*100:.1f}%)',
                    fontsize=13, fontweight='bold', pad=15)

        # Add legend with counts
        legend_labels = [f'{name}: {count:,}' for name, count in zip(class_names, y_counts.values)]
        ax.legend(legend_labels, loc='upper left', bbox_to_anchor=(1, 0, 0.5, 1), fontsize=10)

        # Main title
        plt.suptitle('⚽ Class Imbalance Analysis - REAL Training Data Only (NO SMOTE)\n' +
                    'Purpose: Document natural distribution for modeling decisions',
                    fontsize=14, fontweight='bold', y=1.00)

        plt.tight_layout(rect=[0, 0, 1, 0.96])

        # Save plot
        save_path = os.path.join(output_dir, '04c_class_imbalance_analysis.png')
        plt.savefig(save_path, dpi=600, bbox_inches='tight', facecolor='white')
        savefig_report('04c_class_imbalance_analysis.png', also_png=True)
        plt.close()

        print(f"  ✅ Visualization saved: {os.path.basename(save_path)}")

    print("\n" + "="*80 + "\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 6: RETURN METRICS
    # ════════════════════════════════════════════════════════════════════
    return {
        'class_counts': y_counts.to_dict(),
        'class_distribution': y_dist.to_dict(),
        'imbalance_ratio': imbalance_ratio,
        'majority_class': majority_class,
        'minority_class': minority_class,
        'severity': severity,
        'class_weights': class_weights,
        'recommendations': {
            'primary': 'Use class_weight="balanced" (tree-based models)',
            'alternative': 'SMOTE (only if class weights insufficient)',
            'critical': 'NEVER apply SMOTE in EDA or on validation/test sets',
            'evaluation': 'Use F1-score, Kappa, not accuracy'
        }
    }

# ════════════════════════════════════════════════════════════════════════
# EXECUTE ANALYSIS (DESCRIPTIVE ONLY - NO SMOTE!)
# ════════════════════════════════════════════════════════════════════════

if 'Target' in eda_data.columns:
    print("[INFO] Executing class imbalance analysis...")

    imbalance_metrics = analyze_class_imbalance_descriptive(
        df=eda_data,
        target_col='Target',
        output_dir=IMAGES_DIR
    )

    if imbalance_metrics:
        print("\n[OK] Class imbalance analysis complete (descriptive only).")
        print(f"     → Recommendations saved in analysis output")
        print(f"     → Apply strategies in MODELING phase, not in EDA")
    else:
        print("\n[WARN] Class imbalance analysis returned no metrics.")
else:
    print("[WARN] Skipping class imbalance analysis (Target column missing).")
    imbalance_metrics = None


# ===== CATEGORICAL FEATURES DISTRIBUTION =====
print("\n[INFO] Analyzing Categorical Feature Distributions (Frequency Plots)...")

try:
    temp_cat_features_all = eda_data.select_dtypes(include=['category', 'object']).columns.tolist()
    temp_exclude_chi = ['HomeTeam', 'AwayTeam', 'MatchDate', 'YearMonth', 'Season',
                        'FTResult', 'Target', 'HTResult', 'Referee',
                        'HomeTeam_ManagerName', 'AwayTeam_ManagerName']
    cat_features_to_plot = [f for f in temp_cat_features_all if f not in temp_exclude_chi and f in eda_data.columns and eda_data[f].nunique() < 50]

    if not cat_features_to_plot:
        print("[WARN] No suitable low-cardinality categorical features found for plotting.")
    else:
        print(f"[INFO] Plotting frequency for: {cat_features_to_plot}")
        num_cat_plots = len(cat_features_to_plot)
        cols_cat = 3
        rows_cat = (num_cat_plots + cols_cat - 1) // cols_cat

        fig_cat, axes_cat = plt.subplots(rows_cat, cols_cat, figsize=(6 * cols_cat, 5 * rows_cat), squeeze=False)
        axes_cat = axes_cat.flatten()
        plot_count_cat = 0

        for idx, feature in enumerate(cat_features_to_plot):
            ax = axes_cat[idx]
            try:
                plot_data = eda_data[feature].astype(str).fillna('Missing')
                sns.countplot(y=plot_data, ax=ax, palette="Set2",
                              order = plot_data.value_counts().index[:15])
                ax.set_title(f"Frequency of '{feature}' (Top 15, Train Set)", fontsize=10, fontweight='bold')
                ax.set_xlabel("Count")
                ax.set_ylabel(feature)
                ax.tick_params(axis='y', labelsize=8)
                ax.grid(axis='x', linestyle='--', alpha=0.9)
                plot_count_cat += 1
            except Exception as e_cat_plot:
                ax.text(0.5, 0.5, f"Plot failed for {feature}:\n{str(e_cat_plot)[:100]}", ha='center', va='center', transform=ax.transAxes, fontsize=8)

        for i in range(plot_count_cat, len(axes_cat)):
            axes_cat[i].axis('off')

        if plot_count_cat > 0:
            plt.tight_layout(pad=2.0)
            savefig_report("04b_categorical_distributions.png", also_png=True)
            print(f"[OK] Plotted {plot_count_cat} categorical feature distributions.")
        else:
            plt.close(fig_cat)

except Exception as e_cat_main:
    print(f"[ERROR] Categorical distribution plotting failed: {e_cat_main}")
    try:
        plt.close(fig_cat)
    except:
        pass


# ===== NUMERICAL FEATURES DISTRIBUTION (Home vs Away) =====
print("\n[INFO] Analyzing Numerical Feature Distributions (Home vs Away)...")

home_away_pairs = [
    ('HomeElo', 'AwayElo', 'ELO Rating'),
    ('HomeTeam_ClubValue', 'AwayTeam_ClubValue', 'Club Value'),
    ('HomeTeam_MaxPlayerValue', 'AwayTeam_MaxPlayerValue', 'Max Player Value'),
    ('HomeTeam_ManagerTrophies', 'AwayTeam_ManagerTrophies', 'Manager Trophies'),
    ('HomeTeam_NetTransferSpending', 'AwayTeam_NetTransferSpending', 'Net Transfer Spend'),
    ('HomeTeam_n_players_injured', 'AwayTeam_n_players_injured', 'Injured Players'),
    # 🆕 Temporal Features
    ('Home_Target_Avg_L5', 'Away_Target_Avg_L5', 'Shots on Target (L5)'),
    ('Home_Shots_Avg_L5', 'Away_Shots_Avg_L5', 'Total Shots (L5)'),
]

num_pairs = len(home_away_pairs)
cols_dist = 3
rows_dist = (num_pairs + cols_dist - 1) // cols_dist
fig_dist, axes_dist = plt.subplots(rows_dist, cols_dist, figsize=(5 * cols_dist, 4 * rows_dist))
axes_dist = axes_dist.flatten()
plot_count = 0

for idx, (home_feat, away_feat, label) in enumerate(home_away_pairs):
    ax = axes_dist[idx]
    if home_feat in eda_data.columns and away_feat in eda_data.columns:
        home_data = eda_data[home_feat].dropna()
        away_data = eda_data[away_feat].dropna()

        if len(home_data) > 0 or len(away_data) > 0:
            sns.histplot(home_data, ax=ax, color=UNIFIED_COLORS['main_blue'], label=f'Home ({home_data.mean():.1f} avg)', kde=True, bins=25, alpha=0.9)
            sns.histplot(away_data, ax=ax, color='#e74c3c', label=f'Away ({away_data.mean():.1f} avg)', kde=True, bins=25, alpha=0.9)
            ax.set_title(f'{label}\nDistribution ', fontsize=10, fontweight='bold')
            ax.set_xlabel(label)
            ax.set_ylabel('Frequency')
            ax.legend(fontsize=8)
            ax.grid(alpha=0.3)
            plot_count += 1
        else:
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{label}\nDistribution ', fontsize=10, fontweight='bold')
    else:
        ax.text(0.5, 0.5, f'Features Missing:\n{home_feat}\nor\n{away_feat}', ha='center', va='center', transform=ax.transAxes, fontsize=8)
        ax.set_title(f'{label}\nDistribution ', fontsize=10, fontweight='bold')
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8)

for i in range(idx + 1, len(axes_dist)):
    axes_dist[i].axis('off')

if plot_count > 0:
    plt.tight_layout()
    savefig_report("05_numerical_distributions_home_away.png", also_png=True)
    print(f"[OK] Plotted {plot_count} Home vs Away distributions.")
else:
    print("[WARN] No valid Home/Away pairs found to plot distributions.")
    plt.close(fig_dist)


# ===== DIFFERENCE FEATURES DISTRIBUTION =====
print("\n[INFO] Analyzing Difference Feature Distributions...")

diff_features = [
    'ELO_Diff', 'ClubValue_Diff', 'MaxPlayerValue_Diff', 'ManagerTrophies_Diff',
    'ManagerTenureDays_Diff', 'NetTransferSpending_Diff', 'n_players_injured_Diff',
    'ValueRatio_Diff',
    # 🆕 Temporal Diffs
    'Target_Diff_L5', 'Shots_Diff_L5', 'Corners_Diff_L5', 'ShotAccuracy_Diff_L5'
]

available_diff_features = [f for f in diff_features if f in eda_data.columns]

if not available_diff_features:
    print("[WARN] No difference features (_Diff) found in eda_data. Skipping this analysis.")
else:
    num_diff = len(available_diff_features)
    cols_diff = 3
    rows_diff = (num_diff + cols_diff - 1) // cols_diff
    fig_diff, axes_diff = plt.subplots(rows_diff, cols_diff, figsize=(5 * cols_diff, 4 * rows_diff))
    axes_diff = axes_diff.flatten()
    plot_count_diff = 0

    for idx, feature in enumerate(available_diff_features):
        ax = axes_diff[idx]
        data = eda_data[feature].dropna()

        if len(data) > 0:
            sns.histplot(data, ax=ax, color='#9b59b6', kde=True, bins=30, alpha=0.7)
            mean_val = data.mean()
            median_val = data.median()
            ax.axvline(mean_val, color='red', linestyle='--', linewidth=1.5, label=f'Mean: {mean_val:.2f}')
            ax.axvline(median_val, color='blue', linestyle=':', linewidth=1.5, label=f'Median: {median_val:.2f}')

            ax.set_title(f'{feature}\nDistribution ', fontsize=10, fontweight='bold')
            ax.set_xlabel(feature)
            ax.set_ylabel('Frequency')
            ax.legend(fontsize=8)
            ax.grid(alpha=0.3)
            plot_count_diff += 1
        else:
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{feature}\nDistribution ', fontsize=10, fontweight='bold')
        ax.tick_params(axis='x', labelsize=8)
        ax.tick_params(axis='y', labelsize=8)

    for i in range(idx + 1, len(axes_diff)):
        axes_diff[i].axis('off')

    if plot_count_diff > 0:
        plt.tight_layout()
        savefig_report("06_difference_features_distribution.png", also_png=True)
        print(f"[OK] Plotted {plot_count_diff} Difference feature distributions.")
    else:
        plt.close(fig_diff)


# ===== DRAW ANALYSIS (VIOLINPLOTS) =====
print("\n[INFO] Analyzing Difference Features vs. Target (Draw Analysis)...")

if 'available_diff_features' in locals() and available_diff_features:
    features_for_boxplot = available_diff_features[:9]
    num_box = len(features_for_boxplot)
    cols_box = 3
    rows_box = (num_box + cols_box - 1) // cols_box
    fig_box_draw, axes_box_draw = plt.subplots(rows_box, cols_box,
                                                figsize=(6 * cols_box, 5 * rows_box),
                                                squeeze=False)
    axes_box_draw = axes_box_draw.flatten()
    plot_count_box_draw = 0

    # ✅ V5.2 FIX: ROBUST PALETTE SETUP WITH STRING KEYS
    print("  [INFO] Preparing palette and target encoding...")

    # 🆕 CRITICAL FIX: Clean NaN values BEFORE casting to int
    original_len = len(eda_data)
    n_nan_target = eda_data['Target'].isna().sum()

    if n_nan_target > 0:
        print(f"      ⚠️  Found {n_nan_target} NaN values in Target")
        print(f"         → Removing for violin plot analysis...")
        eda_data = eda_data[eda_data['Target'].notna()].copy()
        print(f"         ✓ Cleaned: {original_len:,} → {len(eda_data):,} samples")

    # Now safe to cast to int
    eda_data['Target'] = eda_data['Target'].astype(int)
    print(f"      ✓ Target converted to integer type")

    # Get unique target values from data (handles missing classes)
    unique_targets = sorted(eda_data['Target'].dropna().unique())

    # Define base palette for reference (integer keys)
    palette_colors_base = {0: '#f39c12', 1: '#2ecc71', 2: '#e74c3c'}

    # ✅ FIX: Create palette with STRING keys for seaborn compatibility
    palette_draw = {str(k): palette_colors_base[k]
                    for k in unique_targets
                    if k in palette_colors_base}
    # Result: palette_draw = {'0': '#f39c12', '1': '#2ecc71', '2': '#e74c3c'}

    # ✅ Also create color list for consistent legend ordering
    colors_list = [palette_colors_base[k] for k in unique_targets]

    print(f"      ✓ Target classes present: {unique_targets}")
    print(f"      ✓ Palette configured for: {list(palette_draw.keys())} (string keys)")

    for idx, feature in enumerate(features_for_boxplot):
        ax = axes_box_draw[idx]

        try:
            # ✅ Additional safety: check if feature has variance
            if eda_data[feature].std() < 1e-6:
                print(f"  [WARN] {feature}: No variance, skipping violin plot")
                ax.text(0.5, 0.5, f'{feature}\n(No variance)',
                       ha='center', va='center', transform=ax.transAxes, fontsize=9)
                ax.set_title(f'{feature}\n(Skipped)', fontsize=10, fontweight='bold')
                continue

            # Create violin plot with filtered data
            plot_data = eda_data[[feature, 'Target']].dropna()

            if len(plot_data) < 10:
                print(f"  [WARN] {feature}: Insufficient data ({len(plot_data)} samples)")
                ax.text(0.5, 0.5, f'{feature}\n(Insufficient data)',
                       ha='center', va='center', transform=ax.transAxes, fontsize=9)
                ax.set_title(f'{feature}\n(Skipped)', fontsize=10, fontweight='bold')
                continue

            # ✅ FIX: Convert Target to string for seaborn compatibility
            plot_data['Target_str'] = plot_data['Target'].astype(str)

            # ✅ NOW THIS WORKS: String Target column matches string palette keys
            sns.violinplot(
                data=plot_data,
                x='Target_str',        # ✅ String column: '0', '1', '2'
                y=feature,
                ax=ax,
                palette=palette_draw,  # ✅ String keys: {'0': ..., '1': ..., '2': ...}
                inner='quartile',
                cut=0
            )

            ax.set_title(f'{feature}\nby Match Outcome ',
                        fontsize=10, fontweight='bold')
            ax.set_xlabel('Outcome', fontweight='bold')

            # ✅ Custom x-tick labels with class names
            class_names = ['Draw', 'Home Win', 'Away Win']
            tick_labels = [f"{class_names[t]} ({t})" for t in unique_targets]
            ax.set_xticklabels(tick_labels, fontsize=9)

            ax.set_ylabel(feature, fontweight='bold')
            ax.axhline(0, color='black', linestyle='--', linewidth=1.5, alpha=0.7,
                      label='Zero Line')
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            plot_count_box_draw += 1

            # ✅ Hypothesis check for ELO_Diff
            if feature == 'ELO_Diff' and 0 in unique_targets:
                median_draw = plot_data[plot_data['Target']==0][feature].median()
                print(f"  [HYPOTHESIS CHECK] Median ELO_Diff for Draws: {median_draw:.2f} (Expected ~0)")

                # ✅ Add annotation to plot
                if pd.notna(median_draw):
                    ax.text(0.02, 0.98, f'Draw Median: {median_draw:.2f}',
                           transform=ax.transAxes, fontsize=8,
                           verticalalignment='top',
                           bbox=dict(boxstyle='round,pad=0.5',
                                   facecolor='yellow', alpha=0.7))

        except Exception as e_violin:
            print(f"  [ERROR] Violin plot failed for {feature}: {e_violin}")
            ax.text(0.5, 0.5, f"Plot failed:\n{feature}\n{str(e_violin)[:50]}",
                   ha='center', va='center', transform=ax.transAxes, fontsize=8,
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='mistyrose', alpha=0.9))
            ax.set_title(f'{feature}\n(Error)', fontsize=10, fontweight='bold')

    for i in range(plot_count_box_draw, len(axes_box_draw)):
        axes_box_draw[i].axis('off')

# ===== NUMERICAL SCATTER PLOTS =====
print("\n[INFO] Analyzing Numerical Scatter Plots vs Target (TRAIN SET ONLY)...")

scatter_pairs = [
    ('ELO_Diff', 'ClubValue_Diff', 'ELO vs Club Value'),
    ('Form5Difference', 'ManagerTrophies_Diff', 'Form (5) vs Manager Trophies'),
    ('ValueRatio_Diff', 'NetTransferSpending_Diff', 'Value Ratio vs Net Transfer Spend'),
    # 🆕 Temporal Features
    ('Target_Diff_L5', 'Shots_Diff_L5', 'Target L5 vs Shots L5')
]

n_samples_scatter = min(5000, len(eda_data))
eda_data_sample_scatter = eda_data.sample(n_samples_scatter, random_state=RNG_SEED).copy()

target_map_scatter = {0: 'Draw', 1: 'Home', 2: 'Away'}
colors_scatter = {0: '#f39c12', 1: '#2ecc71', 2: '#e74c3c'}

valid_scatter_pairs = []
for x_feat, y_feat, label in scatter_pairs:
    if x_feat in eda_data_sample_scatter.columns and y_feat in eda_data_sample_scatter.columns:
        valid_scatter_pairs.append((x_feat, y_feat, label))
    else:
        print(f"[WARN] Scatter plot pair skipped (missing): {x_feat} or {y_feat}")

num_pairs_sc = len(valid_scatter_pairs)

if num_pairs_sc > 0:
    fig_sc, axes_sc = plt.subplots(1, num_pairs_sc, figsize=(7 * num_pairs_sc, 6))
    if num_pairs_sc == 1:
        axes_sc = [axes_sc]
    plot_count_sc = 0

    for idx, (x_feat, y_feat, label) in enumerate(valid_scatter_pairs):
        ax = axes_sc[idx]
        try:
            sns.scatterplot(
                data=eda_data_sample_scatter,
                x=x_feat,
                y=y_feat,
                hue='Target',
                palette=colors_scatter,
                ax=ax,
                alpha=0.9,
                s=30,
                legend='full'
            )
            ax.set_title(f'{label}\n(Sampled, N={n_samples_scatter})', fontsize=10, fontweight='bold')
            ax.set_xlabel(x_feat)
            ax.set_ylabel(y_feat)
            ax.grid(alpha=0.3, linestyle='--')

            try:
                handles, labels = ax.get_legend_handles_labels()
                valid_indices = [i for i, lbl in enumerate(labels) if lbl.strip()]
                if 'Target' in labels:
                    valid_indices = [i for i in valid_indices if labels[i] != 'Target']

                new_handles = [handles[i] for i in valid_indices]
                new_labels = [target_map_scatter.get(int(float(labels[i])), labels[i]) for i in valid_indices]

                if new_labels:
                    ax.legend(handles=new_handles, labels=new_labels, title='Outcome')
                else:
                    ax.legend(title='Outcome')
            except Exception as e_leg:
                print(f"[WARN] Legend formatting failed for {x_feat}: {e_leg}")

            plot_count_sc += 1
        except Exception as e_sc_plot:
            ax.text(0.5, 0.5, f"Plot failed for:\n{x_feat} vs {y_feat}", ha='center', va='center', transform=ax.transAxes, fontsize=8)

    if plot_count_sc > 0:
        plt.tight_layout(pad=2.0)
        savefig_report("06b_numerical_scatter_plots.png", also_png=True)
        print(f"[OK] Plotted {plot_count_sc} numerical scatter plots.")
    else:
        plt.close(fig_sc)
else:
    print("[WARN] No valid scatter plot pairs defined or features available.")


# =============================================================================
# TABLE 1 - DESCRIPTIVE STATISTICS
# =============================================================================
print("\n[INFO] Generating Descriptive Statistics Table (TRAIN SET ONLY)...")

key_features_list = [
    'HomeElo', 'AwayElo', 'ELO_Diff',
    'HomeTeam_ClubValue', 'AwayTeam_ClubValue', 'ClubValue_Diff',
    'HomeTeam_ValueRatio', 'AwayTeam_ValueRatio', 'ValueRatio_Diff',
    'HomeTeam_ManagerTrophies', 'AwayTeam_ManagerTrophies', 'ManagerTrophies_Diff',
    'HomeTeam_NetTransferSpending', 'AwayTeam_NetTransferSpending', 'NetTransferSpending_Diff',
    # 🆕 Temporal Features
    'Home_Target_Avg_L5', 'Away_Target_Avg_L5', 'Target_Diff_L5',
    'Home_Shots_Avg_L5', 'Away_Shots_Avg_L5', 'Shots_Diff_L5',
    'ShotAccuracy_Diff_L5'
]

key_features = [f for f in key_features_list if f in eda_data.columns]
key_features = list(dict.fromkeys(key_features))

if not key_features:
    print("[WARN] No key features found for descriptive stats. Using all numeric columns.")
    key_features = numeric_cols
    if len(key_features) > 20:
         key_features = key_features[:20]

if not key_features:
     print("[ERROR] No numeric features available for descriptive statistics. Skipping.")
else:
    print(f"[INFO] Calculating stats for {len(key_features)} selected features...")
    stat_summary = eda_data[key_features].describe().T
    try:
        stat_summary['Skewness'] = eda_data[key_features].skew()
        stat_summary['Kurtosis'] = eda_data[key_features].kurtosis()
        stat_summary = stat_summary[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'Skewness', 'Kurtosis']]
        stat_summary.columns = ['Count', 'Mean', 'Std Dev', 'Min', 'Q1', 'Median', 'Q3', 'Max', 'Skewness', 'Kurtosis']
    except Exception as e_skew:
        print(f"[WARN] Could not calculate Skewness/Kurtosis: {e_skew}")
        stat_summary = stat_summary[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
        stat_summary.columns = ['Count', 'Mean', 'Std Dev', 'Min', 'Q1', 'Median', 'Q3', 'Max']

    stat_summary_rounded = stat_summary.round(3)
    stat_summary_rounded.to_csv(os.path.join(CSV_DIR, "01_descriptive_statistics_train.csv"))
    save_df_as_image(stat_summary_rounded, "01_descriptive_statistics_train.png",
                     title="Table 1: Descriptive Statistics of Key Numerical Variables ")
    print("[OK] Descriptive statistics saved (CSV + PNG).")


# =============================================================================
# CORRELATION ANALYSIS (CONTINUED - Feature-Target Correlation)
# =============================================================================
print("\n[INFO] Performing Feature-Target Correlation Analysis...")

try:
    if 'correlation_matrix' in locals() and not correlation_matrix.empty:
        if 'Target' in eda_data.columns:
             target_corr = eda_data[numeric_cols + ['Target']].corr()['Target'].drop('Target').sort_values(ascending=False)
             print(f"[OK] Feature-Target correlations calculated (Top 5):")
             print(target_corr.head())
        else:
             target_corr = pd.Series(dtype=float)
             print("[WARN] Target column not available for Feature-Target correlation.")

        # Görselleştirme: Feature-Target Korelasyonları
        if not target_corr.empty:
            print("[INFO] Creating feature-target correlations bar chart...")
            fig_tcorr, ax_tcorr = plt.subplots(figsize=(12, 10))
            try:
                target_corr_sorted = target_corr.sort_values()
                n_show_corr = 10
                top_display = pd.concat([target_corr_sorted.head(n_show_corr), target_corr_sorted.tail(n_show_corr)])
                colors = [UNIFIED_COLORS['away_red'] if x < 0 else UNIFIED_COLORS['main_blue'] for x in top_display.values]

                bars = ax_tcorr.barh(range(len(top_display)), top_display.values, color=colors, height=0.7)
                ax_tcorr.set_yticks(range(len(top_display)))
                ax_tcorr.set_yticklabels(top_display.index, fontsize=10)
                ax_tcorr.set_xlabel('Pearson Correlation with Target', fontweight='bold', fontsize=12)
                ax_tcorr.set_title(f'Feature-Target Correlations (Top & Bottom {n_show_corr}, Train Set)',
                                   fontweight='bold', fontsize=14, pad=20)
                ax_tcorr.grid(axis='x', alpha=0.3)
                for i, v in enumerate(top_display.values):
                     ax_tcorr.text(v + (0.005 if v >= 0 else -0.005), i, f'{v:.3f}',
                                   va='center', ha='left' if v >= 0 else 'right',
                                   fontsize=9, fontweight='bold')
                ax_tcorr.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
                plt.tight_layout()
                savefig_report("09b_feature_target_correlations.png", also_png=True)
                print("[OK] Feature-target correlations bar chart saved.")
            except Exception as e_tcorr:
                print(f"[WARN] Feature-Target correlation plot failed: {e_tcorr}")
                plt.close(fig_tcorr)

        # Focused Heatmap (Stats vs. Market Odds) - TOP 10 & BLUE THEME
        print("\n[INFO] Creating focused heatmap (Stats vs. Market Odds)...")
        # TO BE: Özellik sayısını 3-4'ten 10'a çıkarma (En yüksek korelasyonlu olanları seçme)
        if 'Target' in eda_data.columns:
            # Target ile korelasyonu en yüksek 10 numeric özelliği bul (Odds hariç)
            target_corr_abs = eda_data[numeric_cols + ['Target']].corr()['Target'].abs().sort_values(ascending=False)
            # Target'ın kendisini ve Odds özelliklerini listeden çıkar
            exclude_odds = ['Target', 'OddHome', 'OddDraw', 'OddAway', 'Prob_H_Norm', 'Prob_D_Norm', 'Prob_A_Norm', 'Overround']
            top_10_stats = [f for f in target_corr_abs.index if f not in exclude_odds][:10]
        else:
            # Yedek liste
            top_10_stats = ['ELO_Diff', 'Form5Difference', 'ValueRatio_Diff', 'ClubValue_Diff',
                            'Target_Diff_L5', 'Shots_Diff_L5', 'NetTransferSpending_Diff',
                            'ManagerTrophies_Diff', 'Home_Target_Avg_L5', 'Away_Target_Avg_L5']
        odds_features = ['Prob_H_Norm', 'Prob_D_Norm', 'Prob_A_Norm', 'Overround']
        stats_features_avail = [f for f in top_10_stats if f in eda_data.columns]
        odds_features_avail = [f for f in odds_features if f in eda_data.columns]
        if stats_features_avail and odds_features_avail:
            try:
                focused_cols = stats_features_avail + odds_features_avail
                focused_corr = eda_data[focused_cols].corr()
                cross_corr_matrix = focused_corr.loc[stats_features_avail, odds_features_avail]
                fig_focus, ax_focus = plt.subplots(figsize=(10, 8))

                # TO BE: Renk tonları mavi (cmap='Blues')
                sns.heatmap(cross_corr_matrix,
                            annot=True, fmt='.2f', cmap='Blues', center=None,
                            ax=ax_focus, annot_kws={'fontsize': 9},
                            cbar_kws={'label': 'Pearson Correlation'},
                            linewidths=0.5, linecolor='white')

                ax_focus.set_title('Cross-Correlation: Top 10 Statistical Features vs. Market Odds ',
                                   fontweight='bold', fontsize=12, pad=20)
                ax_focus.set_xticklabels(ax_focus.get_xticklabels(), fontsize=10, rotation=45, ha='right')
                ax_focus.set_yticklabels(ax_focus.get_yticklabels(), fontsize=10, rotation=0)
                plt.tight_layout()

                # TO BE: PDF Çıktısı (savefig_report zaten PDF'e ekler ama ayrıca kaydedelim)
                pdf_path = os.path.join(IMAGES_DIR, "09d_correlation_heatmap_stats_vs_odds.pdf")
                plt.savefig(pdf_path, dpi=600, bbox_inches="tight")
                savefig_report("09d_correlation_heatmap_stats_vs_odds.png", also_png=True)
                print("[OK] Focused Stats vs. Odds heatmap saved (Blue Theme, Top 10).")

                if 'ELO_Diff' in stats_features_avail and 'Prob_H_Norm' in odds_features_avail:
                    elo_prob_corr = cross_corr_matrix.loc['ELO_Diff', 'Prob_H_Norm']
                    print(f"[HYPOTHESIS CHECK] Correlation(ELO_Diff, Prob_H_Norm) = {elo_prob_corr:.3f}")
                    if elo_prob_corr > 0.7:
                        print("[INSIGHT] High correlation suggests market odds heavily incorporate ELO information.")
                    else:
                        print("[INSIGHT] Correlation is not dominant. 'Odds-free' models may find value.")

            except Exception as e_focus_hmap:
                print(f"[WARN] Failed to create Stats vs Odds heatmap: {e_focus_hmap}")
                try: plt.close(fig_focus)
                except:pass
        else:
            print("[WARN] Could not find all required features for Stats vs. Odds heatmap. Skipping.")

        # Save correlation data to CSV
        try:
            correlation_matrix.to_csv(os.path.join(CSV_DIR, "correlation_matrix_full_train.csv"))
            if not target_corr.empty:
                 target_corr.to_csv(os.path.join(CSV_DIR, "feature_target_correlation_train.csv"))
                 top_bottom_df = pd.DataFrame({
                     'Feature': top_display.index,
                     'Correlation': top_display.values,
                     'Type': ['Negative' if x < 0 else 'Positive' for x in top_display.values]
                 }).sort_values('Correlation')
                 top_bottom_df.to_csv(os.path.join(CSV_DIR, f"top_bottom_{n_show_corr}_correlations_train.csv"), index=False)
            print("[OK] Correlation data saved to CSV files.")
        except Exception as e_csv_corr:
             print(f"[WARN] Failed to save correlation data to CSV: {e_csv_corr}")

except Exception as e_corr_main:
    print(f"[ERROR] Correlation analysis continuation failed: {e_corr_main}")
# =============================================================================
# 🆕 PHASE 4.4: VIF (VARIANCE INFLATION FACTOR) ANALYSIS - OPTIMIZED
# =============================================================================
print("\n[INFO] PHASE 4.4: VIF (VARIANCE INFLATION FACTOR) ANALYSIS")
print("="*90)

# ════════════════════════════════════════════════════════════════════════
# 🔥 YENİ: SAFE VIF COMPUTATION FUNCTION
# ════════════════════════════════════════════════════════════════════════

def compute_vif_safe(df, threshold_corr=0.90, threshold_vif=10):
    """
    Önce high correlation temizle, sonra VIF hesapla

    Bu yaklaşım:
    - 3-5x daha hızlı
    - Numerically daha stabil
    - High VIF'leri daha iyi tespit eder
    """
    if not VIF_AVAILABLE:
        print("  ⚠️  VIF not available. Skipping analysis.")
        return None, []

    print("\n🔍 VIF ANALYSIS - OPTIMIZED (Safe Mode)")
    print("="*80 + "\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 0: INITIAL CLEANUP
    # ════════════════════════════════════════════════════════════════════
    print("[STEP 0] Initial data preparation...")

    # Numeric only
    X = df.select_dtypes(include=[np.number]).copy()
    print(f"  • Starting features: {X.shape[1]}")

    # Remove zero variance
    variance_threshold = 1e-6
    zero_var = X.columns[X.std() <= variance_threshold]
    if len(zero_var) > 0:
        print(f"  • Removing {len(zero_var)} zero-variance features")
        X = X.drop(columns=zero_var)

    # Fill missing
    X = X.fillna(X.median())
    print(f"  • After cleanup: {X.shape[1]} features\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 1: AGGRESSIVE CORRELATION FILTERING
    # ════════════════════════════════════════════════════════════════════
    print(f"[STEP 1] Removing high correlations (|r| > {threshold_corr})...")

    corr_matrix = X.corr().abs()
    upper_tri = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    upper = corr_matrix.where(upper_tri)

    # Drop one from each high-corr pair
    to_drop = []
    dropped_pairs = []

    for column in upper.columns:
        high_corr_features = upper.index[upper[column] > threshold_corr].tolist()
        if high_corr_features:
            # Keep first, drop rest
            for feat in high_corr_features:
                if feat not in to_drop and column not in to_drop:
                    to_drop.append(feat)
                    corr_val = corr_matrix.loc[column, feat]
                    dropped_pairs.append((column, feat, corr_val))

    if to_drop:
        print(f"  • Dropping {len(to_drop)} features due to high correlation")
        print(f"  • Sample pairs removed:")
        for pair in dropped_pairs[:5]:
            print(f"      → {pair[0]} ↔ {pair[1]} (r={pair[2]:.3f})")
        if len(dropped_pairs) > 5:
            print(f"      ... and {len(dropped_pairs)-5} more pairs")

        X_clean = X.drop(columns=to_drop, errors='ignore')
    else:
        print(f"  ✓ No features exceeded correlation threshold")
        X_clean = X.copy()

    print(f"  • Features after filtering: {X_clean.shape[1]}\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 2: COMPUTE VIF (Now much faster!)
    # ════════════════════════════════════════════════════════════════════
    print("[STEP 2] Computing VIF on cleaned data...")
    print("  → Expected time: 5-15 seconds (vs 30-60 before)\n")

    import time
    start_time = time.time()

    vif_data = []
    features_list = X_clean.columns.tolist()

    for i, col in enumerate(features_list):
        if (i + 1) % 10 == 0:
            print(f"  Progress: {i+1}/{len(features_list)}", end='\r')

        try:
            vif = variance_inflation_factor(X_clean.values, i)

            # 🔥 Stability check
            if np.isnan(vif) or np.isinf(vif):
                print(f"\n  ⚠️  {col}: VIF=NaN (numerical instability)")
                vif_data.append({'Feature': col, 'VIF': np.nan, 'Status': 'Unstable'})
            elif vif > 1000:
                print(f"\n  ⚠️  {col}: VIF={vif:.1f} (suspiciously high)")
                vif_data.append({'Feature': col, 'VIF': vif, 'Status': 'Very High'})
            else:
                vif_data.append({'Feature': col, 'VIF': vif, 'Status': 'OK'})

        except Exception as e:
            print(f"\n  ❌ {col}: ERROR - {str(e)[:50]}")
            vif_data.append({'Feature': col, 'VIF': np.nan, 'Status': 'Error'})

    elapsed = time.time() - start_time
    print(f"\n  ✅ VIF computation done in {elapsed:.1f} seconds\n")

    vif_df = pd.DataFrame(vif_data).sort_values('VIF', ascending=False)

    # ════════════════════════════════════════════════════════════════════
    # STEP 3: IDENTIFY DROP CANDIDATES
    # ════════════════════════════════════════════════════════════════════
    print(f"[STEP 3] Identifying features to drop (VIF ≥ {threshold_vif})...")

    high_vif = vif_df[vif_df['VIF'] >= threshold_vif]

    if len(high_vif) > 0:
        print(f"  • Found {len(high_vif)} features with VIF ≥ {threshold_vif}:")
        for idx, row in high_vif.head(10).iterrows():
            print(f"      → {row['Feature']:40s} VIF={row['VIF']:.1f}")
        if len(high_vif) > 10:
            print(f"      ... and {len(high_vif)-10} more")

        features_to_drop_vif = high_vif['Feature'].tolist()
    else:
        print(f"  ✓ No features exceed VIF threshold")
        features_to_drop_vif = []

    # ════════════════════════════════════════════════════════════════════
    # STEP 4: SUMMARY
    # ════════════════════════════════════════════════════════════════════
    print("\n" + "="*80)
    print("[SUMMARY] VIF Analysis Results")
    print("="*80)
    print(f"  • Initial features:           {len(df.columns)}")
    print(f"  • After correlation filter:   {X_clean.shape[1]}")
    print(f"  • Features with VIF ≥ {threshold_vif}:     {len(features_to_drop_vif)}")
    print(f"  • Total recommended drops:    {len(to_drop) + len(features_to_drop_vif)}")
    print("="*80 + "\n")

    return vif_df, features_to_drop_vif, to_drop


# ════════════════════════════════════════════════════════════════════════
# 🔥 EXECUTE OPTIMIZED VIF ANALYSIS
# ════════════════════════════════════════════════════════════════════════

if len(numeric_cols) > 5 and VIF_AVAILABLE:
    try:
        print("[INFO] Starting OPTIMIZED VIF analysis...")

        # 🔥 YENİ: Önce correlation-based drops uygula (eğer varsa)
        if 'features_to_drop_list' in locals() and features_to_drop_list:
            print(f"\n[PRE-FILTER] Applying {len(features_to_drop_list)} correlation-based drops first...")
            features_for_vif = [f for f in numeric_cols if f not in features_to_drop_list]
            print(f"  • Reduced features: {len(numeric_cols)} → {len(features_for_vif)}")
        else:
            features_for_vif = numeric_cols

        # Prepare data
        eda_data_vif = eda_data[features_for_vif].copy()

        # 🔥 YENİ: Safe VIF computation
        vif_results, vif_drop_list, corr_drop_list = compute_vif_safe(
            df=eda_data_vif,
            threshold_corr=0.90,  # Aggressive correlation threshold
            threshold_vif=10       # Standard VIF threshold
        )

        if vif_results is not None:
            # Save results
            vif_results.to_csv(os.path.join(CSV_DIR, "09g_vif_results.csv"), index=False)
            print(f"[OK] VIF results saved to CSV.")

            # 🔥 YENİ: Combined drop list
            all_drops_multicollinearity = list(set(
                (features_to_drop_list if 'features_to_drop_list' in locals() else []) +
                corr_drop_list +
                vif_drop_list
            ))

            print(f"\n[FINAL DROP LIST] Total features to remove: {len(all_drops_multicollinearity)}")
            print(f"  • From correlation (Phase 4.2): {len(features_to_drop_list) if 'features_to_drop_list' in locals() else 0}")
            print(f"  • From VIF correlation filter: {len(corr_drop_list)}")
            print(f"  • From VIF threshold (≥10):    {len(vif_drop_list)}")

            # Save combined drop list
            drop_code_path = os.path.join(CSV_DIR, "09h_combined_multicollinearity_drops.py")
            with open(drop_code_path, 'w') as f:
                f.write("# AUTO-GENERATED: Combined multicollinearity drop list\n")
                f.write("# Sources: Phase 4.2 correlation + VIF analysis\n")
                f.write("# Generated by EDA V5.2 (Optimized VIF)\n\n")
                f.write("features_to_drop_multicollinearity = [\n")
                for feat in sorted(all_drops_multicollinearity):
                    f.write(f"    '{feat}',\n")
                f.write("]\n\n")
                f.write("# Usage:\n")
                f.write("# X_train = X_train.drop(columns=features_to_drop_multicollinearity)\n")

            print(f"[OK] Combined drop list saved: {os.path.basename(drop_code_path)}")

            # 🔥 YENİ: Visualization - VIF Analysis Plot (BLUE THEME)
            if IMAGES_DIR and not vif_results.empty:
                print("[INFO] Creating VIF visualization...")

                try:
                    # En yüksek VIF değerine sahip ilk 30 özelliği alalım (okunabilirlik için)
                    top_vif_plot = vif_results.head(30).sort_values('VIF', ascending=True)

                    fig_vif, ax_vif = plt.subplots(figsize=(10, max(6, len(top_vif_plot)*0.3)))

                    # Renkler: Mavi tonları (Düşük VIF açık mavi, Yüksek VIF koyu mavi)
                    # VIF değerine göre normalize edelim
                    norm = plt.Normalize(top_vif_plot['VIF'].min(), top_vif_plot['VIF'].max())
                    colors = plt.cm.Blues(norm(top_vif_plot['VIF'].values))

                    bars = ax_vif.barh(top_vif_plot['Feature'], top_vif_plot['VIF'], color=colors, edgecolor='black', linewidth=0.5)

                    # Eşik çizgisi (VIF = 10)
                    ax_vif.axvline(10, color='red', linestyle='--', linewidth=1.5, label='Threshold (VIF=10)')

                    ax_vif.set_xlabel('Variance Inflation Factor (VIF)', fontweight='bold')
                    ax_vif.set_title('Multicollinearity Check: Top 30 Features by VIF', fontweight='bold')
                    ax_vif.legend(loc='lower right')
                    ax_vif.grid(axis='x', alpha=0.3, linestyle='--')

                    # Değerleri çubukların ucuna yaz
                    for i, v in enumerate(top_vif_plot['VIF']):
                        ax_vif.text(v + 0.2, i, f'{v:.1f}', va='center', fontsize=8, fontweight='bold')
                    plt.tight_layout()

                    # 1. PDF Olarak Kaydet
                    pdf_path_vif = os.path.join(IMAGES_DIR, "09f_vif_analysis.pdf")
                    plt.savefig(pdf_path_vif, dpi=600, bbox_inches='tight')

                    # 2. PNG Olarak Kaydet (Rapora eklemek için)
                    save_path_vif = os.path.join(IMAGES_DIR, "09f_vif_analysis.png")
                    plt.savefig(save_path_vif, dpi=600, bbox_inches='tight')
                    savefig_report("09f_vif_analysis.png", also_png=False)

                    plt.close(fig_vif)
                    print(f"[OK] VIF Analysis plot saved (PDF & PNG).")

                except Exception as e_plot_vif:
                    print(f"[WARN] Could not create VIF plot: {e_plot_vif}")
                    plt.close(fig_vif)

    except Exception as e_vif:
        print(f"[ERROR] VIF analysis failed: {e_vif}")
        import traceback
        traceback.print_exc()
        vif_results = None
        vif_drop_list = []
else:
    print("[WARN] VIF analysis skipped (need >5 features and statsmodels).")
    vif_results = None
    vif_drop_list = []

# =============================================================================
# FEATURE RELEVANCE (Mutual Information)
# =============================================================================
print("\n[INFO] Computing Feature Relevance (Mutual Information) (TRAIN SET ONLY)...")

mi_scores = pd.Series(dtype=float)

try:
    if not numeric_cols:
         print("[WARN] No numeric columns for MI calculation. Skipping.")
    elif 'Target' not in eda_data.columns:
         print("[WARN] Target column missing for MI calculation. Skipping.")
    else:
        X_mi = eda_data[numeric_cols].copy()
        y_mi = eda_data['Target'].copy()

        all_nan_cols = X_mi.columns[X_mi.isnull().all()].tolist()
        if all_nan_cols:
             print(f"[WARN] MI Input: Following columns are all NaN and will be dropped: {all_nan_cols}")
             X_mi = X_mi.drop(columns=all_nan_cols)
             numeric_cols_mi = X_mi.columns.tolist()
        else:
             numeric_cols_mi = numeric_cols

        X_mi = X_mi.fillna(X_mi.mean())
        y_mi = y_mi.fillna(y_mi.mode()[0])

        if len(X_mi) != len(y_mi):
             print(f"[ERROR] MI: Shape mismatch between X ({len(X_mi)}) and y ({len(y_mi)}). Skipping.")
        elif len(X_mi) == 0:
             print("[ERROR] MI: Input data is empty after processing. Skipping.")
        else:
            print(f"[INFO] Calculating MI for {len(numeric_cols_mi)} features...")
            scaler_mi = StandardScaler()
            X_mi_scaled = scaler_mi.fit_transform(X_mi)

            mi_values = mutual_info_classif(X_mi_scaled, y_mi, random_state=RNG_SEED)
            mi_scores = pd.Series(mi_values, index=numeric_cols_mi).sort_values(ascending=False)

            print(f"\n[OK] Mutual Information scores calculated!")
            print("=" * 90)
            print("🏆 TOP 10 MOST PREDICTIVE FEATURES (by MI Score):")
            print("=" * 90)
            for i, (feat, score) in enumerate(mi_scores.head(10).items(), 1):
                print(f"  {i:2d}. {feat:40s} → MI = {score:.4f}")
            print("=" * 90 + "\n")

            # Visualization 1: MI Scores Bar Chart - BLUE THEME
            try:
                fig_mi1, ax_mi1 = plt.subplots(figsize=(10, 8))
                n_show_mi = min(20, len(mi_scores))

                # TO BE: Mavi tonları (Blues colormap)
                colors_grad = plt.cm.Blues(np.linspace(0.4, 1.0, n_show_mi))

                ax_mi1.barh(range(n_show_mi), mi_scores.values[:n_show_mi], color=colors_grad)
                ax_mi1.set_yticks(range(n_show_mi))
                ax_mi1.set_yticklabels(mi_scores.index[:n_show_mi], fontsize=9)
                ax_mi1.invert_yaxis()
                ax_mi1.set_xlabel('Mutual Information Score with Target', fontweight='bold')
                ax_mi1.set_title(f'Mutual Information (MI) Scores with Target (Top {n_show_mi}, Train Set)',
                                 fontweight='bold', fontsize=12)
                ax_mi1.grid(axis='x', alpha=0.3)
                plt.tight_layout()

                # TO BE: PDF Çıktısı
                plt.savefig(os.path.join(IMAGES_DIR, "10a_mutual_information_scores.pdf"), dpi=600, bbox_inches="tight")
                savefig_report("10a_mutual_information_scores.png", also_png=True)
                print("[OK] MI scores bar chart saved (Blue Theme).")
            except Exception as e_mi1:
                 print(f"[WARN] MI bar chart (10a) failed: {e_mi1}")
                 plt.close(fig_mi1)

            # Visualization 2: Figure 4 Format - BLUE THEME
            try:
                fig_mi2 = plt.figure(figsize=(12, 10))
                ax_mi2 = fig_mi2.add_subplot(111)
                top_20_mi = mi_scores.head(20)
                # TO BE: Renk turuncudan (#F39C12) maviye (#1f77b4 veya steelblue) çevrildi
                bars_mi2 = ax_mi2.barh(range(len(top_20_mi)), top_20_mi.values,
                                       color=UNIFIED_COLORS['main_blue'], edgecolor='black', linewidth=1.2)

                ax_mi2.set_yticks(range(len(top_20_mi)))
                ax_mi2.set_yticklabels(top_20_mi.index, fontsize=10)
                ax_mi2.invert_yaxis()
                for i, (feature, value) in enumerate(top_20_mi.items()):
                    ax_mi2.text(value + mi_scores.max()*0.01, i, f'{value:.4f}',
                                va='center', fontsize=9, fontweight='bold')
                ax_mi2.set_xlabel('Information Gain (Mutual Information)', fontweight='bold', fontsize=12)
                ax_mi2.set_title('Feature Relevance: Mutual Information (Information Gain) - Top 20 ',
                                 fontweight='bold', fontsize=12, pad=20)
                ax_mi2.grid(axis='x', alpha=0.3)
                ax_mi2.set_xlim([0, mi_scores.max() * 1.15])
                plt.tight_layout()

                # TO BE: PDF Çıktısı
                plt.savefig(os.path.join(IMAGES_DIR, "10b_feature_relevance_mutual_information.pdf"), dpi=600, bbox_inches="tight")
                savefig_report("10b_feature_relevance_mutual_information.png", also_png=True)
                print("[OK] Feature Relevance (MI) visualization saved (Blue Theme).")
            except Exception as e_mi2:
                print(f"[WARN] MI Figure 4 plot (10b) failed: {e_mi2}")
                plt.close(fig_mi2)

            # Save MI scores to CSV
            try:
                mi_detail_df = pd.DataFrame({
                    'Feature': mi_scores.index,
                    'Mutual_Information': mi_scores.values,
                    'Rank': range(1, len(mi_scores) + 1)
                })
                mi_detail_df.to_csv(os.path.join(CSV_DIR, "02_feature_relevance_mutual_information_train.csv"), index=False)
                print("[OK] Feature relevance (MI) details saved to CSV.")
            except Exception as e_csv_mi:
                 print(f"[WARN] Failed to save MI scores to CSV: {e_csv_mi}")

except Exception as e_mi_main:
    print(f"[ERROR] Mutual Information calculation failed: {e_mi_main}")


# =============================================================================
# 🆕 t-SNE VISUALIZATION (MULTI-PERPLEXITY COMPARISON)
# =============================================================================
print("\n[INFO] Computing t-SNE Visualization (Multi-Perplexity) (TRAIN SET ONLY)...")

def plot_tsne_multiperplexity(df, target_col='Target', n_components=2,
                               perplexities=[5, 15, 30, 50],
                               random_state=42, output_dir=None):
    """
    t-SNE visualization with multiple perplexity values for comparison

    Perplexity Impact:
    - Low (5-10): Focus on local structure
    - Medium (20-30): Balanced view (recommended)
    - High (50+): Focus on global structure
    """

    print("\n🔮 t-SNE MULTI-PERPLEXITY COMPARISON")
    print("="*80 + "\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 1: PREPARE DATA
    # ════════════════════════════════════════════════════════════════════
    print("[INFO] Preparing data...")

    y = df[target_col].copy()
    X = df.drop(columns=[target_col, 'MatchDate', 'HomeTeam', 'AwayTeam'], errors='ignore')
    X = X.select_dtypes(include=[np.number])

    # Remove zero-variance
    zero_var = X.columns[X.std() == 0]
    if len(zero_var) > 0:
        print(f"  ⚠️  Removing {len(zero_var)} zero-variance features")
        X = X.drop(columns=zero_var)

    X = X.fillna(X.median())

    # Subsample if large
    MAX_SAMPLES = 3000
    if len(X) > MAX_SAMPLES:
        print(f"  ⚠️  Subsampling to {MAX_SAMPLES:,} samples")
        sample_idx = np.random.choice(len(X), MAX_SAMPLES, replace=False)
        X_tsne = X.iloc[sample_idx].copy()
        y_tsne = y.iloc[sample_idx].copy()
    else:
        X_tsne = X.copy()
        y_tsne = y.copy()

    print(f"  Dataset: {X_tsne.shape[0]:,} samples × {X_tsne.shape[1]} features\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 2: COMPUTE t-SNE FOR EACH PERPLEXITY
    # ════════════════════════════════════════════════════════════════════
    print("[COMPUTING t-SNE] Testing multiple perplexities...")
    print("  → This may take 2-5 minutes\n")

    embeddings = {}
    computation_times = {}

    for perp in perplexities:
        print(f"  Computing perplexity={perp}...", end=' ')

        start_time = time.time()

        try:
            if perp >= len(X_tsne):
                print(f"⚠️  SKIPPED (perp must be < n_samples)")
                continue

            tsne = TSNE(
                n_components=n_components,
                perplexity=perp,
                random_state=random_state,
                init='pca',
                learning_rate='auto',
                n_iter=1000,
                verbose=0
            )

            embedding = tsne.fit_transform(X_tsne)
            embeddings[perp] = embedding

            elapsed = time.time() - start_time
            computation_times[perp] = elapsed

            print(f"✅ ({elapsed:.1f}s)")

        except Exception as e:
            print(f"❌ ERROR: {str(e)[:50]}")
            continue

    if not embeddings:
        print("\n  ❌ No valid embeddings computed!")
        return None

    print(f"\n  ✅ Successfully computed {len(embeddings)} embeddings")
    print(f"  Total time: {sum(computation_times.values()):.1f}s\n")

    # ════════════════════════════════════════════════════════════════════
    # STEP 3: VISUALIZATION
    # ════════════════════════════════════════════════════════════════════
    if output_dir and len(embeddings) > 0:
        print("[VISUALIZATION] Creating comparison plots...")

        n_plots = len(embeddings)
        if n_plots <= 2:
            nrows, ncols = 1, 2
            figsize = (12, 6)
        elif n_plots <= 4:
            nrows, ncols = 2, 2
            figsize = (14, 12)
        else:
            nrows, ncols = 2, 3
            figsize = (18, 12)

        fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
        axes = axes.flatten() if n_plots > 1 else [axes]

        class_names = ['Draw', 'Home Win', 'Away Win']
        colors = ['#AED6F1', '#2E86C1', '#154360']

        for idx, (perp, embedding) in enumerate(sorted(embeddings.items())):
            ax = axes[idx]

            for class_idx, (class_name, color) in enumerate(zip(class_names, colors)):
                mask = (y_tsne == class_idx)

                if mask.sum() > 0:
                    ax.scatter(
                        embedding[mask, 0],
                        embedding[mask, 1],
                        c=color,
                        label=class_name,
                        alpha=0.9,
                        s=40,
                        edgecolors='black',
                        linewidth=0.5
                    )

            ax.set_xlabel('t-SNE Dimension 1', fontsize=10, fontweight='bold')
            ax.set_ylabel('t-SNE Dimension 2', fontsize=10, fontweight='bold')
            ax.set_title(f'Perplexity = {perp}\n({computation_times[perp]:.1f}s)',
                        fontsize=12, fontweight='bold', pad=10)
            ax.legend(fontsize=9, loc='upper right', framealpha=0.9)
            ax.grid(alpha=0.3)

            # Silhouette score
            if SILHOUETTE_AVAILABLE:
                try:
                    silhouette = silhouette_score(embedding, y_tsne)
                    info_text = f'Samples: {len(y_tsne):,}\nSilhouette: {silhouette:.3f}'
                except:
                    info_text = f'Samples: {len(y_tsne):,}'
            else:
                info_text = f'Samples: {len(y_tsne):,}'

            ax.text(0.02, 0.98, info_text, transform=ax.transAxes,
                   fontsize=8, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        # Hide unused
        for idx in range(n_plots, len(axes)):
            axes[idx].axis('off')

        plt.suptitle('t-SNE Dimensionality Reduction - Perplexity Comparison\n' +
                     'How does perplexity affect cluster visualization?',
                     fontsize=15, fontweight='bold', y=0.995)
        plt.tight_layout(rect=[0, 0, 1, 0.98])

        # 1. PDF OLARAK KAYDET (İstediğin özellik)
        pdf_path = os.path.join(output_dir, '11a_tsne_multiperplexity.pdf')
        try:
            plt.savefig(pdf_path, dpi=600, bbox_inches='tight', facecolor='white')
            print(f"    ✅ Saved PDF: {os.path.basename(pdf_path)}")
        except Exception as e:
            print(f"    ⚠️ Could not save PDF: {e}")

        # 2. PNG OLARAK KAYDET (Fix: save_path değişkenini açıkça tanımlıyoruz)
        save_path = os.path.join(output_dir, '11a_tsne_multiperplexity.png')
        try:
            plt.savefig(save_path, dpi=600, bbox_inches='tight', facecolor='white')
            # Rapora işle (PDF raporuna da ekler)
            savefig_report('11a_tsne_multiperplexity.png', also_png=False)
        except Exception as e:
            print(f"    ⚠️ Could not save PNG: {e}")

        plt.close()
        print(f"    ✅ Saved PNG: {os.path.basename(save_path)}")

    # ════════════════════════════════════════════════════════════════════
    # STEP 4: RECOMMENDATIONS
    # ════════════════════════════════════════════════════════════════════
    print("\n" + "="*80)
    print("[RECOMMENDATIONS] Perplexity Selection")
    print("="*80)

    n_samples = len(X_tsne)

    print(f"\n  Dataset size: {n_samples:,} samples")
    print(f"  Recommended range: {max(5, n_samples//100)} - {min(50, n_samples//10)}")
    print(f"  Default: 30 (balanced)")

    # Find best by silhouette
    if SILHOUETTE_AVAILABLE:
        try:
            silhouette_scores = {}
            for perp, embedding in embeddings.items():
                try:
                    score = silhouette_score(embedding, y_tsne)
                    silhouette_scores[perp] = score
                except:
                    pass

            if silhouette_scores:
                best_perp = max(silhouette_scores, key=silhouette_scores.get)
                best_score = silhouette_scores[best_perp]

                print(f"\n  Silhouette Scores:")
                for perp in sorted(silhouette_scores.keys()):
                    score = silhouette_scores[perp]
                    marker = "🏆" if perp == best_perp else "  "
                    print(f"    {marker} Perplexity={perp:2d}: {score:.4f}")

                print(f"\n  ✅ Best: perplexity={best_perp} (silhouette={best_score:.4f})")
        except:
            pass

    print("\n" + "="*80 + "\n")

    return {
        'embeddings': embeddings,
        'computation_times': computation_times,
        'sample_size': len(X_tsne),
        'n_features': X_tsne.shape[1]
    }

# ════════════════════════════════════════════════════════════════════════
# EXECUTE t-SNE
# ════════════════════════════════════════════════════════════════════════

try:
    if not numeric_cols:
        print("[WARN] No numeric columns for t-SNE. Skipping.")
        tsne_results = None
    elif 'Target' not in eda_data.columns:
        print("[WARN] Target column missing for t-SNE. Skipping.")
        tsne_results = None
    else:
        tsne_results = plot_tsne_multiperplexity(
            df=eda_data,
            target_col='Target',
            n_components=2,
            perplexities=[5, 15, 30, 50],
            random_state=RNG_SEED,
            output_dir=IMAGES_DIR
        )
        print("[OK] t-SNE multi-perplexity analysis complete.")

except ImportError:
    print("[WARN] Could not import TSNE. Skipping.")
    tsne_results = None
except Exception as e_tsne:
    print(f"[ERROR] t-SNE analysis failed: {e_tsne}")
    tsne_results = None


# =============================================================================
# 🆕 PHASE 4.9: FEATURE ENGINEERING EXAMPLES & VISUALIZATION
# =============================================================================
print("\n[INFO] PHASE 4.9: FEATURE ENGINEERING EXAMPLES")
print("="*90)

def visualize_feature_engineering_pipeline(df, output_dir):
    """
    Visualize derived features from model's feature engineering pipeline

    Demonstrates:
    1. ELO_Diff: Performance indicator
    2. ValueRatio_Diff: Financial advantage
    3. Log Transformations: Skewness reduction
    4. Normalized Probabilities: Bookmaker odds
    5. Diff Features Correlation
    6. Lag Features: Temporal patterns
    """

    print("\n🔧 FEATURE ENGINEERING PIPELINE")
    print("="*80 + "\n")

    print("[INFO] Visualizing model feature engineering")
    print("  → Goal: Show why engineered features improve prediction\n")

    # ════════════════════════════════════════════════════════════════════
    # CREATE FIGURE (3×2 grid)
    # ════════════════════════════════════════════════════════════════════
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 2, hspace=0.35, wspace=0.25)

    class_names = ['Draw', 'Home Win', 'Away Win']
    colors = ['#f39c12', '#2ecc71', '#e74c3c']

    # ════════════════════════════════════════════════════════════════════
    # PLOT 1: ELO_Diff Distribution
    # ════════════════════════════════════════════════════════════════════
    ax1 = fig.add_subplot(gs[0, 0])

    if 'ELO_Diff' in df.columns and 'Target' in df.columns:
        print("[PLOT 1] ELO_Diff distribution...")

        for target_val, color, label in zip([0, 1, 2], colors, class_names):
            data = df[df['Target'] == target_val]['ELO_Diff'].dropna()
            if len(data) > 0:
                ax1.hist(data, bins=40, alpha=1.0, color=color,
                        label=f'{label} (n={len(data):,})',
                        edgecolor='black', linewidth=0.5)

        ax1.axvline(0, color='black', linestyle='--', linewidth=2.5,
                   label='Equal ELO', alpha=0.7)
        ax1.set_xlabel('ELO_Diff (Home - Away)', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax1.set_title('ELO Difference as Performance Indicator\n' +
                     'Positive = Home advantage',
                     fontsize=12, fontweight='bold', pad=15)
        ax1.legend(fontsize=9)
        ax1.grid(alpha=0.3)

        # Medians
        for target_val, color in zip([0, 1, 2], colors):
            median_val = df[df['Target'] == target_val]['ELO_Diff'].median()
            if pd.notna(median_val):
                ax1.axvline(median_val, color=color, linestyle=':', linewidth=2, alpha=0.5)
    else:
        ax1.text(0.5, 0.5, 'ELO_Diff not available',
                ha='center', va='center', transform=ax1.transAxes)

    # ════════════════════════════════════════════════════════════════════
    # PLOT 2: ValueRatio_Diff Distribution
    # ════════════════════════════════════════════════════════════════════
    ax2 = fig.add_subplot(gs[0, 1])

    if 'ValueRatio_Diff' in df.columns and 'Target' in df.columns:
        print("[PLOT 2] ValueRatio_Diff distribution...")

        for target_val, color, label in zip([0, 1, 2], colors, class_names):
            data = df[df['Target'] == target_val]['ValueRatio_Diff'].dropna()
            if len(data) > 0:
                ax2.hist(data, bins=40, alpha=0.9, color=color,
                        label=f'{label} (n={len(data):,})',
                        edgecolor='black', linewidth=0.5)

        ax2.axvline(0, color='black', linestyle='--', linewidth=2.5, alpha=0.7)
        ax2.set_xlabel('ValueRatio_Diff', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax2.set_title('Financial Advantage Indicator\n' +
                     'ValueRatio = ClubValue / LeagueValue',
                     fontsize=12, fontweight='bold', pad=15)
        ax2.legend(fontsize=9)
        ax2.grid(alpha=0.3)
    else:
        ax2.text(0.5, 0.5, 'ValueRatio_Diff not available',
                ha='center', va='center', transform=ax2.transAxes)

    # ════════════════════════════════════════════════════════════════════
    # PLOT 3: Log Transformation Effect (FIXED & UNIFIED)
    # ════════════════════════════════════════════════════════════════════
    ax3 = fig.add_subplot(gs[1, 0])
    if 'HomeTeam_ClubValue' in df.columns:
        print("[PLOT 3] Log transformation effect...")
        original = df['HomeTeam_ClubValue'].dropna()

        # 1. Orijinal Veri (Gri - Arka Plan)
        ax3.hist(original, bins=50,
                 alpha=1.0,
                 color=UNIFIED_COLORS['gray'],
                 label='Original',
                 edgecolor='black',
                 linewidth=1.2)

        # Eksen rengi de Gri olsun ki hangi veriye ait olduğu anlaşılsın
        ax3.set_ylabel('Frequency (Original)', fontsize=12, fontweight='bold', color=UNIFIED_COLORS['gray'])
        ax3.tick_params(axis='y', labelcolor=UNIFIED_COLORS['gray'])

        # 2. Log Dönüşümü (Ana Mavi - Ön Plan)
        ax3_twin = ax3.twinx()
        log_transformed = np.log1p(original)

        ax3_twin.hist(log_transformed, bins=50,
                      alpha=0.8,
                      color=UNIFIED_COLORS['main_blue'],
                      label='Log',
                      edgecolor='black',
                      linewidth=1.2)

        # HATA DÜZELTİLDİ: color= parametresi eklendi
        ax3_twin.set_ylabel('Frequency (Log)', fontsize=12, fontweight='bold', color=UNIFIED_COLORS['main_blue'])
        ax3_twin.tick_params(axis='y', labelcolor=UNIFIED_COLORS['main_blue'])

        ax3.set_xlabel('Club Value', fontsize=12, fontweight='bold')
        ax3.set_title('Log Transformation Effect\n' +
                      'Reduces skewness for better training',
                      fontsize=12, fontweight='bold', pad=15)
        ax3.grid(alpha=0.5, axis='x', linestyle='--')

        # Legend (Renkler düzeltildi ve color= parametresi eklendi)
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color=UNIFIED_COLORS['gray'], lw=6, label='Original (skewed)'),
            Line2D([0], [0], color=UNIFIED_COLORS['main_blue'], lw=6, label='Log (normalized)')
        ]
        ax3.legend(handles=legend_elements, loc='upper right', fontsize=9)
    else:
        ax3.text(0.5, 0.5, 'ClubValue not available',
                 ha='center', va='center', transform=ax3.transAxes)

    # ════════════════════════════════════════════════════════════════════
    # PLOT 4: Normalized Odds
    # ════════════════════════════════════════════════════════════════════
    ax4 = fig.add_subplot(gs[1, 1])

    odds_cols = ['OddHome', 'OddDraw', 'OddAway']
    norm_cols = ['Prob_H_Norm', 'Prob_D_Norm', 'Prob_A_Norm']

    if all(c in df.columns for c in odds_cols):
        print("[PLOT 4] Normalized odds...")

        sample = df[odds_cols + norm_cols].dropna().sample(min(1000, len(df)), random_state=42)

        sample['Prob_H_Raw'] = 1 / sample['OddHome']
        sample['Prob_D_Raw'] = 1 / sample['OddDraw']
        sample['Prob_A_Raw'] = 1 / sample['OddAway']
        sample['Sum_Raw'] = sample[['Prob_H_Raw', 'Prob_D_Raw', 'Prob_A_Raw']].sum(axis=1)
        sample['Sum_Norm'] = sample[norm_cols].sum(axis=1)

        ax4.hist(sample['Sum_Raw'], bins=40, alpha=0.9, color='coral',
                label='Raw (with overround)', edgecolor='black', linewidth=0.5)
        ax4.hist(sample['Sum_Norm'], bins=40, alpha=0.9, color='lightgreen',
                label='Normalized', edgecolor='black', linewidth=0.5)

        ax4.axvline(1.0, color='red', linestyle='--', linewidth=2.5,
                   label='Ideal Sum (1.0)', alpha=0.7)

        ax4.set_xlabel('Sum of Probabilities', fontsize=12, fontweight='bold')
        ax4.set_ylabel('Frequency', fontsize=12, fontweight='bold')
        ax4.set_title('Bookmaker Odds Normalization\n' +
                     'Remove profit margin',
                     fontsize=12, fontweight='bold', pad=15)
        ax4.legend(fontsize=9)
        ax4.grid(alpha=0.3)

        mean_raw = sample['Sum_Raw'].mean()
        mean_norm = sample['Sum_Norm'].mean()
        overround = (mean_raw - 1.0) * 100

        stats_text = f'Avg Sum:\nRaw: {mean_raw:.4f}\nNorm: {mean_norm:.4f}\nOverround: {overround:.2f}%'
        ax4.text(0.98, 0.98, stats_text, transform=ax4.transAxes,
                ha='right', va='top', fontsize=9,
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
    else:
        ax4.text(0.5, 0.5, 'Odds not available',
                ha='center', va='center', transform=ax4.transAxes)

    # ════════════════════════════════════════════════════════════════════
    # PLOT 5: Diff Features Correlation
    # ════════════════════════════════════════════════════════════════════
    ax5 = fig.add_subplot(gs[2, 0])

    diff_features = [c for c in df.columns if c.endswith('_Diff')]

    if len(diff_features) >= 3:
        print("[PLOT 5] Diff features correlation...")

        diff_data = df[diff_features].dropna()
        variances = diff_data.var().sort_values(ascending=False)
        top_diff = variances.head(8).index.tolist()

        corr = df[top_diff].corr()

        im = ax5.imshow(corr, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
        ax5.set_xticks(range(len(top_diff)))
        ax5.set_yticks(range(len(top_diff)))
        ax5.set_xticklabels([f[:20] for f in top_diff], rotation=45, ha='right', fontsize=8)
        ax5.set_yticklabels([f[:20] for f in top_diff], fontsize=8)

        for i in range(len(top_diff)):
            for j in range(len(top_diff)):
                text_color = 'white' if abs(corr.iloc[i, j]) > 0.5 else 'black'
                ax5.text(j, i, f'{corr.iloc[i, j]:.2f}',
                        ha='center', va='center', color=text_color,
                        fontsize=8, fontweight='bold')

        ax5.set_title('Diff Features Correlation\n(Home - Away)',
                     fontsize=12, fontweight='bold', pad=15)
        plt.colorbar(im, ax=ax5, label='Correlation', fraction=0.046, pad=0.04)
    else:
        ax5.text(0.5, 0.5, 'Insufficient Diff features',
                ha='center', va='center', transform=ax5.transAxes)

    # ════════════════════════════════════════════════════════════════════
    # PLOT 6: Lag Features (L3 vs L5)
    # ════════════════════════════════════════════════════════════════════
    ax6 = fig.add_subplot(gs[2, 1])

    lag3_features = [c for c in df.columns if 'L3' in c]
    lag5_features = [c for c in df.columns if 'L5' in c]

    if lag3_features and lag5_features:
        print("[PLOT 6] Lag features (L3 vs L5)...")

        l3_col = lag3_features[0]
        l5_col = l3_col.replace('L3', 'L5')

        if l5_col in df.columns:
            sample = df[[l3_col, l5_col]].dropna().sample(min(2000, len(df)), random_state=42)

            ax6.scatter(sample[l3_col], sample[l5_col],
                       alpha=0.4, s=30, c='purple',
                       edgecolors='black', linewidth=0.5)

            min_val = min(sample[l3_col].min(), sample[l5_col].min())
            max_val = max(sample[l3_col].max(), sample[l5_col].max())
            ax6.plot([min_val, max_val], [min_val, max_val],
                    'r--', linewidth=2.5, label='Perfect', alpha=0.7)

            from scipy import stats
            slope, intercept, r_value, _, _ = stats.linregress(sample[l3_col], sample[l5_col])
            line_x = np.array([min_val, max_val])
            line_y = slope * line_x + intercept
            ax6.plot(line_x, line_y, 'g-', linewidth=2.5,
                    label=f'Fit (r={r_value:.3f})', alpha=0.7)

            ax6.set_xlabel(f'{l3_col[:30]}...', fontsize=10, fontweight='bold')
            ax6.set_ylabel(f'{l5_col[:30]}...', fontsize=10, fontweight='bold')
            ax6.set_title('Lag Features: L3 vs L5\nWindow Comparison',
                         fontsize=12, fontweight='bold', pad=15)
            ax6.legend(fontsize=9)
            ax6.grid(alpha=0.3)

            corr_val = sample[[l3_col, l5_col]].corr().iloc[0, 1]
            ax6.text(0.98, 0.02, f'r = {corr_val:.4f}',
                    transform=ax6.transAxes, ha='right', va='bottom',
                    fontsize=12, fontweight='bold',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
        else:
            ax6.text(0.5, 0.5, 'Paired L5 not found',
                    ha='center', va='center', transform=ax6.transAxes)
    else:
        ax6.text(0.5, 0.5, 'Lag features not available',
                ha='center', va='center', transform=ax6.transAxes)

    # ════════════════════════════════════════════════════════════════════
    # SAVE
    # ════════════════════════════════════════════════════════════════════
    plt.suptitle('Feature Engineering Pipeline (Model v18.0)\n' +
                 'Derived features that improve prediction accuracy',
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout(rect=[0, 0, 1, 0.985])

    save_path = os.path.join(output_dir, '12_feature_engineering_examples.png')
    plt.savefig(save_path, dpi=600, bbox_inches='tight', facecolor='white')
    savefig_report('12_feature_engineering_examples.png', also_png=True)
    plt.close()

    print(f"\n  ✅ Saved: {os.path.basename(save_path)}")

    # ════════════════════════════════════════════════════════════════════
    # SUMMARY
    # ════════════════════════════════════════════════════════════════════
    print("\n" + "="*80)
    print("[SUMMARY] Feature Engineering Impact")
    print("="*80)

    print(f"\n  ✅ Difference Features: ELO_Diff, ValueRatio_Diff, etc.")
    print(f"  ✅ Ratio Features: ValueRatio = ClubValue / LeagueValue")
    print(f"  ✅ Log Transformations: Reduce skewness")
    print(f"  ✅ Normalized Probabilities: Remove overround")
    print(f"  ✅ Temporal Features: L3/L5 windows with shift(1)")

    n_engineered = len([c for c in df.columns if any(x in c for x in
                       ['_Diff', '_Ratio', '_log', 'Prob_', '_L3', '_L5'])])
    print(f"\n  📊 Total Engineered Features: {n_engineered}")

    print("\n" + "="*80 + "\n")

# ════════════════════════════════════════════════════════════════════════
# EXECUTE FEATURE ENGINEERING EXAMPLES
# ════════════════════════════════════════════════════════════════════════

try:
    visualize_feature_engineering_pipeline(
        df=eda_data,
        output_dir=IMAGES_DIR
    )
    print("[OK] Feature engineering examples complete.")
except Exception as e_fe:
    print(f"[WARN] Feature engineering visualization failed: {e_fe}")


print("\n[INFO] ========== END OF PART 2A ==========")
print("[INFO] Continue with PART 2B for Outlier Detection, Temporal Analysis, Chi-Square, and Summary")

# =============================================================================
# EDA V5.0 - PART 2B: OUTLIER DETECTION → FINAL SUMMARY
# Bu dosya PART 2A'nın devamıdır. PART 1 ve 2A'dan sonra çalıştırın.
# =============================================================================

print("\n[INFO] ========== STARTING PART 2B ==========")

# =============================================================================
# OUTLIER DETECTION & ANALYSIS
# =============================================================================
print("\n[INFO] PHASE 4.5: OUTLIER DETECTION & ANALYSIS (TRAIN SET ONLY)")
print("="*90)

outlier_found = False
outliers_by_feature = pd.Series(dtype=int)
outlier_indices = pd.Index([])
outlier_mask_df = pd.DataFrame()

try:
    if not numeric_cols:
        print("[WARN] No numeric columns for Outlier analysis. Skipping.")
    else:
        df_for_iqr = eda_data[numeric_cols].copy()

        all_nan_cols_iqr = df_for_iqr.columns[df_for_iqr.isnull().all()].tolist()
        if all_nan_cols_iqr:
            print(f"[WARN] IQR Outlier Check: Dropping all-NaN columns: {all_nan_cols_iqr}")
            df_for_iqr = df_for_iqr.drop(columns=all_nan_cols_iqr)

        if df_for_iqr.empty or df_for_iqr.shape[1] == 0:
            print("[WARN] No valid numeric data left for IQR calculation. Skipping outlier detection.")
        else:
            Q1 = df_for_iqr.quantile(0.25)
            Q3 = df_for_iqr.quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - (1.5 * IQR)
            upper_bound = Q3 + (1.5 * IQR)

            outlier_mask_df = (df_for_iqr < lower_bound) | (df_for_iqr > upper_bound)

            outlier_mask_any_row = outlier_mask_df.any(axis=1)

            outlier_indices = eda_data.index[outlier_mask_any_row]
            outlier_count_iqr = len(outlier_indices)
            outlier_found = outlier_count_iqr > 0

            outlier_threshold_iqr_str = "IQR (Q1 - 1.5*IQR or Q3 + 1.5*IQR)"
            print(f"[OK] Outlier detection method: {outlier_threshold_iqr_str}")
            print(f"     Total samples in training set: {len(eda_data):,}")
            if outlier_found:
                print(f"     Found {outlier_count_iqr:,} samples ({100*outlier_count_iqr/len(eda_data):.2f}%) with at least one outlier feature.")
            else:
                print("     No outliers found using IQR method.")

            outliers_by_feature = outlier_mask_df.sum(axis=0).sort_values(ascending=False)
            outliers_by_feature = outliers_by_feature[outliers_by_feature > 0]

            if not outliers_by_feature.empty:
                outliers_by_feature_df = pd.DataFrame({
                    'Feature': outliers_by_feature.index,
                    'Outlier_Count': outliers_by_feature.values,
                    'Percentage_of_Samples': (outliers_by_feature.values / len(eda_data) * 100).round(2)
                }).reset_index(drop=True)

                print(f"\n[OK] Features contributing to outliers (Top 15 or all if fewer):")
                print(outliers_by_feature_df.head(15))
                outliers_by_feature_df.to_csv(os.path.join(CSV_DIR, "10a_outliers_by_feature_train_IQR.csv"), index=False)
                print(f"[OK] Outlier counts by feature (IQR) saved to CSV.")
            else:
                print("\n[INFO] No individual features exceeded the IQR threshold.")

            if outlier_found:
                outliers_df = eda_data.loc[outlier_indices].copy()

                outlier_cols_list = []
                relevant_mask = outlier_mask_df.loc[outlier_indices]

                for idx, row in relevant_mask.iterrows():
                    cols = row[row].index.tolist()
                    outlier_cols_list.append(", ".join(cols))

                outliers_df['Outlier_Features_IQR'] = outlier_cols_list

                print(f"\n[INFO] Saving {len(outliers_df)} outlier samples details to CSV...")
                outliers_df.to_csv(os.path.join(CSV_DIR, "10b_outlier_samples_train_IQR.csv"), index=False)
                print(f"[OK] Outlier samples saved.")

                # Visualization - Box plots
                print("[INFO] Creating outlier visualizations (Boxplots for key features)...")
                features_for_boxplot = outliers_by_feature.head(12).index.tolist()
                if not features_for_boxplot:
                    key_features_list_check = [f for f in key_features_list if f in eda_data.columns]
                    features_for_boxplot = key_features_list_check[:12]

                if not features_for_boxplot:
                    print("[WARN] No features selected for outlier boxplots.")
                else:
                    num_box_plots = len(features_for_boxplot)
                    cols_box = 4
                    rows_box = (num_box_plots + cols_box - 1) // cols_box
                    fig_box, axes_box = plt.subplots(rows_box, cols_box, figsize=(4.5 * cols_box, 3.5 * rows_box), squeeze=False)
                    axes_box = axes_box.flatten()
                    plot_count_box = 0

                    for idx, feature in enumerate(features_for_boxplot):
                        ax = axes_box[idx]
                        if feature not in eda_data.columns:
                            continue

                        all_data = eda_data[feature].dropna()
                        non_outlier_data = eda_data.loc[~eda_data.index.isin(outlier_indices), feature].dropna()
                        outlier_data = eda_data.loc[outlier_indices, feature].dropna()

                        if not non_outlier_data.empty:
                            bp = ax.boxplot(non_outlier_data, positions=[0], widths=0.6, patch_artist=True, showfliers=False,
                                   boxprops=dict(facecolor='lightblue', alpha=0.8),
                                   medianprops=dict(color='red', linewidth=2))
                        else:
                            ax.text(0.5, 0.5, "No non-outlier data", ha='center', va='center', transform=ax.transAxes, fontsize=8)

                        if not outlier_data.empty:
                            jitter = np.random.normal(0, 0.04, size=len(outlier_data))
                            ax.scatter(jitter, outlier_data,
                                       color='red', s=40, alpha=0.9, label=f'Outliers (IQR) ({len(outlier_data)})', zorder=3, marker='x')

                        ax.set_xticks([])
                        ax.set_ylabel(feature, fontsize=9)
                        ax.set_title(f'{feature}', fontsize=10, fontweight='bold')
                        ax.grid(axis='y', alpha=0.4, linestyle='--')
                        if not outlier_data.empty:
                            ax.legend(loc='best', fontsize=7)
                        plot_count_box += 1

                    for i in range(plot_count_box, len(axes_box)):
                        axes_box[i].axis('off')

                    if plot_count_box > 0:
                        fig_box.suptitle('Distribution of Key Features Highlighting Outliers (IQR Method)', fontsize=14, fontweight='bold', y=1.02)
                        plt.tight_layout(pad=2.0)
                        savefig_report("10c_outlier_boxplots_IQR.png", also_png=True)
                        print("[OK] Outlier boxplots (IQR) visualization saved.")
                    else:
                        plt.close(fig_box)

            else:
                print("[INFO] No outliers detected with IQR method. Skipping detailed outlier analysis and plots.")

except ImportError:
    print("[WARN] Could not import required libraries for IQR (e.g., numpy/pandas). Skipping Outlier detection.")
except Exception as e_outlier:
    print(f"[ERROR] Outlier analysis (IQR) failed: {e_outlier}")
    try:
        plt.close(fig_box)
    except:
        pass


# =============================================================================
# TEMPORAL & SEASONALITY ANALYSIS
# =============================================================================
print("\n[INFO] PHASE 4.7: TEMPORAL & SEASONALITY ANALYSIS (TRAIN SET ONLY)")
print("="*90)

try:
    if 'Season' not in eda_data.columns:
        if 'MatchDate' in eda_data.columns:
            eda_data['Season'] = eda_data['MatchDate'].dt.year
            print("[INFO] 'Season' column (year) created from MatchDate for temporal analysis.")
        else:
            print("[WARN] 'MatchDate' column not found. Cannot create 'Season' column. Skipping temporal analysis.")
            raise ValueError("'MatchDate' column missing.")

    agg_dict_temporal = {
        'Target': ['count', lambda x: (x == 1).sum() / len(x) * 100 if len(x) > 0 else 0]
    }
    if 'HomeElo' in eda_data.columns: agg_dict_temporal['HomeElo'] = 'mean'
    if 'AwayElo' in eda_data.columns: agg_dict_temporal['AwayElo'] = 'mean'

    # 🆕 Temporal averages için analiz
    if 'Home_Target_Avg_L5' in eda_data.columns: agg_dict_temporal['Home_Target_Avg_L5'] = 'mean'
    if 'Away_Target_Avg_L5' in eda_data.columns: agg_dict_temporal['Away_Target_Avg_L5'] = 'mean'

    print("[INFO] Calculating temporal trends by season (year)...")
    seasonal_trends = eda_data.groupby('Season').agg(agg_dict_temporal).round(2)

    new_cols = []
    for col in seasonal_trends.columns:
        if isinstance(col, tuple):
            if col[1] == '<lambda_0>': new_cols.append('Home_Win_Pct')
            elif col[1] == 'count': new_cols.append('Match_Count')
            else: new_cols.append(f"{col[0]}_{col[1].capitalize()}")
        else:
            new_cols.append(col)
    seasonal_trends.columns = new_cols

    print(f"\n[OK] Temporal trends by season (year) for Train Set:")
    print(seasonal_trends)

    seasonal_trends.to_csv(os.path.join(CSV_DIR, "11a_temporal_trends_by_season_train.csv"))
    print("[OK] Seasonal trends saved to CSV.")

    print("[INFO] Creating temporal analysis visualization (Train Set)...")
    fig_temp, axes_temp = plt.subplots(2, 2, figsize=(15, 11))
    axes_temp = axes_temp.flatten()
    plotted_temp_any = False

    # Plot 1: ELO Trends
    ax = axes_temp[0]
    plotted_elo = False
    if 'HomeElo_Mean' in seasonal_trends.columns:
        ax.plot(seasonal_trends.index, seasonal_trends['HomeElo_Mean'],
                marker='o', label='Avg Home Elo', linewidth=2, markersize=6, color=UNIFIED_COLORS['main_blue'])
        plotted_elo = True
    if 'AwayElo_Mean' in seasonal_trends.columns:
        ax.plot(seasonal_trends.index, seasonal_trends['AwayElo_Mean'],
                marker='s', label='Avg Away Elo', linewidth=2, markersize=6, color='#e74c3c')
        plotted_elo = True
    if plotted_elo:
        ax.set_ylabel('Average Elo Rating', fontweight='bold')
        ax.set_title('ELO Rating Trend ', fontsize=12, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(alpha=0.4, linestyle='--')
        ax.set_xlabel('Season (Year)')
        plotted_temp_any = True
    else: ax.text(0.5, 0.5, 'ELO data not available', ha='center', va='center', transform=ax.transAxes); ax.set_title('ELO Trend')

    # Plot 2: Home Advantage Trend
    ax = axes_temp[1]
    if 'Home_Win_Pct' in seasonal_trends.columns:
        bars_ha = ax.bar(seasonal_trends.index, seasonal_trends['Home_Win_Pct'],
                         color=UNIFIED_COLORS['main_blue'], alpha=0.9, edgecolor='black', linewidth=1)
        ax.set_ylabel('Home Win %', fontweight='bold')
        ax.set_title('Home Advantage Trend ', fontsize=12, fontweight='bold')
        ax.grid(axis='y', alpha=0.4, linestyle='--')
        ax.set_xlabel('Season (Year)')
        min_hw_trend = seasonal_trends['Home_Win_Pct'].min()
        max_hw_trend = seasonal_trends['Home_Win_Pct'].max()
        ax.set_ylim([max(0, np.floor(min_hw_trend / 5)*5 - 2), min(100, np.ceil(max_hw_trend / 5)*5 + 2)])
        plotted_temp_any = True
    else: ax.text(0.5, 0.5, 'Home Win % data not available', ha='center', va='center', transform=ax.transAxes); ax.set_title('Home Advantage Trend')

    # Plot 3: Temporal Features Trends (NEW!)
    ax = axes_temp[2]
    plotted_temporal = False
    if 'Home_Target_Avg_L5_Mean' in seasonal_trends.columns:
        ax.plot(seasonal_trends.index, seasonal_trends['Home_Target_Avg_L5_Mean'],
                marker='o', label='Avg Home Target (L5)', linewidth=2, markersize=6, color='#2ecc71')
        plotted_temporal = True
    if 'Away_Target_Avg_L5_Mean' in seasonal_trends.columns:
        ax.plot(seasonal_trends.index, seasonal_trends['Away_Target_Avg_L5_Mean'],
                marker='s', label='Avg Away Target (L5)', linewidth=2, markersize=6, color='#f39c12')
        plotted_temporal = True
    if plotted_temporal:
        ax.set_ylabel('Average Shots on Target (L5)', fontweight='bold')
        ax.set_title('Temporal Features Trend', fontsize=12, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(alpha=0.4, linestyle='--')
        ax.set_xlabel('Season (Year)')
        plotted_temp_any = True
    else: ax.text(0.5, 0.5, 'Temporal feature data not available', ha='center', va='center', transform=ax.transAxes); ax.set_title('Temporal Features Trend')

    # Plot 4: Data Availability
    try:
        ax = axes_temp[3]
        if 'Match_Count' in seasonal_trends.columns:
            ax.bar(seasonal_trends.index, seasonal_trends['Match_Count'],
                   color='#95a5a6', alpha=0.7, edgecolor='black', linewidth=1)
            ax.set_ylabel('Number of Matches', fontweight='bold')
            ax.set_title('Data Availability by Season', fontsize=12, fontweight='bold')
            ax.grid(axis='y', alpha=0.4, linestyle='--')
            ax.set_xlabel('Season (Year)')
            plotted_temp_any = True
        else:
            ax.text(0.5, 0.5, 'Match count data not available', ha='center', va='center',
                    transform=ax.transAxes)
            ax.set_title('Data Availability')

        if plotted_temp_any:
            plt.tight_layout()
            savefig_report("11b_temporal_analysis_4panel_train.png", also_png=True)
            print("[OK] Temporal analysis visualization saved.")
        else:
            plt.close(fig_temp)

    except Exception as e_plot_4:
        print(f"[WARN] Failed during plot 4 or saving: {e_plot_4}")
        try:
            plt.close(fig_temp)
        except:
            pass

except Exception as e_temporal:
    print(f"[ERROR] Temporal analysis failed: {e_temporal}")
    try:
        plt.close(fig_temp)
    except:
        pass


# =============================================================================
# CHI-SQUARE TEST
# =============================================================================
print("\n[INFO] Performing Chi-Square Test for Categorical Features (TRAIN SET ONLY)")
print("-" * 90)

chi_square_df = pd.DataFrame()
chi_square_validation_df = pd.DataFrame()

try:
    if 'Target' not in eda_data.columns:
        print("[WARN] Target column missing. Skipping Chi-Square test.")
    else:
        categorical_features_all = eda_data.select_dtypes(include=['category', 'object']).columns.tolist()

        exclude_features_chi = [
            'HomeTeam', 'AwayTeam',
            'MatchDate',
            'YearMonth',
            'Season',
            'FTResult',
            'Target',
            'HTResult'
        ]
        if 'Referee' in categorical_features_all: exclude_features_chi.append('Referee')
        for col in ['HomeTeam_ManagerName', 'AwayTeam_ManagerName']:
            if col in categorical_features_all: exclude_features_chi.append(col)

        categorical_features = [f for f in categorical_features_all if f not in exclude_features_chi]

        if 'Target' in categorical_features: categorical_features.remove('Target')
        if 'FTResult' in categorical_features: categorical_features.remove('FTResult')

        if not categorical_features:
            print("[INFO] No suitable categorical features found for Chi-Square test after exclusions.")
        else:
            print(f"[INFO] Testing {len(categorical_features)} categorical features against Target:")
            print(f"         {', '.join(categorical_features)}\n")

            chi_square_results = []
            chi_square_validation = []

            bonferroni_alpha = 0.05 / len(categorical_features)
            print(f"[INFO] Multiple Comparison Correction:")
            print(f"         Original α = 0.05")
            print(f"         Bonferroni Adjusted α = {bonferroni_alpha:.6f} ({len(categorical_features)} tests)\n")

            for feature in categorical_features:
                if eda_data[feature].nunique() > 50:
                    print(f"  ✗ {feature}: Skipped (High Cardinality > 50 uniques)")
                    continue
                try:
                    ct = pd.crosstab(eda_data[feature], eda_data['Target'])
                    if ct.empty or ct.shape[0] < 2 or ct.shape[1] < 2:
                        print(f"  ✗ {feature}: Skipped (Insufficient data for cross-tabulation)")
                        continue

                    chi2, p_value, dof, expected = chi2_contingency(ct)

                    valid_test = True
                    warning_msg = ""
                    if expected.min() < 1:
                        valid_test = False
                        warning_msg = "⚠ (Expected < 1)"
                    elif (expected < 5).sum() / expected.size > 0.2:
                        valid_test = False
                        warning_msg = "⚠ (>20% Expected < 5)"

                    significant_standard = p_value < 0.05
                    significant_bonferroni = p_value < bonferroni_alpha

                    result = {
                        'Feature': feature,
                        'Chi2': chi2,
                        'p-value': p_value,
                        'df': dof,
                        'Significant (α=0.05)': 'Yes' if significant_standard else 'No',
                        'Significant (Bonferroni)': 'Yes' if significant_bonferroni else 'No',
                        'Test_Valid': 'Yes' if valid_test else 'No'
                    }
                    chi_square_results.append(result)

                    validation = {
                        'Feature': feature,
                        'Min_Expected_Freq': expected.min(),
                        'Pct_Expected_LT5': (expected < 5).sum() / expected.size * 100,
                        'Valid': 'Yes' if valid_test else 'No'
                    }
                    chi_square_validation.append(validation)

                    status_icon = "✓" if significant_standard else "✗"
                    print(f"  {status_icon} {feature}: Chi2={chi2:.2f}, p={p_value:.4f}, Valid={valid_test} {warning_msg}")

                except Exception as e_chi:
                    print(f"  [ERROR] Chi-Square failed for {feature}: {str(e_chi)}")

            if chi_square_results:
                chi_square_df = pd.DataFrame(chi_square_results)
                chi_square_validation_df = pd.DataFrame(chi_square_validation)

                chi_square_df.round(4).to_csv(os.path.join(CSV_DIR, "02a_chi_square_results_train.csv"), index=False)
                chi_square_validation_df.round(2).to_csv(os.path.join(CSV_DIR, "02b_chi_square_validation_train.csv"), index=False)
                print("\n[OK] Chi-Square results and validation saved to CSV.")

                save_df_as_image(chi_square_df.round(4), "02a_chi_square_results_table.png",
                                     title='Table 2: Chi-Square Test of Independence ')
                save_df_as_image(chi_square_validation_df.round(2), "02b_chi_square_validation_table.png",
                                     title='Chi-Square Assumption Validation ')

                fig_chi_bars, axes_chi_bars = plt.subplots(1, 2, figsize=(16, max(6, 0.4 * len(chi_square_df))))
                colors_chi = ['#2ecc71' if row['Significant (α=0.05)'] == 'Yes' else '#95a5a6' for _, row in chi_square_df.iterrows()]
                bars_chi_val = axes_chi_bars[0].barh(chi_square_df['Feature'], chi_square_df['Chi2'], color=colors_chi)
                axes_chi_bars[0].set_xlabel('Chi-Square Statistic', fontweight='bold')
                axes_chi_bars[0].set_title('Chi-Square Test Statistic ', fontweight='bold', fontsize=12)
                axes_chi_bars[0].invert_yaxis()
                axes_chi_bars[0].grid(axis='x', alpha=0.3)

                log_p_values = -np.log10(chi_square_df['p-value'].replace(0, 1e-300))
                bars_chi_p = axes_chi_bars[1].barh(chi_square_df['Feature'], log_p_values, color=colors_chi)
                axes_chi_bars[1].set_xlabel('-log10(p-value)', fontweight='bold')
                axes_chi_bars[1].set_title('Statistical Significance ', fontweight='bold', fontsize=12)
                axes_chi_bars[1].axvline(-np.log10(0.05), color='red', linestyle='--', linewidth=2, label='α = 0.05')
                axes_chi_bars[1].axvline(-np.log10(bonferroni_alpha), color='orange', linestyle=':', linewidth=2, label=f'Bonferroni α ({bonferroni_alpha:.2E})')
                axes_chi_bars[1].invert_yaxis()
                axes_chi_bars[1].grid(axis='x', alpha=0.3)
                axes_chi_bars[1].legend()
                plt.tight_layout()
                savefig_report("12_chi_square_test_bars.png", also_png=True)
                print("[OK] Chi-Square results visualized.")

            else:
                print("[INFO] No Chi-Square tests were successfully performed.")

except ImportError:
    print("[WARN] Could not import chi2_contingency from scipy.stats. Skipping Chi-Square test.")
except Exception as e_chisq_main:
    print(f"[ERROR] Chi-Square analysis failed: {e_chisq_main}")


# =============================================================================
# DATA QUALITY ANALYSIS (Missing Values)
# =============================================================================
print("\n[INFO] Performing Data Quality Analysis (Missing Values, TRAIN SET ONLY)...")

try:
    if not numeric_cols:
         print("[WARN] No numeric columns to check for missing values. Skipping.")
    else:
        missing_pct = (eda_data[numeric_cols].isnull().sum() / len(eda_data) * 100).sort_values(ascending=False)
        missing_pct = missing_pct[missing_pct > 0]

        if missing_pct.empty:
            print("[OK] No missing values found in the selected numeric features for the training set.")
        else:
            print(f"[WARN] Found {len(missing_pct)} numeric features with missing values in the training set (Top 15):")
            print(missing_pct.head(15))

            fig_missing, ax_missing = plt.subplots(figsize=(12, 8))
            n_show_missing = min(20, len(missing_pct))
            missing_data_plot = missing_pct.head(n_show_missing)

            colors_quality = ['#e74c3c' if x > 10 else '#f39c12' if x > 5 else '#2ecc71'
                              for x in missing_data_plot.values]

            bars_missing = ax_missing.barh(range(n_show_missing), missing_data_plot.values,
                                       color=colors_quality, height=0.7, edgecolor='black')
            ax_missing.set_yticks(range(n_show_missing))
            ax_missing.set_yticklabels(missing_data_plot.index, fontsize=10)
            ax_missing.invert_yaxis()
            ax_missing.set_xlabel('Percentage of Missing Values (%)', fontweight='bold', fontsize=12)
            ax_missing.set_title(f'Data Quality - Top {n_show_missing} Numeric Features with Missing Values',
                                 fontweight='bold', fontsize=12, pad=15)
            max_missing = missing_data_plot.max()
            ax_missing.set_xlim([0, max(10, max_missing * 1.1)])
            ax_missing.grid(axis='x', alpha=0.3)

            for i, (bar, val) in enumerate(zip(bars_missing, missing_data_plot.values)):
                 ax_missing.text(val + max(1, max_missing*0.01), i, f'{val:.1f}%', va='center', fontsize=9, fontweight='bold')

            plt.tight_layout()
            savefig_report("13_data_quality_missing_values.png", also_png=True)
            print("[OK] Missing values analysis plot saved.")

            missing_pct.to_csv(os.path.join(CSV_DIR, "missing_values_train.csv"))
            print("[OK] Missing values percentages saved to CSV.")

except Exception as e_missing:
    print(f"[ERROR] Missing value analysis failed: {e_missing}")
    try: plt.close(fig_missing)
    except: pass


# =============================================================================
# PHASE 5: SUMMARY
# =============================================================================
print("\n[INFO] PHASE 5: GENERATING EDA SUMMARY (Based on Train Set Analysis)")
print("="*90)

try:
    train_n = len(train_matches) if 'train_matches' in locals() else 'N/A'
    val_n = len(val_matches) if 'val_matches' in locals() else 'N/A'
    test_n = len(test_matches) if 'test_matches' in locals() else 'N/A'
    total_n = len(matches_all) if 'matches_all' in locals() else 'N/A'

    target_dist_summary = "N/A"
    if 'vc' in locals() and isinstance(vc, pd.Series):
        target_dist_summary = f"Home Win: {vc.get(1,0):.1f}%, Draw: {vc.get(0,0):.1f}%, Away Win: {vc.get(2,0):.1f}%"

    num_feat_summary = len(numeric_cols) if 'numeric_cols' in locals() else 'N/A'
    key_feat_summary = len(key_features) if 'key_features' in locals() else 'N/A'

    # 🆕 Temporal features count
    temporal_feat_summary = len(temporal_features_created) if 'temporal_features_created' in locals() else 0

    top_feature_1, top_feature_2, top_feature_3 = "N/A", "N/A", "N/A"
    if 'mi_scores' in locals() and isinstance(mi_scores, pd.Series) and not mi_scores.empty and len(mi_scores) >= 3:
        top_feature_1 = f"{mi_scores.index[0]} (MI: {mi_scores.values[0]:.3f})"
        top_feature_2 = f"{mi_scores.index[1]} (MI: {mi_scores.values[1]:.3f})"
        top_feature_3 = f"{mi_scores.index[2]} (MI: {mi_scores.values[2]:.3f})"

    avg_pre_summary = f"{avg_pre:.1f}%" if 'avg_pre' in locals() and pd.notna(avg_pre) else "N/A"
    avg_pandemic_summary = "N/A"
    avg_post_summary = "N/A"
    pandemic_impact_str = ""
    post_impact_str = ""
    if 'avg_pandemic' in locals() and pd.notna(avg_pandemic):
        avg_pandemic_summary = f"{avg_pandemic:.1f}%"
        if 'avg_pre' in locals() and pd.notna(avg_pre):
            pandemic_impact_str = f" ({avg_pandemic - avg_pre:+.1f}pp vs Pre)"
    if 'avg_post' in locals() and pd.notna(avg_post):
        avg_post_summary = f"{avg_post:.1f}%"
        if 'avg_pre' in locals() and pd.notna(avg_pre):
            post_impact_str = f" ({avg_post - avg_pre:+.1f}pp vs Pre)"

    cat_feat_summary = ', '.join(categorical_features) if 'categorical_features' in locals() and categorical_features else 'None'
    sig_std_summary = "N/A"
    sig_bonf_summary = "N/A"
    bonf_alpha_summary = "N/A"
    if 'chi_square_df' in locals() and isinstance(chi_square_df, pd.DataFrame) and not chi_square_df.empty:
        sig_std_list = chi_square_df[chi_square_df['Significant (α=0.05)'] == 'Yes']['Feature'].tolist()
        sig_std_summary = ', '.join(sig_std_list) if sig_std_list else 'None'
        sig_bonf_list = chi_square_df[chi_square_df['Significant (Bonferroni)'] == 'Yes']['Feature'].tolist()
        sig_bonf_summary = ', '.join(sig_bonf_list) if sig_bonf_list else 'None'
        if 'bonferroni_alpha' in locals():
            bonf_alpha_summary = f"{bonferroni_alpha:.4f}"

    high_corr_summary = "N/A"
    features_dropped_count = 0
    if 'high_corr_df' in locals() and isinstance(high_corr_df, pd.DataFrame) and not high_corr_df.empty:
        high_corr_summary = f"{len(high_corr_df)} pairs found (see '09_high_correlation_pairs.csv')"
        if 'features_to_drop_list' in locals():
            features_dropped_count = len(features_to_drop_list)

    tsne_summary = "Suggests non-linear separation needed"

    summary_insights = f"""
    =============================================================
    EDA SUMMARY - V5.0 (DATA LEAKAGE CLEANED + TEMPORAL FEATURES)
    =============================================================

    🚀 MAJOR CHANGES IN V5.0:
       ✅ ALL match-event features converted to temporal averages (L3/L5)
       ✅ Data leakage completely eliminated
       ✅ {temporal_feat_summary} temporal features created
       ✅ {features_dropped_count} features identified for removal (multicollinearity)

    1. DATASET OVERVIEW
       - Total Processed Matches (2015-Present): {total_n:,}
       - Training Set (Used for this EDA): {train_n:,} matches
       - Validation Set (Pandemic Gap Excluded): {val_n:,} matches
       - Test Set (Most Recent): {test_n:,} matches
       - Leagues Analyzed: 5 (PL, Bundesliga, Serie A, La Liga, Ligue 1)

    2. TARGET DISTRIBUTION (Train Set)
       - {target_dist_summary}
       - Imbalance observed: Home wins are most frequent.

    3. DATA QUALITY (Train Set)
       - Duplicates: Checked and likely 0 after processing.
       - Missing Values: Analyzed in numeric features (see '13_data_quality...' plot/CSV).

    4. FEATURE ANALYSIS (Train Set)
       - Total Numeric Features Analyzed: {num_feat_summary}
       - Key Features Examined: {key_feat_summary}
       - Temporal Features Created: {temporal_feat_summary}

       📊 Top 3 Features by Relevance (Mutual Information - LEAKAGE-FREE):
         1. {top_feature_1}
         2. {top_feature_2}
         3. {top_feature_3}

       ⚠️  CRITICAL - MULTICOLLINEARITY DETECTED:
         * High correlations (>0.85): {high_corr_summary}
         * Features to DROP: {features_dropped_count} (see '09b_features_to_drop_code.py')
         * MANDATORY: Apply these removals BEFORE modeling
         * Rationale: Redundant normalized probabilities, Max vs Odd duplicates

       ✅ DATA LEAKAGE PREVENTION:
         * Original match-event features (HomeTarget, HomeShots, etc.) → REMOVED
         * Replaced with: Historical averages (L3, L5 windows)
         * Example: HomeTarget → Home_Target_Avg_L5
         * This ensures NO future information leaks into predictions

       - Outliers: Detected via IQR (see '10a/b/c...' plots/CSVs).
         Strategy: RobustScaler + log transformation for skewed features recommended.

    5. PANDEMIC IMPACT (Full Data Context)
       - Average Home Win %:
         - Pre-Pandemic: {avg_pre_summary}
         - During Pandemic: {avg_pandemic_summary}{pandemic_impact_str}
         - Post-Pandemic: {avg_post_summary}{post_impact_str}
       - Conclusion: Pandemic significantly reduced home advantage, partially recovered post-pandemic.

    6. TEMPORAL TRENDS (Train Set)
       - ELO: Slight upward trend over seasons (see '11b...' plot).
       - Home Advantage: Fluctuates seasonally (see '11b...' plot).
       - Temporal Features: Stable patterns in L5 averages (see '11b...' plot).

    7. CATEGORICAL FEATURES (Train Set vs Target)
       - Features Tested: {cat_feat_summary}
       - Significant Features (α=0.05): {sig_std_summary}
       - Significant Features (Bonferroni α≈{bonf_alpha_summary}): {sig_bonf_summary}
       - Validation: Chi-Square assumptions checked (see '02b...' CSV/Table).

    8. DIMENSIONALITY & SEPARATION (Train Set)
       - t-SNE Plot: {tsne_summary}. Classes appear highly overlapping.

    9. 🎯 CRITICAL ACTION ITEMS FOR MODELING

       ⭐ PRIORITY 1: REMOVE MULTICOLLINEAR FEATURES
       ```python
       # Load the auto-generated drop list
       exec(open('09b_features_to_drop_code.py').read())
       X_train = X_train.drop(columns=features_to_drop_multicollinearity)
       X_val = X_val.drop(columns=features_to_drop_multicollinearity)
       X_test = X_test.drop(columns=features_to_drop_multicollinearity)
       ```

       ⭐ PRIORITY 2: VERIFY NO LEAKAGE FEATURES REMAIN
       ```python
       leakage_check = ['FTHome', 'FTAway', 'HTHome', 'HTAway',
                        'HomeTarget', 'AwayTarget', 'HomeShots', 'AwayShots',
                        'HomeCorners', 'AwayCorners', 'HomeFouls', 'AwayFouls']
       assert not any(feat in X_train.columns for feat in leakage_check), "LEAKAGE DETECTED!"
       ```

       ⭐ PRIORITY 3: FEATURE SELECTION (MI-Based)
       ```python
       # Use features with MI > 0.04 (see '02_feature_relevance...' CSV)
       selected_features = mi_scores[mi_scores > 0.04].index.tolist()
       ```

       ⭐ PRIORITY 4: HANDLE OUTLIERS & SKEWNESS
       ```python
       from sklearn.preprocessing import RobustScaler
       import numpy as np

       # Log transform skewed financial features
       skewed_features = ['ClubValue', 'MaxPlayerValue', 'ManagerTrophies']
       for feat in skewed_features:
           for prefix in ['HomeTeam_', 'AwayTeam_']:
               col = f'{{prefix}}{{feat}}'
               if col in X_train.columns:
                   X_train[f'{{col}}_log'] = np.log1p(X_train[col])

       # Apply RobustScaler (IQR-based, outlier-resistant)
       scaler = RobustScaler()
       X_train_scaled = scaler.fit_transform(X_train)
       ```

       OTHER RECOMMENDATIONS:
       - Address Class Imbalance: Use class_weight='balanced' (PREFERRED)
       - SMOTE (if needed): Apply ONLY in modeling pipeline, NEVER in EDA
       - Model Selection: Non-linear models (XGBoost, LGBM, RF) recommended
       - Reason: EDA analyzes REAL data, SMOTE creates SYNTHETIC data
       - Evaluation Metrics: F1-weighted, Kappa, Brier Score, RPS, ECE
       - TEMPORAL SPLIT IS MANDATORY: NO random shuffling allowed

    10. FILES GENERATED (V5.0)
        - PNG Visualizations: ~27+ (in {IMAGES_DIR})
        - CSV Data Exports: ~15+ (in {CSV_DIR})
        - PDF Report(s): (in {COMPACT_DIR})
        - Python Code: features_to_drop_multicollinearity list

        **CRITICAL FILES FOR MODELING:**
        1. ⭐ 09b_features_to_drop_code.py → MANDATORY: Remove multicollinear features
        2. ⭐ 02_feature_relevance_mutual_information_train.csv → Feature selection guide
        3. 09_high_correlation_pairs.csv → Review correlation issues
        4. 09c_multicollinearity_recommendations.csv → Detailed removal rationale
        5. 01_descriptive_statistics_train.csv → Understand feature scales
        6. 10a_outliers_by_feature_train_IQR.csv → Confirm robust scaling needs

    11. ✅ VALIDATION CHECKLIST BEFORE MODELING:
        □ Loaded '09b_features_to_drop_code.py' and applied removals
        □ Verified NO leakage features (FTHome, HomeTarget, etc.) in X_train
        □ Confirmed temporal features (L3/L5 averages) are present
        □ Applied log transformation to skewed financial features
        □ Used RobustScaler for outlier-resistant scaling
        □ Selected top N features based on MI scores (>0.04 threshold)
        □ Used temporal split (NO random shuffling)
        □ Set appropriate class weights for imbalance
    """

    print(summary_insights)

    summary_path = os.path.join(OUT_DIR, "eda_summary_v5_0_leakage_cleaned.txt")
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write(summary_insights)
    print(f"\n[OK] EDA Summary saved to: {summary_path}")

except Exception as e_summary:
    print(f"[ERROR] Failed to generate summary: {e_summary}")


# =============================================================================
# FINAL CLEANUP
# =============================================================================
print("\n" + "="*90)
print("[DONE] FULL EDA COMPLETE - V5.0 (DATA LEAKAGE CLEANED + TEMPORAL FEATURES)")
print("Updates: All match-event features converted to temporal averages")
print("         Multicollinearity automatically detected and drop list generated")
print("         Complete leakage prevention implemented")
print("="*90)

try:
    output_stats = {
        "PNG Images": len(glob.glob(os.path.join(IMAGES_DIR, "*.png"))),
        "CSV Files": len(glob.glob(os.path.join(CSV_DIR, "*.csv"))),
        "HTML Files": len(glob.glob(os.path.join(IMAGES_DIR, "*.html"))),
        "PDF Files": len(glob.glob(os.path.join(COMPACT_DIR, "*.pdf"))),
        "Text Files": len(glob.glob(os.path.join(OUT_DIR, "*.txt"))),
        "Python Code Files": len(glob.glob(os.path.join(CSV_DIR, "*.py")))
    }
    print(f"\n[FINAL OUTPUT STATS]")
    for key, value in output_stats.items():
        print(f"  {key}: {value}")

    print(f"\nOutput directories:")
    print(f"  - Images: {IMAGES_DIR}")
    print(f"  - Data Exports: {CSV_DIR}")
    print(f"  - PDF Reports: {COMPACT_DIR}")
    print(f"  - Summary Text: {OUT_DIR}\n")
except Exception as e_stats:
    print(f"[WARN] Could not gather final output stats: {e_stats}")

try:
    _pdf.close()
    print("[OK] Final PDF file closed successfully.")
except Exception as e_pdf_close:
    print(f"[WARN] PDF close error (might be already closed or empty): {e_pdf_close}")

SCRIPT_END = time.time()
elapsed = SCRIPT_END - SCRIPT_START
print(f"\n[TIMER] Total EDA execution time: {elapsed:.2f} seconds ({elapsed/60:.2f} minutes)")
print(f"[INFO] Script V5.1 ended at {time.strftime('%Y-%m-%d %H:%M:%S')}")


# =============================================================================
# 🆕 FINAL: EKLEME KONTROLÜ (V5.1 Validation)
# =============================================================================
print("\n" + "="*100)
print("📋 EKLEME KONTROLÜ (V5.1 New Features)")
print("="*100)

checks = {
    'SMOTE Available': SMOTE_AVAILABLE,
    'VIF Available': VIF_AVAILABLE,
    'Silhouette Available': SILHOUETTE_AVAILABLE,
    'Temporal Validation': 'validation_results' in locals() and validation_results is not None,
    'SMOTE Metrics': 'smote_metrics' in locals() and smote_metrics is not None,
    'VIF Results': 'vif_results' in locals() and vif_results is not None,
    't-SNE Results': 'tsne_results' in locals() and tsne_results is not None
}

print("\n[KÜTÜPHANE DURUMU]")
for check_name in ['SMOTE Available', 'VIF Available', 'Silhouette Available']:
    status = checks[check_name]
    symbol = "✅" if status else "❌"
    print(f"  {symbol} {check_name:25s}: {status}")

print("\n[ANALİZ SONUÇLARI]")
for check_name in ['Temporal Validation', 'SMOTE Metrics', 'VIF Results', 't-SNE Results']:
    status = checks[check_name]
    symbol = "✅" if status else "❌"
    print(f"  {symbol} {check_name:25s}: {status}")

# Detaylı sonuçlar
all_passed = all(checks.values())

if all_passed:
    print("\n" + "="*100)
    print("🎉 TÜM EKLEMELER BAŞARIYLA ÇALIŞTI!")
    print("="*100)

    # Ekstra detaylar
    if 'validation_results' in locals() and validation_results:
        print(f"\n[Temporal Validation Details]")
        print(f"  • Leakage Detected: {validation_results.get('leakage_detected', 'N/A')}")
        print(f"  • Min Coverage: {validation_results.get('min_coverage', 0):.1f}%")
        print(f"  • Overall Pass: {validation_results.get('overall_pass', False)}")

    if 'smote_metrics' in locals() and smote_metrics:
        print(f"\n[SMOTE Metrics Details]")
        print(f"  • Success: {smote_metrics.get('smote_success', False)}")
        print(f"  • Imbalance Before: {smote_metrics.get('imbalance_before', 0):.2f}x")
        print(f"  • Imbalance After: {smote_metrics.get('imbalance_after', 0):.2f}x")

    if 'vif_results' in locals() and vif_results is not None:
        print(f"\n[VIF Results Details]")
        n_high_vif = (vif_results['VIF'] >= 10).sum()
        print(f"  • Features Analyzed: {len(vif_results)}")
        print(f"  • High VIF (≥10): {n_high_vif}")
        if 'vif_drop_list' in locals():
            print(f"  • Recommended Drops: {len(vif_drop_list)}")

    if 'tsne_results' in locals() and tsne_results:
        print(f"\n[t-SNE Results Details]")
        print(f"  • Embeddings Computed: {len(tsne_results.get('embeddings', {}))}")
        print(f"  • Sample Size: {tsne_results.get('sample_size', 0):,}")
        print(f"  • Features Used: {tsne_results.get('n_features', 0)}")

else:
    print("\n" + "="*100)
    print("⚠️  BAZI EKLEMELER ÇALIŞMADI - KONTROL GEREKLİ")
    print("="*100)

    print("\n[SORUN TESPİTİ]")

    if not checks['SMOTE Available']:
        print("  ❌ SMOTE kütüphanesi yüklü değil")
        print("     → Çözüm: pip install imbalanced-learn")

    if not checks['VIF Available']:
        print("  ❌ VIF (statsmodels) yüklü değil")
        print("     → Çözüm: pip install statsmodels")

    if not checks['Silhouette Available']:
        print("  ❌ Silhouette Score yüklü değil")
        print("     → Çözüm: pip install scikit-learn (güncel versiyon)")

    if not checks['Temporal Validation']:
        print("  ❌ Temporal Validation çalışmadı")
        print("     → Kontrol: Phase 1.6 kodu doğru eklenmiş mi?")
        print("     → validation_results değişkeni oluşturulmuş mu?")

    if not checks['SMOTE Metrics']:
        print("  ❌ SMOTE Metrics çalışmadı")
        print("     → Kontrol: Phase 4.3 kodu doğru eklenmiş mi?")
        print("     → SMOTE_AVAILABLE = True mu?")

    if not checks['VIF Results']:
        print("  ❌ VIF Results çalışmadı")
        print("     → Kontrol: Phase 4.4 kodu doğru eklenmiş mi?")
        print("     → VIF_AVAILABLE = True mu?")
        print("     → numeric_cols listesi yeterli mi? (>5 özellik gerekli)")

    if not checks['t-SNE Results']:
        print("  ❌ t-SNE Results çalışmadı")
        print("     → Kontrol: t-SNE kodu doğru replace edilmiş mi?")
        print("     → Target column mevcut mu?")

print("\n" + "="*100)

# Dosya çıktı kontrolü
try:
    output_files = {
        'PNG Images': len(glob.glob(os.path.join(IMAGES_DIR, "*.png"))),
        'CSV Files': len(glob.glob(os.path.join(CSV_DIR, "*.csv"))),
        'PDF Files': len(glob.glob(os.path.join(COMPACT_DIR, "*.pdf"))),
        'Text Files': len(glob.glob(os.path.join(OUT_DIR, "*.txt")))
    }

    print("\n[DOSYA ÇIKTI İSTATİSTİKLERİ]")
    for file_type, count in output_files.items():
        print(f"  📁 {file_type:15s}: {count:3d} dosya")

    # Yeni eklemelere özel dosyalar
    print("\n[YENİ EKLEME DOSYALARI (V5.1)]")
    new_files = {
        '01d_temporal_features_validation.png': 'Temporal Validation',
        '04c_class_imbalance_smote.png': 'SMOTE Analysis',
        '09f_vif_analysis.png': 'VIF Analysis',
        '09g_vif_results.csv': 'VIF Results CSV',
        '11a_tsne_multiperplexity.png': 't-SNE Multi-Perplexity',
        '12_feature_engineering_examples.png': 'Feature Engineering'
    }

    for filename, description in new_files.items():
        # PNG için IMAGES_DIR, CSV için CSV_DIR kontrol et
        if filename.endswith('.png'):
            filepath = os.path.join(IMAGES_DIR, filename)
        else:
            filepath = os.path.join(CSV_DIR, filename)

        exists = os.path.exists(filepath)
        symbol = "✅" if exists else "❌"
        print(f"  {symbol} {description:30s}: {filename}")

except Exception as e_files:
    print(f"\n[WARN] Dosya istatistikleri alınamadı: {e_files}")

print("\n" + "="*100)


gc.collect()
plt.close('all')
print("\n[INFO] Cleanup complete. EDA V5.1 Finished.")
print("\n" + "="*90)
print("🎉 THANK YOU FOR USING EDA V5.1!")
print("   ✨ New Features Added:")
print("      • Temporal Features Validation (Phase 1.6)")
print("      • SMOTE Class Imbalance Analysis (Phase 4.3)")
print("      • VIF Multicollinearity Detection (Phase 4.4)")
print("      • t-SNE Multi-Perplexity Comparison (Enhanced)")
print("      • Feature Engineering Examples (Phase 4.9)")
print("\n   Next steps: Review generated files and apply recommendations to your modeling pipeline.")
print("="*90)


[OK] imbalanced-learn (SMOTE) available.
[OK] statsmodels VIF available.
[OK] silhouette_score available.


[INFO] EDA Script V5.1 (COMPLETE REVISION) started at 2025-11-27 02:05:34
🚀 MAJOR UPDATES IN V5.1:
   ✅ FIXED: matches_all loading (Phase 1 added)
   ✅ ALL match-event features converted to temporal averages
   ✅ Data leakage completely eliminated
   ✅ High correlation pairs automatically handled
   ✅ SMOTE + VIF + t-SNE + Feature Engineering visualizations

✅ Thesis-compliant visualization settings loaded (V5.2)
   • Color palette: Mavi tonları (akademik standart)
   • Output format: PDF (vektörel) + PNG (preview)
   • DPI: 600 (yayın kalitesi)
[INFO] Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[OK] Drive mounted successfully
[OK] Output directories validated/created: /content/drive/My Drive/Thesis Data/EDA_Outputs_27.11.25 V1


[INFO] PHASE 1: LOADING AND PREPROCESSING DATA

[INFO] PHASE 1.0: FUZZY MATCHING & DATA INTEGRATION SETUP
[INFO] Initializing fuzzy matcher...
  📄 Loa